# breath detection all in on:

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import find_peaks, butter, filtfilt, iirnotch
from scipy.stats import skew, kurtosis
from scipy.interpolate import interp1d

# Global constant for minimum breath separation (in seconds)
DEFAULT_MIN_BREATH_SEC = 2.5
record = pt

#########################################
# SECTION 1: BREATH DETECTION FUNCTIONS
#########################################

def detect_edr_breaths(edr_time, edr_signal, min_breath_sec=DEFAULT_MIN_BREATH_SEC):
    """
    Detect inspiration and expiration breaths from the EDR signal.
    """
    min_distance_samples = int(10 * min_breath_sec)
    in_peaks, _ = find_peaks(-edr_signal, distance=min_distance_samples)
    ex_peaks, _ = find_peaks(edr_signal, distance=min_distance_samples)
    in_times = edr_time[in_peaks]
    ex_times = edr_time[ex_peaks]
    return in_times, ex_times

def auto_detect_edr_breaths(edr_time, edr_signal, init_min_breath_sec=2.5,
                            adjust_factor=0.5, max_iterations=5, tol=0.05,
                            min_allowed=0.5, max_allowed=10.0):
    """
    Automatically adapts min_breath_sec with multiple passes until stable.
    """
    current_min_breath_sec = init_min_breath_sec

    for iteration in range(max_iterations):
        in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
        all_breaths = np.sort(np.concatenate([in_times, ex_times]))
        if len(all_breaths) < 2:
            print(f"Not enough breath events (iteration {iteration+1}). Stopping.")
            return in_times, ex_times
        
        # Compute median interval (with outlier rejection)
        breath_intervals = np.diff(all_breaths)
        median_rough = np.median(breath_intervals)
        upper_cut = 2.0 * median_rough
        lower_cut = 0.2 * median_rough
        cleaned_intervals = breath_intervals[(breath_intervals <= upper_cut) & (breath_intervals >= lower_cut)]
        median_interval = np.median(cleaned_intervals) if len(cleaned_intervals) >= 2 else median_rough

        new_min_breath_sec = adjust_factor * median_interval
        new_min_breath_sec = np.clip(new_min_breath_sec, min_allowed, max_allowed)
        ratio = new_min_breath_sec / current_min_breath_sec
        print(f"Iteration {iteration+1}: old={current_min_breath_sec:.2f}, new={new_min_breath_sec:.2f} (ratio={ratio:.2f})")
        if abs(ratio - 1.0) < tol:
            current_min_breath_sec = new_min_breath_sec
            break
        else:
            current_min_breath_sec = new_min_breath_sec

    # Final detection pass
    in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
    print(f"Final min_breath_sec after {iteration+1} iteration(s): {current_min_breath_sec:.2f}")
    return in_times, ex_times

#########################################
# SECTION 2: FEATURE ANALYSIS FUNCTIONS
#########################################

# -- 2.1 EDR-derived Breathing Features --

def compute_mean_insp_exp_edr(in_times, ex_times, t_start, t_end):
    """
    Compute the mean inspiration and expiration intervals within a window.
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    valid_ex = ex_times[(ex_times >= t_start) & (ex_times <= t_end)]
    
    if len(valid_in) < 2:
        mean_insp_interval = np.nan
    else:
        mean_insp_interval = np.mean(np.diff(valid_in))
    
    if len(valid_ex) < 2:
        mean_exp_interval = np.nan
    else:
        mean_exp_interval = np.mean(np.diff(valid_ex))
        
    return mean_insp_interval, mean_exp_interval

def compute_breath_features(in_times, t_start, t_end):
    """
    Compute additional breath features within a window:
      - Number of breaths (count of valid inspirations)
      - Mean breath duration (average time between consecutive inspirations)
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    n_breaths = len(valid_in)
    if n_breaths >= 2:
        mean_breath_duration = np.mean(np.diff(valid_in))
    else:
        mean_breath_duration = np.nan
    return n_breaths, mean_breath_duration

def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Compute the breathing rate (BPM) from inspiration times within a window.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    mean_interval = np.mean(np.diff(valid))
    return 60.0 / mean_interval

# -- 2.2 ECG Heart Rate Analysis --

def compute_ecg_heart_rate(ecg_signal, fs):
    """
    Compute ECG heart rate from the filtered ECG signal.
    Requires preprocessing.detect_ecg_peaks to be defined.
    """
    R_peaks = preprocessing.detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        print("⚠ Not enough R-peaks to compute heart rate.")
        return np.array([]), np.array([]), np.nan

    rr_intervals = np.diff(R_peaks) / fs  
    inst_hr = 60.0 / rr_intervals 
    hr_time = 0.5 * (R_peaks[:-1] + R_peaks[1:]) / fs
    mean_hr = np.mean(inst_hr)
    return hr_time, inst_hr, mean_hr

def segment_ecg_hr(ecg_signal, fs, start_time, end_time, interval_size=30):
    """
    Segment ECG heart rate in intervals.
    """
    R_peaks = preprocessing.detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        return np.array([]), np.array([])

    seg_times = np.arange(start_time, end_time, interval_size)
    hr_values = []
    for t0 in seg_times:
        t1 = t0 + interval_size
        segment_peaks = R_peaks[(R_peaks/fs >= t0) & (R_peaks/fs < t1)]
        if len(segment_peaks) < 2:
            hr_values.append(np.nan)
        else:
            rr = np.diff(segment_peaks) / fs
            hr_values.append(60.0 / np.mean(rr))
    return seg_times, np.array(hr_values)

# -- 2.3 QRS Amplitude Statistics per Breath --

def compute_qrs_amp_per_breath(edr_time, edr_signal, in_times):
    """
    Compute QRS amplitude statistics for each breath cycle.
    Returns breath_times, mean QRS, standard deviation of QRS,
    and beat-to-beat variability (std of consecutive differences).
    """
    breath_times = []
    mean_qrs = []
    std_qrs = []
    btb_var = []
    for i in range(len(in_times) - 1):
        start = in_times[i]
        end = in_times[i+1]
        mask = (edr_time >= start) & (edr_time < end)
        segment = edr_signal[mask]
        breath_time = 0.5 * (start + end)
        if len(segment) < 2:
            breath_times.append(breath_time)
            mean_qrs.append(np.nan)
            std_qrs.append(np.nan)
            btb_var.append(np.nan)
        else:
            mean_val = np.mean(segment)
            std_val = np.std(segment)
            diffs = np.diff(segment)
            btb_val = np.std(diffs) if len(diffs) > 0 else np.nan
            breath_times.append(breath_time)
            mean_qrs.append(mean_val)
            std_qrs.append(std_val)
            btb_var.append(btb_val)
    return (np.array(breath_times),
            np.array(mean_qrs),
            np.array(std_qrs),
            np.array(btb_var))

# -- 2.4 Plotting Function for EDR Breath Detection --

def plot_edr_breaths(edr_time, edr_signal, in_times, ex_times):
    """
    Plot the EDR signal with detected inspiration and expiration points.
    """
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal (QRS Amplitude)')
    plt.plot(in_times, np.interp(in_times, edr_time, edr_signal), 'o', label='Inspirations')
    plt.plot(ex_times, np.interp(ex_times, edr_time, edr_signal), 'o', label='Expirations')
    plt.title('ECG-Derived Respiration with Detected Breaths')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

#########################################
# SECTION 3: FEATURE TABLE CONSTRUCTION
#########################################

def build_feature_table(patient, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=30):
    exercises = session[dt.Exercise]  # ✅ This is the correct way to access Exercise annotations



    all_ex_timestamps = exercises.timestamp
    full_start = float(np.min(all_ex_timestamps))
    full_end = float(np.max(all_ex_timestamps))
    print(f"build_feature_table: Using exercise interval {full_start:.1f}–{full_end:.1f} s")

    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]

    # (A) Breath detection on the EDR signal
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    # (B) Detect ECG R-peaks (ensure preprocessing.detect_ecg_peaks is defined)
    R_peaks = preprocessing.detect_ecg_peaks(ecg_signal, fs)

    feature_rows = []
    seg_times = np.arange(full_start, full_end, interval_size)
    
    # Pre-compute QRS amplitude per breath stats (used for BTB variability)
    btqrs_times, mean_qrs_breath, std_qrs_breath, btb_var_breath = compute_qrs_amp_per_breath(edr_time_qrs, edr_signal_qrs, in_times)
    
    for window_start in seg_times:
        window_end = window_start + interval_size

        # 1. EDR-derived breathing rate (BPM)
        edr_bpm = compute_bpm_from_inspiration_times(in_times, window_start, window_end)
        cycle_duration = 60.0 / edr_bpm if not np.isnan(edr_bpm) else np.nan

        # 2. Mean inspiration and expiration durations
        mean_insp, mean_exp = compute_mean_insp_exp_edr(in_times, ex_times, window_start, window_end)

        # 3. Breath count and mean breath duration (from inspirations)
        n_breaths, mean_breath_duration = compute_breath_features(in_times, window_start, window_end)

        # 4. ECG-derived heart rate (BPM) from R-peaks in the window
        window_peaks = R_peaks[(R_peaks/fs >= window_start) & (R_peaks/fs < window_end)]
        if len(window_peaks) < 2:
            ecg_bpm = np.nan
        else:
            rr = np.diff(window_peaks) / fs
            ecg_bpm = 60.0 / np.mean(rr)

        # 5. QRS amplitude statistics from the EDR signal in the window
        mask_qrs = (edr_time_qrs >= window_start) & (edr_time_qrs < window_end)
        qrs_window = edr_signal_qrs[mask_qrs]
        if len(qrs_window) < 1:
            qrs_mean = np.nan
            qrs_std = np.nan
        else:
            qrs_mean = np.mean(qrs_window)
            qrs_std = np.std(qrs_window)
        
        # 6. Beat-to-beat variability (BTB) of QRS amplitude for breaths in the window
        mask_btb = (btqrs_times >= window_start) & (btqrs_times < window_end)
        if np.any(mask_btb):
            qrs_btb = np.nanmean(btb_var_breath[mask_btb])
        else:
            qrs_btb = np.nan

        row = {
            "subject_id": patient.id,
            "window_start": window_start,
            "window_end": window_end,
            "EDR_BPM": edr_bpm,
            "BreathingCycleDuration": cycle_duration,
            "MeanInsp": mean_insp,
            "MeanExp": mean_exp,
            "n_breaths": n_breaths,
            "MeanBreathDuration": mean_breath_duration,
            "ECG_BPM": ecg_bpm,
            "QRS_mean": qrs_mean,
            "QRS_std": qrs_std,
            "QRS_BTB": qrs_btb
        }
        feature_rows.append(row)
    
    feature_df = pd.DataFrame(feature_rows)
    return feature_df

#########################################
# SECTION 4: MAIN EXECUTION & PLOTTING
#########################################

if __name__ == "__main__":
    # NOTE: Ensure that the following variables are defined in your environment:
    #   record, ecg_signal, filtered_ecg, fs, QRS_amplitude_resampled, preprocessing
    
    # --- ECG Heart Rate Analysis (for plotting) ---
    hr_time, inst_hr, mean_hr = compute_ecg_heart_rate(filtered_ecg, fs)
    print(f"Heart Rate from ECG: {mean_hr:.2f} BPM (average)")
    
    plt.figure(figsize=(8, 4))
    plt.plot(hr_time, inst_hr, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    session = pt[dt.SeatedSession]  # or SupineSession depending on your use case
    exercise_ann = session[dt.Exercise]

    seg_times_hr, hr_10s = segment_ecg_hr(
        filtered_ecg,
        fs,
        start_time=np.min(exercise_ann.timestamp),
        end_time=np.max(exercise_ann.timestamp),
        interval_size=30
        )

    print("Segmented HR (10s intervals):", hr_10s)
    
    plt.figure(figsize=(8, 4))
    plt.plot(seg_times_hr, hr_10s, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate (10s Intervals)')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # --- SECTION 1: EDR Breath Detection ---
    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]
    
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    
    interval_size = 30  # 10-second samples for segmentation here
    seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio = [], [], [], []
    seg_times = np.arange(np.min(exercise.timestamp), np.max(exercise.timestamp), interval_size)

    for t0 in seg_times:
        t1 = t0 + interval_size
        mi, me = compute_mean_insp_exp_edr(in_times, ex_times, t0, t1)
        seg_times_edr.append(t0)
        edr_mean_insp.append(mi)
        edr_mean_exp.append(me)
        ratio = mi/me if (not np.isnan(mi) and not np.isnan(me) and me > 0) else np.nan
        edr_ie_ratio.append(ratio)
    
    print("=== EDR-derived Mean Intervals (10s windows) ===")
    for t0, mi, me, ratio in zip(seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio):
        print(f"[{t0:.0f}-{t0+interval_size:.0f}s]: Insp={mi:.2f}s, Exp={me:.2f}s, I:E={ratio:.2f}")
    
    plot_edr_breaths(edr_time_qrs, edr_signal_qrs, in_times, ex_times)
    
    # --- Build Feature Table ---
    feature_df = build_feature_table(record, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=30)
    print("Feature Table:")
    print(feature_df)
    
    # 2) Create a label for each time window (e.g., "0–30s", "30–60s", etc.)
    labels = [
        f"{int(ws)}–{int(we)}s"
        for ws, we in zip(feature_df["window_start"], feature_df["window_end"])
    ]

    
    # --- Additional Plots for Feature Analysis ---
    # Calculate mid-time for each window from the feature table (using window length 30s)
    mid_times = feature_df['window_start'] + (30 / 2)

    # Plot: Mean Inspiration and Expiration Durations
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['MeanInsp'], marker='o', linestyle='-', label='Mean Inspiration Duration')
    plt.plot(mid_times, feature_df['MeanExp'], marker='o', linestyle='-', label='Mean Expiration Duration')
    plt.title('Mean Inspiration and Expiration Durations')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot: Number of Breaths per Window
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['n_breaths'], marker='o', linestyle='-', label='Number of Breaths')
    plt.title('Number of Breaths (per 10s window)')
    plt.xlabel('Time (s)')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot: Mean Breath Duration vs. Breathing Cycle Duration
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['MeanBreathDuration'], marker='o', linestyle='-', label='Mean Breath Duration')
    plt.title('Mean Breath Duration')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # Plot: Breathing Rate per Minute
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['EDR_BPM'], marker='o', linestyle='-', label='Breathing Rate (BPM)')
    plt.title('Breathing Rate per Minute')
    plt.xlabel('Time (s)')
    plt.ylabel('BPM')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 3) Bar Plot: Breathing Rate per Minute (EDR_BPM)
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["EDR_BPM"], color="royalblue", alpha=0.7)
    plt.title("Breathing Rate per Minute (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("BPM")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

    # 4) Bar Plot: Mean Breath Duration
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["MeanBreathDuration"], color="seagreen", alpha=0.7)
    plt.title("Mean Breath Duration (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("Duration (s)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()
    
    # --- Additional Plots for QRS Amplitude Statistics ---
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_mean'], marker='o', linestyle='-', label='Mean QRS per Breath')
    plt.title('Mean QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Mean QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_std'], marker='o', linestyle='-', color='orange', label='Std of QRS per Breath')
    plt.title('Std of QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Std QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_BTB'], marker='o', linestyle='-', color='green', label='Beat-to-beat Variability')
    plt.title('Beat-to-beat Variability of QRS Amplitude')
    plt.xlabel('Time (s)')
    plt.ylabel('Std of Consecutive Differences')
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import find_peaks, butter, filtfilt, iirnotch
from scipy.stats import skew, kurtosis
from scipy.interpolate import interp1d

# Global constant for minimum breath separation (in seconds)
DEFAULT_MIN_BREATH_SEC = 2.5
record = pt

#########################################
# SECTION 1: BREATH DETECTION FUNCTIONS
#########################################

def detect_edr_breaths(edr_time, edr_signal, min_breath_sec=DEFAULT_MIN_BREATH_SEC):
    """
    Detect inspiration and expiration breaths from the EDR signal.
    """
    min_distance_samples = int(10 * min_breath_sec)
    in_peaks, _ = find_peaks(-edr_signal, distance=min_distance_samples)
    ex_peaks, _ = find_peaks(edr_signal, distance=min_distance_samples)
    in_times = edr_time[in_peaks]
    ex_times = edr_time[ex_peaks]
    return in_times, ex_times

def auto_detect_edr_breaths(edr_time, edr_signal, init_min_breath_sec=2.5,
                            adjust_factor=0.5, max_iterations=5, tol=0.05,
                            min_allowed=0.5, max_allowed=10.0):
    """
    Automatically adapts min_breath_sec with multiple passes until stable.
    """
    current_min_breath_sec = init_min_breath_sec

    for iteration in range(max_iterations):
        in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
        all_breaths = np.sort(np.concatenate([in_times, ex_times]))
        if len(all_breaths) < 2:
            print(f"Not enough breath events (iteration {iteration+1}). Stopping.")
            return in_times, ex_times
        
        # Compute median interval (with outlier rejection)
        breath_intervals = np.diff(all_breaths)
        median_rough = np.median(breath_intervals)
        upper_cut = 2.0 * median_rough
        lower_cut = 0.2 * median_rough
        cleaned_intervals = breath_intervals[(breath_intervals <= upper_cut) & (breath_intervals >= lower_cut)]
        median_interval = np.median(cleaned_intervals) if len(cleaned_intervals) >= 2 else median_rough

        new_min_breath_sec = adjust_factor * median_interval
        new_min_breath_sec = np.clip(new_min_breath_sec, min_allowed, max_allowed)
        ratio = new_min_breath_sec / current_min_breath_sec
        print(f"Iteration {iteration+1}: old={current_min_breath_sec:.2f}, new={new_min_breath_sec:.2f} (ratio={ratio:.2f})")
        if abs(ratio - 1.0) < tol:
            current_min_breath_sec = new_min_breath_sec
            break
        else:
            current_min_breath_sec = new_min_breath_sec

    # Final detection pass
    in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
    print(f"Final min_breath_sec after {iteration+1} iteration(s): {current_min_breath_sec:.2f}")
    return in_times, ex_times

#########################################
# SECTION 2: FEATURE ANALYSIS FUNCTIONS
#########################################

# -- 2.1 EDR-derived Breathing Features --

def compute_mean_insp_exp_edr(in_times, ex_times, t_start, t_end):
    """
    Compute the mean inspiration and expiration intervals within a window.
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    valid_ex = ex_times[(ex_times >= t_start) & (ex_times <= t_end)]
    
    if len(valid_in) < 2:
        mean_insp_interval = np.nan
    else:
        mean_insp_interval = np.mean(np.diff(valid_in))
    
    if len(valid_ex) < 2:
        mean_exp_interval = np.nan
    else:
        mean_exp_interval = np.mean(np.diff(valid_ex))
        
    return mean_insp_interval, mean_exp_interval

def compute_breath_features(in_times, t_start, t_end):
    """
    Compute additional breath features within a window:
      - Number of breaths (count of valid inspirations)
      - Mean breath duration (average time between consecutive inspirations)
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    n_breaths = len(valid_in)
    if n_breaths >= 2:
        mean_breath_duration = np.mean(np.diff(valid_in))
    else:
        mean_breath_duration = np.nan
    return n_breaths, mean_breath_duration

def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Compute the breathing rate (BPM) from inspiration times within a window.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    mean_interval = np.mean(np.diff(valid))
    return 60.0 / mean_interval

# -- 2.2 ECG Heart Rate Analysis --

def compute_ecg_heart_rate(ecg_signal, fs):
    """
    Compute ECG heart rate from the filtered ECG signal.
    Requires preprocessing.detect_ecg_peaks to be defined.
    """
    R_peaks = preprocessing.detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        print("⚠ Not enough R-peaks to compute heart rate.")
        return np.array([]), np.array([]), np.nan

    rr_intervals = np.diff(R_peaks) / fs  
    inst_hr = 60.0 / rr_intervals 
    hr_time = 0.5 * (R_peaks[:-1] + R_peaks[1:]) / fs
    mean_hr = np.mean(inst_hr)
    return hr_time, inst_hr, mean_hr

def segment_ecg_hr(ecg_signal, fs, start_time, end_time, interval_size=30):
    """
    Segment ECG heart rate in intervals.
    """
    R_peaks = preprocessing.detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        return np.array([]), np.array([])

    seg_times = np.arange(start_time, end_time, interval_size)
    hr_values = []
    for t0 in seg_times:
        t1 = t0 + interval_size
        segment_peaks = R_peaks[(R_peaks/fs >= t0) & (R_peaks/fs < t1)]
        if len(segment_peaks) < 2:
            hr_values.append(np.nan)
        else:
            rr = np.diff(segment_peaks) / fs
            hr_values.append(60.0 / np.mean(rr))
    return seg_times, np.array(hr_values)

# -- 2.3 QRS Amplitude Statistics per Breath --

def compute_qrs_amp_per_breath(edr_time, edr_signal, in_times):
    """
    Compute QRS amplitude statistics for each breath cycle.
    Returns breath_times, mean QRS, standard deviation of QRS,
    and beat-to-beat variability (std of consecutive differences).
    """
    breath_times = []
    mean_qrs = []
    std_qrs = []
    btb_var = []
    for i in range(len(in_times) - 1):
        start = in_times[i]
        end = in_times[i+1]
        mask = (edr_time >= start) & (edr_time < end)
        segment = edr_signal[mask]
        breath_time = 0.5 * (start + end)
        if len(segment) < 2:
            breath_times.append(breath_time)
            mean_qrs.append(np.nan)
            std_qrs.append(np.nan)
            btb_var.append(np.nan)
        else:
            mean_val = np.mean(segment)
            std_val = np.std(segment)
            diffs = np.diff(segment)
            btb_val = np.std(diffs) if len(diffs) > 0 else np.nan
            breath_times.append(breath_time)
            mean_qrs.append(mean_val)
            std_qrs.append(std_val)
            btb_var.append(btb_val)
    return (np.array(breath_times),
            np.array(mean_qrs),
            np.array(std_qrs),
            np.array(btb_var))

# -- 2.4 Plotting Function for EDR Breath Detection --

def plot_edr_breaths(edr_time, edr_signal, in_times, ex_times):
    """
    Plot the EDR signal with detected inspiration and expiration points.
    """
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal (QRS Amplitude)')
    plt.plot(in_times, np.interp(in_times, edr_time, edr_signal), 'o', label='Inspirations')
    plt.plot(ex_times, np.interp(ex_times, edr_time, edr_signal), 'o', label='Expirations')
    plt.title('ECG-Derived Respiration with Detected Breaths')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

#########################################
# SECTION 3: FEATURE TABLE CONSTRUCTION
#########################################

def build_feature_table(patient, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=30):
    exercises = session[dt.Exercise]  # ✅ This is the correct way to access Exercise annotations



    all_ex_timestamps = exercises.timestamp
    full_start = float(np.min(all_ex_timestamps))
    full_end = float(np.max(all_ex_timestamps))
    print(f"build_feature_table: Using exercise interval {full_start:.1f}–{full_end:.1f} s")

    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]

    # (A) Breath detection on the EDR signal
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    # (B) Detect ECG R-peaks (ensure preprocessing.detect_ecg_peaks is defined)
    R_peaks = preprocessing.detect_ecg_peaks(ecg_signal, fs)

    feature_rows = []
    seg_times = np.arange(full_start, full_end, interval_size)
    
    # Pre-compute QRS amplitude per breath stats (used for BTB variability)
    btqrs_times, mean_qrs_breath, std_qrs_breath, btb_var_breath = compute_qrs_amp_per_breath(edr_time_qrs, edr_signal_qrs, in_times)
    
    for window_start in seg_times:
        window_end = window_start + interval_size

        # 1. EDR-derived breathing rate (BPM)
        edr_bpm = compute_bpm_from_inspiration_times(in_times, window_start, window_end)
        cycle_duration = 60.0 / edr_bpm if not np.isnan(edr_bpm) else np.nan

        # 2. Mean inspiration and expiration durations
        mean_insp, mean_exp = compute_mean_insp_exp_edr(in_times, ex_times, window_start, window_end)

        # 3. Breath count and mean breath duration (from inspirations)
        n_breaths, mean_breath_duration = compute_breath_features(in_times, window_start, window_end)

        # 4. ECG-derived heart rate (BPM) from R-peaks in the window
        window_peaks = R_peaks[(R_peaks/fs >= window_start) & (R_peaks/fs < window_end)]
        if len(window_peaks) < 2:
            ecg_bpm = np.nan
        else:
            rr = np.diff(window_peaks) / fs
            ecg_bpm = 60.0 / np.mean(rr)

        # 5. QRS amplitude statistics from the EDR signal in the window
        mask_qrs = (edr_time_qrs >= window_start) & (edr_time_qrs < window_end)
        qrs_window = edr_signal_qrs[mask_qrs]
        if len(qrs_window) < 1:
            qrs_mean = np.nan
            qrs_std = np.nan
        else:
            qrs_mean = np.mean(qrs_window)
            qrs_std = np.std(qrs_window)
        
        # 6. Beat-to-beat variability (BTB) of QRS amplitude for breaths in the window
        mask_btb = (btqrs_times >= window_start) & (btqrs_times < window_end)
        if np.any(mask_btb):
            qrs_btb = np.nanmean(btb_var_breath[mask_btb])
        else:
            qrs_btb = np.nan

        row = {
            "subject_id": patient.id,
            "window_start": window_start,
            "window_end": window_end,
            "EDR_BPM": edr_bpm,
            "BreathingCycleDuration": cycle_duration,
            "MeanInsp": mean_insp,
            "MeanExp": mean_exp,
            "n_breaths": n_breaths,
            "MeanBreathDuration": mean_breath_duration,
            "ECG_BPM": ecg_bpm,
            "QRS_mean": qrs_mean,
            "QRS_std": qrs_std,
            "QRS_BTB": qrs_btb
        }
        feature_rows.append(row)
    
    feature_df = pd.DataFrame(feature_rows)
    return feature_df

#########################################
# SECTION 4: MAIN EXECUTION & PLOTTING
#########################################

if __name__ == "__main__":
    # NOTE: Ensure that the following variables are defined in your environment:
    #   record, ecg_signal, filtered_ecg, fs, QRS_amplitude_resampled, preprocessing
    
    # --- ECG Heart Rate Analysis (for plotting) ---
    hr_time, inst_hr, mean_hr = compute_ecg_heart_rate(filtered_ecg, fs)
    print(f"Heart Rate from ECG: {mean_hr:.2f} BPM (average)")
    
    plt.figure(figsize=(8, 4))
    plt.plot(hr_time, inst_hr, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    session = pt[dt.SeatedSession]  # or SupineSession depending on your use case
    exercise_ann = session[dt.Exercise]

    seg_times_hr, hr_10s = segment_ecg_hr(
        filtered_ecg,
        fs,
        start_time=np.min(exercise_ann.timestamp),
        end_time=np.max(exercise_ann.timestamp),
        interval_size=30
        )

    print("Segmented HR (10s intervals):", hr_10s)
    
    plt.figure(figsize=(8, 4))
    plt.plot(seg_times_hr, hr_10s, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate (10s Intervals)')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # --- SECTION 1: EDR Breath Detection ---
    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]
    
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    
    interval_size = 30  # 10-second samples for segmentation here
    seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio = [], [], [], []
    seg_times = np.arange(np.min(exercise.timestamp), np.max(exercise.timestamp), interval_size)

    for t0 in seg_times:
        t1 = t0 + interval_size
        mi, me = compute_mean_insp_exp_edr(in_times, ex_times, t0, t1)
        seg_times_edr.append(t0)
        edr_mean_insp.append(mi)
        edr_mean_exp.append(me)
        ratio = mi/me if (not np.isnan(mi) and not np.isnan(me) and me > 0) else np.nan
        edr_ie_ratio.append(ratio)
    
    print("=== EDR-derived Mean Intervals (10s windows) ===")
    for t0, mi, me, ratio in zip(seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio):
        print(f"[{t0:.0f}-{t0+interval_size:.0f}s]: Insp={mi:.2f}s, Exp={me:.2f}s, I:E={ratio:.2f}")
    
    plot_edr_breaths(edr_time_qrs, edr_signal_qrs, in_times, ex_times)
    
    # --- Build Feature Table ---
    feature_df = build_feature_table(record, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=30)
    print("Feature Table:")
    print(feature_df)
    
    # 2) Create a label for each time window (e.g., "0–30s", "30–60s", etc.)
    labels = [
        f"{int(ws)}–{int(we)}s"
        for ws, we in zip(feature_df["window_start"], feature_df["window_end"])
    ]

    
    # --- Additional Plots for Feature Analysis ---
    # Calculate mid-time for each window from the feature table (using window length 30s)
    mid_times = feature_df['window_start'] + (30 / 2)

    # Plot: Mean Inspiration and Expiration Durations
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['MeanInsp'], marker='o', linestyle='-', label='Mean Inspiration Duration')
    plt.plot(mid_times, feature_df['MeanExp'], marker='o', linestyle='-', label='Mean Expiration Duration')
    plt.title('Mean Inspiration and Expiration Durations')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot: Number of Breaths per Window
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['n_breaths'], marker='o', linestyle='-', label='Number of Breaths')
    plt.title('Number of Breaths (per 10s window)')
    plt.xlabel('Time (s)')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot: Mean Breath Duration vs. Breathing Cycle Duration
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['MeanBreathDuration'], marker='o', linestyle='-', label='Mean Breath Duration')
    plt.title('Mean Breath Duration')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # Plot: Breathing Rate per Minute
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['EDR_BPM'], marker='o', linestyle='-', label='Breathing Rate (BPM)')
    plt.title('Breathing Rate per Minute')
    plt.xlabel('Time (s)')
    plt.ylabel('BPM')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 3) Bar Plot: Breathing Rate per Minute (EDR_BPM)
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["EDR_BPM"], color="royalblue", alpha=0.7)
    plt.title("Breathing Rate per Minute (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("BPM")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

    # 4) Bar Plot: Mean Breath Duration
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["MeanBreathDuration"], color="seagreen", alpha=0.7)
    plt.title("Mean Breath Duration (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("Duration (s)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()
    
    # --- Additional Plots for QRS Amplitude Statistics ---
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_mean'], marker='o', linestyle='-', label='Mean QRS per Breath')
    plt.title('Mean QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Mean QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_std'], marker='o', linestyle='-', color='orange', label='Std of QRS per Breath')
    plt.title('Std of QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Std QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_BTB'], marker='o', linestyle='-', color='green', label='Beat-to-beat Variability')
    plt.title('Beat-to-beat Variability of QRS Amplitude')
    plt.xlabel('Time (s)')
    plt.ylabel('Std of Consecutive Differences')
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
if __name__ == "__main__":
    # NOTE: Ensure that the following variables are defined in your environment:
    #   record, ecg_signal, filtered_ecg, fs, QRS_amplitude_resampled, preprocessing
    
    # --- ECG Heart Rate Analysis (for plotting) ---
    hr_time, inst_hr, mean_hr = compute_ecg_heart_rate(filtered_ecg, fs)
    print(f"Heart Rate from ECG: {mean_hr:.2f} BPM (average)")
    
    # Plot 1: ECG-Derived Heart Rate
    fig_hr = go.Figure()
    fig_hr.add_trace(go.Scatter(x=hr_time, y=inst_hr, mode='markers+lines', name='Instantaneous HR'))
    fig_hr.update_layout(title='ECG-Derived Heart Rate',
                         xaxis_title='Time (s)',
                         yaxis_title='Heart Rate (BPM)',
                         template='plotly_white')
    fig_hr.show()
    
    # --- Segment ECG Heart Rate (10s intervals) ---
    session = pt[dt.SeatedSession]  # or SupineSession depending on your use case
    exercise_ann = session[dt.Exercise]
    
    seg_times_hr, hr_10s = segment_ecg_hr(
        filtered_ecg,
        fs,
        start_time=np.min(exercise_ann.timestamp),
        end_time=np.max(exercise_ann.timestamp),
        interval_size=30
    )
    print("Segmented HR (10s intervals):", hr_10s)
    
    fig_seg_hr = go.Figure()
    fig_seg_hr.add_trace(go.Scatter(x=seg_times_hr, y=hr_10s, mode='markers+lines', name='Instantaneous HR'))
    fig_seg_hr.update_layout(title='ECG-Derived Heart Rate (10s Intervals)',
                             xaxis_title='Time (s)',
                             yaxis_title='Heart Rate (BPM)',
                             template='plotly_white')
    fig_seg_hr.show()
    
    # --- SECTION 1: EDR Breath Detection ---
    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]
    
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    
    interval_size = 30  # 30-second intervals for segmentation here
    seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio = [], [], [], []
    seg_times = np.arange(np.min(exercise_ann.timestamp), np.max(exercise_ann.timestamp), interval_size)

    for t0 in seg_times:
        t1 = t0 + interval_size
        mi, me = compute_mean_insp_exp_edr(in_times, ex_times, t0, t1)
        seg_times_edr.append(t0)
        edr_mean_insp.append(mi)
        edr_mean_exp.append(me)
        ratio = mi/me if (not np.isnan(mi) and not np.isnan(me) and me > 0) else np.nan
        edr_ie_ratio.append(ratio)
    
    print("=== EDR-derived Mean Intervals (30s windows) ===")
    for t0, mi, me, ratio in zip(seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio):
        print(f"[{t0:.0f}-{t0+interval_size:.0f}s]: Insp={mi:.2f}s, Exp={me:.2f}s, I:E={ratio:.2f}")
    
    # Plot EDR Breath Detection using Plotly
    plot_edr_breaths_plotly(edr_time_qrs, edr_signal_qrs, in_times, ex_times)
    
    # --- Build Feature Table ---
    feature_df = build_feature_table(record, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=30)
    print("Feature Table:")
    print(feature_df)
    
    # 2) Create a label for each time window (e.g., "0–30s", "30–60s", etc.)
    labels = [
        f"{int(ws)}–{int(we)}s"
        for ws, we in zip(feature_df["window_start"], feature_df["window_end"])
    ]
    
    # Calculate mid-time for each window (window length assumed 30s)
    mid_times = feature_df['window_start'] + (30 / 2)

    # --- Additional Plots for Feature Analysis ---
    # Plot: Mean Inspiration and Expiration Durations
    fig_insp_exp = go.Figure()
    fig_insp_exp.add_trace(go.Scatter(x=mid_times, y=feature_df['MeanInsp'], mode='markers+lines', name='Mean Inspiration Duration'))
    fig_insp_exp.add_trace(go.Scatter(x=mid_times, y=feature_df['MeanExp'], mode='markers+lines', name='Mean Expiration Duration'))
    fig_insp_exp.update_layout(title='Mean Inspiration and Expiration Durations',
                               xaxis_title='Time (s)',
                               yaxis_title='Duration (s)',
                               template='plotly_white')
    fig_insp_exp.show()

    # Plot: Number of Breaths per Window
    fig_n_breaths = go.Figure()
    fig_n_breaths.add_trace(go.Scatter(x=mid_times, y=feature_df['n_breaths'], mode='markers+lines', name='Number of Breaths'))
    fig_n_breaths.update_layout(title='Number of Breaths (per 30s window)',
                                xaxis_title='Time (s)',
                                yaxis_title='Count',
                                template='plotly_white')
    fig_n_breaths.show()

    # Plot: Mean Breath Duration
    fig_mean_breath_duration = go.Figure()
    fig_mean_breath_duration.add_trace(go.Scatter(x=mid_times, y=feature_df['MeanBreathDuration'], mode='markers+lines', name='Mean Breath Duration'))
    fig_mean_breath_duration.update_layout(title='Mean Breath Duration',
                                           xaxis_title='Time (s)',
                                           yaxis_title='Duration (s)',
                                           template='plotly_white')
    fig_mean_breath_duration.show()
    
    # Plot: Breathing Rate per Minute
    fig_breathing_rate = go.Figure()
    fig_breathing_rate.add_trace(go.Scatter(x=mid_times, y=feature_df['EDR_BPM'], mode='markers+lines', name='Breathing Rate (BPM)'))
    fig_breathing_rate.update_layout(title='Breathing Rate per Minute',
                                     xaxis_title='Time (s)',
                                     yaxis_title='BPM',
                                     template='plotly_white')
    fig_breathing_rate.show()

    # Bar Plot: Breathing Rate per Minute (EDR_BPM)
    fig_bar_edr = go.Figure()
    fig_bar_edr.add_trace(go.Bar(x=labels, y=feature_df["EDR_BPM"], name='Breathing Rate (BPM)', marker_color='royalblue'))
    fig_bar_edr.update_layout(title='Breathing Rate per Minute (Segmented Intervals)',
                              xaxis_title='Time Intervals',
                              yaxis_title='BPM',
                              template='plotly_white')
    fig_bar_edr.show()

    # Bar Plot: Mean Breath Duration
    fig_bar_breath_duration = go.Figure()
    fig_bar_breath_duration.add_trace(go.Bar(x=labels, y=feature_df["MeanBreathDuration"], name='Mean Breath Duration', marker_color='seagreen'))
    fig_bar_breath_duration.update_layout(title='Mean Breath Duration (Segmented Intervals)',
                                          xaxis_title='Time Intervals',
                                          yaxis_title='Duration (s)',
                                          template='plotly_white')
    fig_bar_breath_duration.show()
    
    # --- Additional Plots for QRS Amplitude Statistics ---
    fig_qrs_mean = go.Figure()
    fig_qrs_mean.add_trace(go.Scatter(x=mid_times, y=feature_df['QRS_mean'], mode='markers+lines', name='Mean QRS per Breath'))
    fig_qrs_mean.update_layout(title='Mean QRS Amplitude per Breath',
                               xaxis_title='Time (s)',
                               yaxis_title='Mean QRS Amplitude',
                               template='plotly_white')
    fig_qrs_mean.show()

    fig_qrs_std = go.Figure()
    fig_qrs_std.add_trace(go.Scatter(x=mid_times, y=feature_df['QRS_std'], mode='markers+lines', name='Std of QRS per Breath',
                                     line=dict(color='orange')))
    fig_qrs_std.update_layout(title='Std of QRS Amplitude per Breath',
                              xaxis_title='Time (s)',
                              yaxis_title='Std QRS Amplitude',
                              template='plotly_white')
    fig_qrs_std.show()

    fig_qrs_btb = go.Figure()
    fig_qrs_btb.add_trace(go.Scatter(x=mid_times, y=feature_df['QRS_BTB'], mode='markers+lines', name='Beat-to-beat Variability',
                                     line=dict(color='green')))
    fig_qrs_btb.update_layout(title='Beat-to-beat Variability of QRS Amplitude',
                              xaxis_title='Time (s)',
                              yaxis_title='Std of Consecutive Differences',
                              template='plotly_white')
    fig_qrs_btb.show()


# maybe some bland altmaplots wprksking:

In [ ]:
##############################
#       IMPORTS
##############################
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import find_peaks, butter, filtfilt, iirnotch
from scipy.stats import skew, kurtosis
from scipy.interpolate import interp1d
record = pt
# Global constant for minimum breath separation (in seconds)
DEFAULT_MIN_BREATH_SEC = 2.5

def detect_edr_breaths(edr_time, edr_signal, min_breath_sec=DEFAULT_MIN_BREATH_SEC):
    """
    Detect inspiration and expiration breaths from the EDR signal.
    """
    min_distance_samples = int(10 * min_breath_sec)
    in_peaks, _ = find_peaks(-edr_signal, distance=min_distance_samples)
    ex_peaks, _ = find_peaks(edr_signal, distance=min_distance_samples)
    in_times = edr_time[in_peaks]
    ex_times = edr_time[ex_peaks]
    return in_times, ex_times

def auto_detect_edr_breaths(edr_time, edr_signal, init_min_breath_sec=2.5,
                            adjust_factor=0.5, max_iterations=5, tol=0.05,
                            min_allowed=0.5, max_allowed=10.0):
    """
    Automatically adapts the minimum breath separation time (min_breath_sec) 
    using multiple passes until stable.
    """
    current_min_breath_sec = init_min_breath_sec

    for iteration in range(max_iterations):
        in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
        all_breaths = np.sort(np.concatenate([in_times, ex_times]))
        if len(all_breaths) < 2:
            print(f"Not enough breath events (iteration {iteration+1}). Stopping.")
            return in_times, ex_times
        
        # Compute median interval with outlier rejection
        breath_intervals = np.diff(all_breaths)
        median_rough = np.median(breath_intervals)
        upper_cut = 2.0 * median_rough
        lower_cut = 0.2 * median_rough
        cleaned_intervals = breath_intervals[(breath_intervals <= upper_cut) & (breath_intervals >= lower_cut)]
        median_interval = np.median(cleaned_intervals) if len(cleaned_intervals) >= 2 else median_rough

        new_min_breath_sec = adjust_factor * median_interval
        new_min_breath_sec = np.clip(new_min_breath_sec, min_allowed, max_allowed)
        ratio = new_min_breath_sec / current_min_breath_sec
        print(f"Iteration {iteration+1}: old={current_min_breath_sec:.2f}, new={new_min_breath_sec:.2f} (ratio={ratio:.2f})")
        if abs(ratio - 1.0) < tol:
            current_min_breath_sec = new_min_breath_sec
            break
        else:
            current_min_breath_sec = new_min_breath_sec

    # Final detection pass
    in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
    print(f"Final min_breath_sec after {iteration+1} iteration(s): {current_min_breath_sec:.2f}")
    return in_times, ex_times



def compute_mean_insp_exp_edr(in_times, ex_times, t_start, t_end):
    """
    Computes mean intervals (in seconds) between consecutive inspirations
    and expirations over [t_start, t_end].
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    valid_ex = ex_times[(ex_times >= t_start) & (ex_times <= t_end)]

    if len(valid_in) < 2:
        mean_insp_interval = np.nan
    else:
        intervals_in = np.diff(valid_in)
        mean_insp_interval = np.mean(intervals_in)

    if len(valid_ex) < 2:
        mean_exp_interval = np.nan
    else:
        intervals_ex = np.diff(valid_ex)
        mean_exp_interval = np.mean(intervals_ex)

    return mean_insp_interval, mean_exp_interval


def segment_mean_insp_exp_edr(in_times, ex_times, start_time, end_time, interval_size):
    """
    Segments [start_time, end_time] into intervals of length 'interval_size'
    and computes mean inspiration/expiration intervals in each segment.
    """
    seg_times = np.arange(start_time, end_time, interval_size)
    mean_insp_vals = []
    mean_exp_vals  = []
    ie_ratios      = []

    for t0 in seg_times:
        t1 = t0 + interval_size
        mi, me = compute_mean_insp_exp_edr(in_times, ex_times, t0, t1)
        mean_insp_vals.append(mi)
        mean_exp_vals.append(me)
        ratio = np.nan
        if not np.isnan(mi) and not np.isnan(me) and me > 0:
            ratio = mi / me
        ie_ratios.append(ratio)

    return seg_times, np.array(mean_insp_vals), np.array(mean_exp_vals), np.array(ie_ratios)


def compute_gs_mean_intervals(record, start_time, end_time, interval_size):
    """
    Computes golden standard mean inspiration and expiration intervals
    from record annotations over segmented intervals.
    """


    breaths = session[dt.Breaths]
    gs_in_times = np.array([breaths.timestamp[k] for k, label in enumerate(breaths.ann) if label == '(In'])
    gs_ex_times = np.array([breaths.timestamp[k] for k, label in enumerate(breaths.ann) if label == '(Ex'])
    return segment_mean_insp_exp_edr(gs_in_times, gs_ex_times, start_time, end_time, interval_size)


##############################
#  SECTION 3: PAIRING & BLAND–ALTMAN ANALYSIS
##############################

def pair_events_for_bland_altman(golden_times, edr_times, max_tolerance=2.0):
    """
    Pairs each EDR event with the closest golden standard event (if within max_tolerance).
    Returns matched arrays (golden, edr).
    """
    matched_golden = []
    matched_edr    = []
    for edr_t in edr_times:
        if len(golden_times) == 0:
            continue
        abs_diffs = np.abs(golden_times - edr_t)
        min_idx   = np.argmin(abs_diffs)
        min_diff  = abs_diffs[min_idx]
        if min_diff <= max_tolerance:
            matched_golden.append(golden_times[min_idx])
            matched_edr.append(edr_t)
    return np.array(matched_golden), np.array(matched_edr)


def bland_altman_plot(golden_vals, edr_vals, title="", ax=None,
                      x_label="Average of times (s)",
                      y_label="EDR time - Golden time (s)"):
    """
    Creates a Bland–Altman plot comparing EDR events to Golden Standard events.
    Plots difference (EDR - Golden) vs. average.
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 5))
    difference = edr_vals - golden_vals
    average    = 0.5 * (edr_vals + golden_vals)
    mean_diff = np.mean(difference)
    sd_diff   = np.std(difference)
    upper_loa = mean_diff + 1.96 * sd_diff
    lower_loa = mean_diff - 1.96 * sd_diff
    ax.scatter(average, difference, alpha=0.7, edgecolor='k')
    ax.axhline(mean_diff,  color='red',   linestyle='--', label=f'Mean diff = {mean_diff:.3f}')
    ax.axhline(upper_loa,  color='green', linestyle='--', label=f'Upper LoA = {upper_loa:.2f}')
    ax.axhline(lower_loa,  color='green', linestyle='--', label=f'Lower LoA = {lower_loa:.2f}')
    ax.set_title(title)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.legend()
    return ax


##############################
#  SECTION 4: ADDITIONAL BPM FUNCTIONS (EDR vs. GOLD)
##############################

def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Computes breaths per minute (BPM) using detected inspiration events within [t_start, t_end].
    Returns NaN if fewer than two events.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    intervals = np.diff(valid)
    mean_interval = np.mean(intervals)
    return 60.0 / mean_interval


def segment_bpm_edr(record, edr_time, edr_signal,
                    start_time, end_time, interval_size=10,
                    min_breath_sec=DEFAULT_MIN_BREATH_SEC):
    """
    Segments the EDR signal (already 10 Hz) into intervals & computes BPM from EDR inspirations.
    Also computes golden standard BPM for each interval.
    """
    # Detect EDR inspirations & expirations with minimal separation
    edr_in_times, _ = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=min_breath_sec)

    def compute_resp_rate_in_interval(record, t_start, t_end):
        """
        Sub-function to compute BPM from golden standard inspirations in [t_start, t_end].
        """
        if not record or not hasattr(record, 'session'):
            return np.nan
        breaths = record.session[dt.Breaths]
        gs_in_times = np.array([t for label, t in zip(breaths.ann, breaths.timestamp) if label=='(In'])
        valid = gs_in_times[(gs_in_times >= t_start) & (gs_in_times <= t_end)]
        if valid.size < 2:
            return np.nan
        intervals = np.diff(valid)
        return 60.0 / np.mean(intervals)

    seg_times = np.arange(start_time, end_time, interval_size)
    edr_bpms  = []
    gs_bpms   = []
    for t in seg_times:
        t_end = t + interval_size
        edr_bpm = compute_bpm_from_inspiration_times(edr_in_times, t, t_end)
        gs_bpm  = compute_resp_rate_in_interval(record, t, t_end)
        edr_bpms.append(edr_bpm)
        gs_bpms.append(gs_bpm)

    return seg_times, np.array(edr_bpms), np.array(gs_bpms)



##############################
#  SECTION 6: MAIN SCRIPT
##############################
breaths = session[dt.Breaths]       # Returns Breaths annotation object
exercise = session[dt.Exercise]     # Returns Exercise annotation object

print(breaths.timestamp)
print(breaths.ann)
print(exercise.timestamp)
print(exercise.ann)

if __name__ == "__main__":

    
    exercises = [dt.Exercise] 
    all_ex_timestamps = exercise.timestamp
    full_start = np.min(all_ex_timestamps)
    full_end   = np.max(all_ex_timestamps)
    print(f"Using exercise interval from {full_start:.1f} to {full_end:.1f} (based on 'EXERCISE' annotation).")
   

    edr_time_qrs   = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]



    # --- SECTION 1: EDR Breath Detection ---
    # Use the adaptive detection so that a single min_breath_sec value is used everywhere.
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs, init_min_breath_sec=DEFAULT_MIN_BREATH_SEC, adjust_factor=0.6)

    # We'll do 30s segments for intervals
    interval_size = 10
    seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio = segment_mean_insp_exp_edr(
        in_times, ex_times,
        start_time=full_start,
        end_time=full_end,
        interval_size=interval_size
    )


    # Plot with golden standard annotations
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(edr_time_qrs, edr_signal_qrs, label='EDR Signal')
    ax.set_title('EDR Signal with Golden Standard Annotations')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('EDR Amplitude')
    ax.grid(True)
    ax.legend()
    plt.show()

    # --- SECTION 2: GOLDEN STANDARD INTERVALS ---
    seg_times_gs, gs_mean_insp, gs_mean_exp, gs_ie_ratio = compute_gs_mean_intervals(
        record, full_start, full_end, interval_size
    )
    print("\n=== Golden Standard Mean Intervals (exercise intervals) ===")
    for t0, mi, me, ratio in zip(seg_times_gs, gs_mean_insp, gs_mean_exp, gs_ie_ratio):
        t1 = t0 + interval_size
        print(f"[{t0:.0f}-{t1:.0f}s]: Insp={mi:.2f}s, Exp={me:.2f}s, I:E={ratio:.2f}")

    # --- SECTION 3: BLAND–ALTMAN (Inspiration vs Expiration) ---

    golden_in_times = np.array([t for label, t in zip(breaths.ann, breaths.timestamp) if label == '(In'])
    golden_ex_times = np.array([t for label, t in zip(breaths.ann, breaths.timestamp) if label == '(Ex'])
    matched_golden_in, matched_edr_in = pair_events_for_bland_altman(golden_in_times, in_times, max_tolerance=2.0)
    fig_ba_in, ax_ba_in = plt.subplots(figsize=(7, 5))
    bland_altman_plot(matched_golden_in, matched_edr_in,
                      title="Bland–Altman: EDR Inspiration vs. Golden",
                      ax=ax_ba_in)
    plt.show()

    matched_golden_ex, matched_edr_ex = pair_events_for_bland_altman(golden_ex_times, ex_times, max_tolerance=2.0)
    fig_ba_ex, ax_ba_ex = plt.subplots(figsize=(7, 5))
    bland_altman_plot(matched_golden_ex, matched_edr_ex,
                      title="Bland–Altman: EDR Expiration vs. Golden",
                      ax=ax_ba_ex)
    plt.show()

    # --- SECTION 4: BPM ANALYSIS ---
    interval_bpm = 10  # e.g. 10s
    seg_times_bpm, edr_bpms, gs_bpms = segment_bpm_edr(
        record,
        edr_time_qrs,
        edr_signal_qrs,
        start_time=full_start,
        end_time=full_end,
        interval_size=interval_bpm,
        min_breath_sec=DEFAULT_MIN_BREATH_SEC
    )

    print(f"\n=== BPM Comparison in {interval_bpm}-second intervals (exercise intervals) ===")
    for t, e_bpm, g_bpm in zip(seg_times_bpm, edr_bpms, gs_bpms):
        print(f"[{t:.0f}-{t+interval_bpm:.0f}s]: EDR BPM={e_bpm:.2f}, Gold BPM={g_bpm:.2f}")

    mask_bpm = ~np.isnan(edr_bpms) & ~np.isnan(gs_bpms)
    matched_edr_bpm = edr_bpms[mask_bpm]
    matched_gold_bpm = gs_bpms[mask_bpm]
    print(f"Number of valid EDR BPM points: {np.sum(~np.isnan(edr_bpms))}")
    print(f"Number of valid GS BPM points: {np.sum(~np.isnan(gs_bpms))}")
    print(f"Number of matched points: {len(matched_edr_bpm)}")

    if len(matched_edr_bpm) > 1:
        fig_ba_bpm, ax_ba_bpm = plt.subplots(figsize=(7, 5))
        differences = matched_edr_bpm - matched_gold_bpm
        averages = 0.5 * (matched_edr_bpm + matched_gold_bpm)
        mean_diff = np.mean(differences)
        sd_diff = np.std(differences)
        upper_loa = mean_diff + 1.96 * sd_diff
        lower_loa = mean_diff - 1.96 * sd_diff
        ax_ba_bpm.scatter(averages, differences, alpha=0.7, edgecolor='k')
        ax_ba_bpm.axhline(mean_diff, color='red', linestyle='--', label=f'Mean diff = {mean_diff:.2f}')
        ax_ba_bpm.axhline(upper_loa, color='green', linestyle='--', label=f'Upper LoA = {upper_loa:.2f}')
        ax_ba_bpm.axhline(lower_loa, color='green', linestyle='--', label=f'Lower LoA = {lower_loa:.2f}')
        ax_ba_bpm.set_title("Bland–Altman: EDR BPM vs. Gold BPM (10s intervals)")
        ax_ba_bpm.set_xlabel("Average BPM")
        ax_ba_bpm.set_ylabel("EDR BPM - Gold BPM")
        ax_ba_bpm.legend()
        plt.show()
    else:
        print("\nNot enough valid BPM points to do a Bland–Altman on BPM.")





# using the other peak detection: preprocessing fie:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, iirnotch, find_peaks
from scipy.interpolate import interp1d
from scipy.fft import fft, fftfreq
import sys
import os
from sklearn.decomposition import PCA, KernelPCA

def preprocess_signal(signal, fs): 
    b, a = butter(N=2, Wn=1, btype='high', fs=fs)
    filt_signal = filtfilt(b, a, signal)

    notch_cut_off = 50
    for co in np.arange(notch_cut_off, fs / 2, notch_cut_off):
        b, a = iirnotch(co / (fs / 2), 10)
        filt_signal = filtfilt(b, a, filt_signal)

    return filt_signal


def detect_ecg_peaks(signal, fs):
    from pan_tompkins import PanTompkins
    from pan_tompkins import HeartRate
    
    pan = PanTompkins(signal, fs)
    integrated_signal = pan.fit()
    bandpassed_signal = pan.filtered_BandPass

    hr_obj = HeartRate(signal, fs, integrated_signal, bandpassed_signal)
    R_peaks = hr_obj.find_r_peaks()
    print(f"Detected {len(R_peaks)} R-peaks with Pan-Tompkins")
    return np.array(R_peaks)  


def detect_ecg_peaks_engzee(signal, fs):
    from engzee_tompkins import EngzeeTompkins, EngzeeHeartRate

    engzee = EngzeeTompkins(signal, fs)
    integrated_signal = engzee.fit()
    bandpassed_signal = engzee.filtered_BandPass  # if you store it similarly
    hr_obj = EngzeeHeartRate(signal, fs, integrated_signal, bandpassed_signal)
    R_peaks = hr_obj.find_r_peaks()
    print(f"Detected {len(R_peaks)} R-peaks with EngzeeTompkins")
    return np.array(R_peaks)



def interpolate_from_R_peaks(loc, val, fs,
                             resampled_fs=10,
                             interp_method="linear",
                             filter_type="band"):

    sig_time = loc / fs
    interp_func = interp1d(sig_time, val, kind=interp_method, fill_value="extrapolate")

    new_time = np.arange(sig_time[0], sig_time[-1], 1.0 / resampled_fs)
    resampled_sig = interp_func(new_time)

    # bandpass for QRS amplitude might be narrower, e.g. 0.1–0.5
    if filter_type == "band":
        b, a = butter(N=2, Wn=[0.1, 0.5], btype='band', fs=resampled_fs)
    elif filter_type == "low":
        b, a = butter(N=2, Wn=0.4, btype='low', fs=resampled_fs)
    else:
        raise ValueError("Invalid filter_type. Choose 'band' or 'low'.")

    filtered_sig = filtfilt(b, a, resampled_sig)

    return (new_time, resampled_sig, filtered_sig)



def get_QRS_amplitude(ecg, fs, R_peaks=None, default_window_len=0.1):
    """
    Compute the QRS amplitude for each R-peak in the ECG signal.
    
    Parameters
    ----------
    ecg : np.array
        The ECG signal.
    fs : float
        Sampling frequency.
    R_peaks : np.array or None
        Precomputed R-peak indices. If None, the function calls detect_ecg_peaks().
    default_window_len : float, optional
        The default half-window length (in seconds) around each R-peak.
    
    Returns
    -------
    QRS_amplitude : np.array
        The computed QRS amplitudes.
    amplitude_resampled : tuple
        A tuple (new_time, resampled_signal, filtered_signal).
    """
    if R_peaks is None:
        R_peaks = detect_ecg_peaks(ecg, fs)
    # Ensure R_peaks is an array. If accidentally a single integer was passed, wrap it:
    if np.isscalar(R_peaks):
        R_peaks = np.array([R_peaks])
    elif not isinstance(R_peaks, np.ndarray):
        R_peaks = np.array(R_peaks)
    
    if len(R_peaks) < 2:
        return np.array([]), (np.array([]), None, np.array([]))
    
    QRS_amplitude = np.zeros(len(R_peaks))
    RR_samples = np.diff(R_peaks)
    
    for i in range(len(R_peaks)):
        if i == 0 or i == len(R_peaks) - 1:
            half_win = int(default_window_len * fs)
        else:
            local_RR = 0.5 * (RR_samples[i-1] + RR_samples[i])
            adaptive_len_sec = min(default_window_len, 0.15 * (local_RR / fs))
            half_win = int(adaptive_len_sec * fs)
    
        center = R_peaks[i]
        start = max(center - half_win, 0)
        end = min(center + half_win, len(ecg))
        window = ecg[start:end]
    
        baseline_win = int(0.02 * fs)  # 20ms
        baseline_start = max(center - baseline_win, start)
        baseline_segment = ecg[baseline_start:center]
        baseline_value = 0.0 if len(baseline_segment) < 1 else np.mean(baseline_segment)
    
        window_corrected = window - baseline_value
        QRS_amplitude[i] = window_corrected.max() - window_corrected.min()
    
    amplitude_resampled = interpolate_from_R_peaks(
        R_peaks, QRS_amplitude, fs,
        resampled_fs=10,
        interp_method="linear",
        filter_type="band"
    )
    
    return QRS_amplitude, amplitude_resampled


def get_RRI_from_signal(signal, fs, R_peaks=None, RRI_fs=10):
    """
    Compute the R-R intervals (RRI) from the ECG signal.
    
    Parameters
    ----------
    signal : np.array
        The ECG signal.
    fs : float
        Sampling frequency.
    R_peaks : np.array or None
        Precomputed R-peak indices. If None, the function calls detect_ecg_peaks().
    RRI_fs : float, optional
        The target sampling frequency for resampled RRI.
    
    Returns
    -------
    R_peaks : np.array
        The detected R-peak indices.
    RRI : np.array
        The R-R intervals in seconds.
    RRI_resampled : tuple
        A tuple (new_time, resampled_RRI, filtered_RRI).
    """
    if R_peaks is None:
        R_peaks = detect_ecg_peaks(signal, fs)
    # Ensure R_peaks is a NumPy array
    if np.isscalar(R_peaks):
        R_peaks = np.array([R_peaks])
    elif not isinstance(R_peaks, np.ndarray):
        R_peaks = np.array(R_peaks)
    
    RRI = np.diff(R_peaks) / fs  # Convert to seconds
    
    RRI_resampled = interpolate_from_R_peaks(
        R_peaks[:-1], RRI, fs, RRI_fs, interp_method="linear", filter_type="low"
    )
    
    return R_peaks, RRI, RRI_resampled



def get_QRS_effects(ecg, R_peaks, fs, search_window_len=0.06):
    BW_effect = np.zeros(len(R_peaks))
    AM_effect = np.zeros(len(R_peaks))
    half_win = int(search_window_len * fs)

    for i in range(len(R_peaks)):
        start = R_peaks[i] - half_win
        end = R_peaks[i] + half_win
        if start < 0 or end >= len(ecg):
            continue
        window = ecg[start:end]
        BW_effect[i] = np.mean([window.max(), window.min()])
        AM_effect[i] = window.max() - window.min()

    return BW_effect, AM_effect


def get_R_peak_amplitude(ecg, R_peaks):
    return ecg[R_peaks]


def get_EDR_from_ecg(ecg_signal, R_peaks, fs, method):
    if method == 'R_peak':
        edr_raw = ecg_signal[R_peaks]  
    else:
        search_window_len = 0.01
        edr_raw = np.zeros(len(R_peaks))
        for i, peak in enumerate(R_peaks):
            start_idx = peak - int(search_window_len * fs)
            end_idx = peak + int(search_window_len * fs)
            if start_idx < 0 or end_idx >= len(ecg_signal):
                continue
            window = ecg_signal[start_idx:end_idx]
            edr_raw[i] = window.max() - window.min()

    rpeak_times = R_peaks / fs
    interp_func = interp1d(rpeak_times, edr_raw, kind='linear', fill_value='extrapolate')
    resampled_fs = 10
    new_time = np.arange(rpeak_times[0], rpeak_times[-1], 1/resampled_fs)
    edr_interpolated = interp_func(new_time)

    b, a = butter(2, 0.5, btype='low', fs=resampled_fs)
    edr_smooth = filtfilt(b, a, edr_interpolated)

    return new_time, edr_smooth


def get_EDR_from_RRI(R_peaks, fs, resampled_fs=10, lp_cutoff=1.0):
    RR_intervals = np.diff(R_peaks) / fs
    if len(RR_intervals) < 2:
        return np.array([]), np.array([])

    rr_times = R_peaks[1:] / fs
    rr_interp = interp1d(rr_times, RR_intervals, kind='linear', fill_value="extrapolate")

    t_min, t_max = rr_times[0], rr_times[-1]
    edr_time = np.arange(t_min, t_max, 1/resampled_fs)
    edr_raw = rr_interp(edr_time)

    b, a = butter(2, lp_cutoff/(resampled_fs/2), btype='low')
    edr_smooth = filtfilt(b, a, edr_raw)

    return edr_time, edr_smooth


def annotate_axes(axes, record):
    exercises = [ann for ann in record.annotations if ann.name == 'EXERCISE' and ann.configuration == 'SEATED'][0]
    breaths = [ann for ann in record.annotations if ann.name == 'BREATHS' and ann.configuration == 'SEATED'][0]
    
    for ax in axes:
        # exercise intervals
        for j in range(0, len(exercises.ann), 2):
            if j + 1 < len(exercises.ann):
                start = exercises.timestamp[j]
                end = exercises.timestamp[j+1]
                ax.axvspan(start, end, color='red', alpha=0.2)
                
        #  exercises
        for j in range(len(exercises.ann) - 1):
            if exercises.timestamp[j] < ax.get_xlim()[1] and exercises.timestamp[j] > ax.get_xlim()[0]:
                ax.text(exercises.timestamp[j+1], ax.get_ylim()[0], exercises.ann[j+1], ha='left', va='bottom')
        
        # inspiration and expiration
        inspiration_idx = [k for k, breath in enumerate(breaths.ann) if breath == '(In']
        expiration_idx = [k for k, breath in enumerate(breaths.ann) if breath == '(Ex']
        
        for j in inspiration_idx:
            ax.axvline(breaths.timestamp[j], color='green', alpha=0.3, label='Inspiration' if j == inspiration_idx[0] else "")
        for j in expiration_idx:
            ax.axvline(breaths.timestamp[j], linestyle='--', color='blue', alpha=0.3, label='Expiration' if j == expiration_idx[0] else "")

DEFAULT_MIN_BREATH_SEC = 2.5
def detect_edr_breaths(edr_time, edr_signal, min_breath_sec=DEFAULT_MIN_BREATH_SEC):
    """
    Detect inspiration and expiration breaths from the EDR signal.
    """
    min_distance_samples = int(10 * min_breath_sec)
    in_peaks, _ = find_peaks(-edr_signal, distance=min_distance_samples)
    ex_peaks, _ = find_peaks(edr_signal, distance=min_distance_samples)
    in_times = edr_time[in_peaks]
    ex_times = edr_time[ex_peaks]
    return in_times, ex_times
def auto_detect_edr_breaths(edr_time, edr_signal, init_min_breath_sec=2.5,
                            adjust_factor=0.5, max_iterations=5, tol=0.05,
                            min_allowed=0.5, max_allowed=10.0):
    """
    Automatically adapts min_breath_sec with multiple passes until stable.
    """
    current_min_breath_sec = init_min_breath_sec

    for iteration in range(max_iterations):
        in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
        all_breaths = np.sort(np.concatenate([in_times, ex_times]))
        if len(all_breaths) < 2:
            print(f"Not enough breath events (iteration {iteration+1}). Stopping.")
            return in_times, ex_times
        
        # Compute median interval (with outlier rejection)
        breath_intervals = np.diff(all_breaths)
        median_rough = np.median(breath_intervals)
        upper_cut = 2.0 * median_rough
        lower_cut = 0.2 * median_rough
        cleaned_intervals = breath_intervals[(breath_intervals <= upper_cut) & (breath_intervals >= lower_cut)]
        median_interval = np.median(cleaned_intervals) if len(cleaned_intervals) >= 2 else median_rough

        new_min_breath_sec = adjust_factor * median_interval
        new_min_breath_sec = np.clip(new_min_breath_sec, min_allowed, max_allowed)
        ratio = new_min_breath_sec / current_min_breath_sec
        print(f"Iteration {iteration+1}: old={current_min_breath_sec:.2f}, new={new_min_breath_sec:.2f} (ratio={ratio:.2f})")
        if abs(ratio - 1.0) < tol:
            current_min_breath_sec = new_min_breath_sec
            break
        else:
            current_min_breath_sec = new_min_breath_sec

    # Final detection pass
    in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
    print(f"Final min_breath_sec after {iteration+1} iteration(s): {current_min_breath_sec:.2f}")
    return in_times, ex_times


import numpy as np
from scipy.signal import butter, filtfilt

def get_EDR_signal(ecg, fs, resampled_fs=10):
    """
    Create an ECG-derived respiration (EDR) signal by combining QRS amplitude and RRI signals.
    
    Parameters:
        ecg (np.array): The raw (or preprocessed) ECG signal.
        fs (float): The sampling frequency of the ECG signal.
        resampled_fs (float): The target sampling frequency for resampled features.
        
    Returns:
        t_common (np.array): The common time axis for the EDR signal.
        edr_filtered (np.array): The combined, filtered EDR signal.
    """
    # --- Step 1: Compute QRS Amplitude signal ---
    R_peaks, _, _ = get_RRI_from_signal(ecg, fs, RRI_fs=resampled_fs)
    QRS_amp, QRS_amp_resampled = get_QRS_amplitude(ecg, R_peaks, fs)
    t_qrs, qrs_resampled, _ = QRS_amp_resampled  # time axis and resampled QRS amplitude signal

    # --- Step 2: Compute RRI signal ---
    _, _, RRI_resampled = get_RRI_from_signal(ecg, fs, RRI_fs=resampled_fs)
    t_rri, rri_resampled, _ = RRI_resampled  # time axis and resampled RRI signal

    # --- Step 3: Define a common time axis ---
    # Determine the overlapping time range between the two signals.
    t_start = max(t_qrs[0], t_rri[0])
    t_end = min(t_qrs[-1], t_rri[-1])
    t_common = np.arange(t_start, t_end, 1.0/resampled_fs)

    # --- Step 4: Interpolate both signals onto the common time axis ---
    qrs_common = np.interp(t_common, t_qrs, qrs_resampled)
    rri_common = np.interp(t_common, t_rri, rri_resampled)

    # --- Step 5: Normalize both signals ---
    norm_qrs = (qrs_common - np.mean(qrs_common)) / np.std(qrs_common)
    norm_rri = (rri_common - np.mean(rri_common)) / np.std(rri_common)

    # --- Step 6: Combine the normalized signals ---
    edr_raw = 0.5 * (norm_qrs + norm_rri)

    # --- Step 7: Smooth the combined EDR signal ---
    cutoff = 0.6  # Hz
    nyquist = 0.5 * resampled_fs
    b, a = butter(N=2, Wn=cutoff/nyquist, btype='low')
    edr_filtered = filtfilt(b, a, edr_raw)

    # Return the common time axis and the filtered EDR signal.
    return t_common, edr_filtered


# and calling them:

In [ ]:
# Preprocess your signal first
filtered_ecg = preprocessing.preprocess_signal(ecg_signal, fs)

# Get old R-peaks using Pan-Tompkins:
R_peaks_old, RRI_old, RRI_resampled_old = preprocessing.get_RRI_from_signal(filtered_ecg, fs)
QRS_amp_old, QRS_amp_resampled_old = preprocessing.get_QRS_amplitude(filtered_ecg, fs, R_peaks=R_peaks_old)

# Get new R-peaks using the Engzee method:
R_peaks_new = preprocessing.detect_ecg_peaks_engzee(filtered_ecg, fs)
R_peaks_new, RRI_new, RRI_resampled_new = preprocessing.get_RRI_from_signal(filtered_ecg, fs, R_peaks=R_peaks_new)
QRS_amp_new, QRS_amp_resampled_new = preprocessing.get_QRS_amplitude(filtered_ecg, fs, R_peaks=R_peaks_new)


# Full golden analysis of a patient code:

In [ ]:
from diamonds import data as dt
import numpy as np
import matplotlib.pyplot as plt


class GoldenBreathAnalyzer:
    def __init__(self, patient, session_type=dt.SeatedSession, interval_size=30):
        self.record = patient
        self.session = self.record[session_type]
        self.breaths = self.session[dt.Breaths]
        self.exercise = self.session[dt.Exercise]
        self.interval_size = interval_size

        self.in_times, self.ex_times = self._get_golden_breath_events()
        self.start_time = np.min(self.exercise.timestamp)
        self.end_time = np.max(self.exercise.timestamp)

        print(f"Exercise interval: {self.start_time:.1f}s to {self.end_time:.1f}s")

    def _get_golden_breath_events(self):
        in_times = np.array([t for t, l in zip(self.breaths.timestamp, self.breaths.ann) if l == '(In'])
        ex_times = np.array([t for t, l in zip(self.breaths.timestamp, self.breaths.ann) if l == '(Ex'])
        return in_times, ex_times

    def _annotate_exercises(self, ax):
        for tstamp, label in zip(self.exercise.timestamp, self.exercise.ann):
            ax.axvline(tstamp, color='magenta', linestyle='--', alpha=0.5)
            ax.text(tstamp, ax.get_ylim()[1] * 0.95, label,
                    rotation=90, verticalalignment='top', color='magenta')

    def _compute_bpm(self, t_start, t_end):
        valid_in = self.in_times[(self.in_times >= t_start) & (self.in_times <= t_end)]
        if len(valid_in) < 2:
            return np.nan
        return 60.0 / np.mean(np.diff(valid_in))

    def segment_bpm(self):
        seg_times = np.arange(self.start_time, self.end_time, self.interval_size)
        bpm_vals = [self._compute_bpm(t, t + self.interval_size) for t in seg_times]
        return seg_times, np.array(bpm_vals)

    def _compute_mean_insp_exp(self, t_start, t_end):
        valid_in = self.in_times[(self.in_times >= t_start) & (self.in_times <= t_end)]
        valid_ex = self.ex_times[(self.ex_times >= t_start) & (self.ex_times <= t_end)]

        mi = np.mean(np.diff(valid_in)) if len(valid_in) >= 2 else np.nan
        me = np.mean(np.diff(valid_ex)) if len(valid_ex) >= 2 else np.nan
        return mi, me

    def segment_mean_intervals(self):
        seg_times = np.arange(self.start_time, self.end_time, self.interval_size)
        mean_insp, mean_exp, ie_ratios = [], [], []

        for t in seg_times:
            mi, me = self._compute_mean_insp_exp(t, t + self.interval_size)
            mean_insp.append(mi)
            mean_exp.append(me)
            ie_ratios.append(mi / me if not np.isnan(mi) and not np.isnan(me) and me > 0 else np.nan)

        return seg_times, np.array(mean_insp), np.array(mean_exp), np.array(ie_ratios)

    def plot_bpm(self, seg_times, bpm_vals):
        plt.figure(figsize=(8, 4))
        ax = plt.gca()
        ax.plot(seg_times, bpm_vals, '-o', label='Golden BPM (Inspiration)')
        ax.set_title("Golden Standard: BPM in Exercise Intervals")
        ax.set_xlabel("Segment Start Time (s)")
        ax.set_ylabel("Breaths Per Minute")
        ax.grid(True)
        ax.legend()
        self._annotate_exercises(ax)
        plt.show()

    def plot_mean_insp_exp(self, seg_times, mean_insp, mean_exp, ie_ratios):
        plt.figure(figsize=(10, 4))
        ax1 = plt.gca()
        ax1.plot(seg_times, mean_insp, '-o', label="Mean Insp Interval (s)")
        ax1.plot(seg_times, mean_exp, '-o', label="Mean Exp Interval (s)")
        ax1.set_title("Mean Inspiration & Expiration (Exercise)")
        ax1.set_xlabel("Segment Start Time (s)")
        ax1.set_ylabel("Interval (s)")
        ax1.grid(True)
        ax1.legend()
        self._annotate_exercises(ax1)
        plt.show()

        plt.figure(figsize=(8, 4))
        ax2 = plt.gca()
        ax2.plot(seg_times, ie_ratios, '-o', color='orange', label="I:E Ratio")
        ax2.set_title("I:E Ratio in Exercise Intervals")
        ax2.set_xlabel("Segment Start Time (s)")
        ax2.set_ylabel("I:E Ratio")
        ax2.grid(True)
        ax2.legend()
        self._annotate_exercises(ax2)
        plt.show()

    def print_total_breath_count(self):
        total = len(self.breaths.ann)
        print(f"Total breath events: {total}")
        print(f"Estimated breath cycles: {total / 2:.1f}")

    def print_breath_counts_per_interval(self):
        seg_times = np.arange(self.start_time, self.end_time, self.interval_size)
        print(f"\nBreath event counts per {self.interval_size}-second interval:")
        for t0 in seg_times:
            t1 = t0 + self.interval_size
            mask = (np.array(self.breaths.timestamp) >= t0) & (np.array(self.breaths.timestamp) < t1)
            print(f"[{t0:.0f}-{t1:.0f}s]: {np.sum(mask)} events")

    def run_analysis(self):
        print("\n=== Running Golden Breath Analysis ===")
        bpm_times, bpm_vals = self.segment_bpm()
        self.plot_bpm(bpm_times, bpm_vals)

        seg_times, mean_insp, mean_exp, ie_ratios = self.segment_mean_intervals()
        self.plot_mean_insp_exp(seg_times, mean_insp, mean_exp, ie_ratios)

        print("\n=== Summary Table ===")
        for t0, mi, me, ratio in zip(seg_times, mean_insp, mean_exp, ie_ratios):
            t1 = t0 + self.interval_size
            print(f"[{t0:.0f}-{t1:.0f}s] Insp={mi:.2f}s, Exp={me:.2f}s, I:E={ratio:.2f}")

        print("\n=== BPM Summary ===")
        for t0, bpm in zip(bpm_times, bpm_vals):
            t1 = t0 + self.interval_size
            print(f"[{t0:.0f}-{t1:.0f}s]: BPM={bpm:.2f}")

        self.print_total_breath_count()
        self.print_breath_counts_per_interval()


# working bland altman plots:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#############################################
# 1) Bland–Altman Helper
#############################################
def bland_altman_plot(method1, method2, title="Bland–Altman Plot"):
    """
    Create a simple Bland–Altman plot comparing method1 vs. method2 arrays.
    """
    m1 = np.array(method1)
    m2 = np.array(method2)
    # Remove NaNs so lengths match
    valid_mask = ~np.isnan(m1) & ~np.isnan(m2)
    m1 = m1[valid_mask]
    m2 = m2[valid_mask]

    if len(m1) < 2:
        print(f"[WARNING] Not enough data to plot {title}")
        return

    diff = m1 - m2
    avg = 0.5 * (m1 + m2)

    md = np.mean(diff)                # mean difference
    sd = np.std(diff, ddof=1)         # sample std dev
    upper = md + 1.96 * sd
    lower = md - 1.96 * sd

    plt.figure(figsize=(7, 5))
    plt.scatter(avg, diff, alpha=0.7, edgecolor='k')
    plt.axhline(md,    color='red',  linestyle='--', label=f'Mean diff={md:.2f}')
    plt.axhline(upper, color='gray', linestyle='--', label=f'+1.96SD={upper:.2f}')
    plt.axhline(lower, color='gray', linestyle='--', label=f'-1.96SD={lower:.2f}')
    plt.title(title)
    plt.xlabel('Average of Two Methods')
    plt.ylabel('Difference (Method1 - Method2)')
    plt.legend()
    plt.grid(True)
    plt.show()

#############################################
# 2) The Main Function: Compare Gold vs. EDR
#############################################
def bland_altman_gold_vs_edr(
    breaths,            # annotation object
    edr_time,           # EDR time array
    edr_signal,         # EDR amplitude array
    compute_bpm_from_inspiration_times,  # your function
    compute_mean_insp_exp_edr,           # your function
    auto_detect_edr_breaths,             # your function
    seg_start,          # e.g. np.min(exercise.timestamp)
    seg_end,            # e.g. np.max(exercise.timestamp)
    interval_size=10
):
    """
    1) Use 'breaths' to define gold-standard in/ex times (Method1).
    2) Auto-detect EDR-based in/ex times from (edr_time, edr_signal) (Method2).
    3) Segment both methods in 10s intervals from seg_start to seg_end.
    4) Compute BPM, MeanInsp, MeanExp for each method in each interval.
    5) Do Bland–Altman plots for each measure.
    """

    # A) Gold standard in/ex times (Method1)
    in_idx = [i for i, lbl in enumerate(breaths.ann) if lbl == '(In']
    ex_idx = [i for i, lbl in enumerate(breaths.ann) if lbl == '(Ex']
    in_times_gold = np.array(breaths.timestamp)[in_idx]
    ex_times_gold = np.array(breaths.timestamp)[ex_idx]


    # B) EDR-based in/ex times (Method2)
    in_times_edr, ex_times_edr = auto_detect_edr_breaths(edr_time, edr_signal)

    # Arrays to store each method's BPM, Insp, Exp per interval
    gold_bpm_list, gold_insp_list, gold_exp_list = [], [], []
    edr_bpm_list, edr_insp_list, edr_exp_list = [], [], []

    # 10-second windows
    seg_edges = np.arange(seg_start, seg_end, interval_size)
    for start_t in seg_edges:
        end_t = start_t + interval_size

        # Gold standard metrics
        gold_bpm = compute_bpm_from_inspiration_times(in_times_gold, start_t, end_t)
        gold_insp, gold_exp = compute_mean_insp_exp_edr(in_times_gold, ex_times_gold, start_t, end_t)

        gold_bpm_list.append(gold_bpm)
        gold_insp_list.append(gold_insp)
        gold_exp_list.append(gold_exp)

        # EDR-based metrics
        edr_bpm = compute_bpm_from_inspiration_times(in_times_edr, start_t, end_t)
        edr_insp, edr_exp = compute_mean_insp_exp_edr(in_times_edr, ex_times_edr, start_t, end_t)

        edr_bpm_list.append(edr_bpm)
        edr_insp_list.append(edr_insp)
        edr_exp_list.append(edr_exp)

    # C) Bland–Altman
    bland_altman_plot(gold_insp_list, edr_insp_list, title="Bland–Altman: Inspiration Duration (Gold vs EDR)")
    bland_altman_plot(gold_exp_list, edr_exp_list, title="Bland–Altman: Expiration Duration (Gold vs EDR)")
    bland_altman_plot(gold_bpm_list, edr_bpm_list, title="Bland–Altman: BPM (Gold vs EDR)")

    # Optionally return these arrays if you want to do further analysis
    return seg_edges, gold_bpm_list, edr_bpm_list, gold_insp_list, edr_insp_list, gold_exp_list, edr_exp_list

seg_start = np.min(exercise.timestamp)
seg_end   = np.max(exercise.timestamp)

bland_altman_gold_vs_edr(
    breaths=breaths,
    edr_time=edr_time,
    edr_signal=edr_signal,
    compute_bpm_from_inspiration_times=compute_bpm_from_inspiration_times,
    compute_mean_insp_exp_edr=compute_mean_insp_exp_edr,
    auto_detect_edr_breaths=auto_detect_edr_breaths,
    seg_start=seg_start,
    seg_end=seg_end,
    interval_size=10
)


Old maybe working bland altman?:

In [ ]:
#!/usr/bin/env python
"""
breath_bland_altman.py

Provides a convenient function `run_bland_altman_for_breaths` that:
1) Extracts annotation-based (gold) in/ex times,
2) Detects EDR-based in/ex times,
3) Segments both methods into intervals and computes BPM, inspiration, expiration durations,
4) Plots Bland–Altman comparisons for BPM, inspiration duration, and expiration duration.

Dependencies:
  - numpy
  - matplotlib
  - your 'breath_detection' module (for auto_detect_edr_breaths, etc.)
  - your 'compute_bpm_from_inspiration_times' and 'compute_mean_insp_exp_edr' functions
"""

import numpy as np
import matplotlib.pyplot as plt

# If you have your breath_detection utilities, import them:
from breath_detection import auto_detect_edr_breaths, DEFAULT_MIN_BREATH_SEC
from features import compute_bpm_from_inspiration_times, compute_mean_insp_exp_edr
###############################################################################
# HELPER FUNCTIONS
###############################################################################

def count_inspirations(in_times, start_t, end_t):
    """Return how many inspirations fall in [start_t, end_t)."""
    mask = (in_times >= start_t) & (in_times < end_t)
    return np.sum(mask)

def inspirations_per_minute(in_times, start_t, end_t):
    """Compute BPM from the number of inspirations in [start_t, end_t)."""
    count = count_inspirations(in_times, start_t, end_t)
    window_sec = end_t - start_t
    return 60.0 * count / window_sec if window_sec > 0 else np.nan

def gold_breath_metrics(in_times, ex_times, seg_start, seg_end, interval_size=10):
    """
    Segment the annotation-based (gold-standard) in/ex times into intervals of length
    'interval_size' (in seconds). For each interval, compute:
      - gold_BPM       : # of inspirations * (60 / interval_size)
      - gold_mean_insp : mean interval between consecutive in_times in that segment
      - gold_mean_exp  : mean interval between consecutive ex_times in that segment

    Returns (seg_edges, gold_BPM_array, gold_insp_array, gold_exp_array)
    """
    seg_edges = np.arange(seg_start, seg_end, interval_size)
    
    gold_BPM_array = []
    gold_insp_array = []
    gold_exp_array = []
    
    for start_t in seg_edges:
        end_t = start_t + interval_size

        # All inspirations in this window
        mask_in = (in_times >= start_t) & (in_times < end_t)
        these_ins = in_times[mask_in]

        # All expirations in this window
        mask_ex = (ex_times >= start_t) & (ex_times < end_t)
        these_ex = ex_times[mask_ex]

        # BPM from # of inspirations
        breath_count = len(these_ins)
        gold_bpm = 60.0 * breath_count / interval_size if interval_size > 0 else np.nan

        # Mean inspiration interval
        if len(these_ins) >= 2:
            mean_insp = np.mean(np.diff(these_ins))
        else:
            mean_insp = np.nan

        # Mean expiration interval
        if len(these_ex) >= 2:
            mean_exp = np.mean(np.diff(these_ex))
        else:
            mean_exp = np.nan

        gold_BPM_array.append(gold_bpm)
        gold_insp_array.append(mean_insp)
        gold_exp_array.append(mean_exp)
    
    return seg_edges, np.array(gold_BPM_array), np.array(gold_insp_array), np.array(gold_exp_array)

def edr_breath_metrics(in_times_edr, ex_times_edr,
                       compute_bpm_from_inspiration_times,
                       compute_mean_insp_exp_edr,
                       seg_start, seg_end, interval_size=10):
    """
    Same logic as gold_breath_metrics, but uses EDR-detected in/ex times
    plus your own functions for BPM and mean insp/exp intervals.
    
    Returns (seg_edges, edr_BPM_array, edr_insp_array, edr_exp_array).
    """
    seg_edges = np.arange(seg_start, seg_end, interval_size)
    
    edr_BPM_array = []
    edr_insp_array = []
    edr_exp_array = []
    
    for start_t in seg_edges:
        end_t = start_t + interval_size
        
        # Use your existing helper functions to get BPM, MeanInsp, MeanExp
        bpm_val = compute_bpm_from_inspiration_times(in_times_edr, start_t, end_t)
        mean_insp_val, mean_exp_val = compute_mean_insp_exp_edr(in_times_edr, ex_times_edr,
                                                                start_t, end_t)

        edr_BPM_array.append(bpm_val)
        edr_insp_array.append(mean_insp_val)
        edr_exp_array.append(mean_exp_val)
    
    return seg_edges, np.array(edr_BPM_array), np.array(edr_insp_array), np.array(edr_exp_array)

def bland_altman_plot(method1, method2, title="Bland–Altman Plot"):
    """
    Create a simple Bland–Altman plot comparing method1 vs. method2 arrays.
    """
    # Convert to arrays, remove NaNs so lengths match
    m1 = np.array(method1)
    m2 = np.array(method2)
    mask = ~np.isnan(m1) & ~np.isnan(m2)
    m1 = m1[mask]
    m2 = m2[mask]

    if len(m1) == 0:
        print(f"[WARNING] No valid data for {title}")
        return

    # Differences and averages
    diff = m1 - m2
    avg = 0.5 * (m1 + m2)

    # Mean and ±1.96 SD
    md = np.mean(diff)
    sd = np.std(diff, ddof=1)
    upper = md + 1.96 * sd
    lower = md - 1.96 * sd

    # Plot
    plt.figure(figsize=(7, 5))
    plt.scatter(avg, diff, alpha=0.7, edgecolor='k')
    plt.axhline(md,    color='red',  linestyle='--', label=f'Mean diff={md:.2f}')
    plt.axhline(upper, color='gray', linestyle='--', label=f'+1.96SD={upper:.2f}')
    plt.axhline(lower, color='gray', linestyle='--', label=f'-1.96SD={lower:.2f}')
    plt.title(title)
    plt.xlabel('Average of Two Methods')
    plt.ylabel('Difference (Method1 - Method2)')
    plt.legend()
    plt.grid(True)
    plt.show()

###############################################################################
# 2) MAIN ENTRY POINT
###############################################################################

def run_bland_altman_for_breaths(
    breaths,
    exercise, 
    edr_time, 
    edr_signal,
    compute_bpm_from_inspiration_times,
    compute_mean_insp_exp_edr,
    interval_size=10
):
    """
    1) Extract gold-standard in/ex times from 'breaths'.
    2) Detect EDR-based breath times from 'edr_time'/'edr_signal'.
    3) Segment both gold and EDR in 10s intervals (or 'interval_size').
    4) Plot Bland–Altman for BPM, mean inspiration, mean expiration.

    You must provide:
      - compute_bpm_from_inspiration_times(in_times, start_t, end_t)
      - compute_mean_insp_exp_edr(in_times, ex_times, start_t, end_t)

    Example usage:
      from breath_bland_altman import run_bland_altman_for_breaths
      run_bland_altman_for_breaths(
          breaths=breaths, 
          exercise=exercise,
          edr_time=edr_time,
          edr_signal=edr_signal,
          compute_bpm_from_inspiration_times=compute_bpm_from_inspiration_times,
          compute_mean_insp_exp_edr=compute_mean_insp_exp_edr,
          interval_size=10
      )
    """

    # 1) Extract gold-standard in/ex times from 'breaths'
    in_idx = [i for i, label in enumerate(breaths.ann) if label == '(In']
    ex_idx = [i for i, label in enumerate(breaths.ann) if label == '(Ex']

    in_times = np.array(breaths.timestamp)[in_idx]
    ex_times = np.array(breaths.timestamp)[ex_idx]

    # 2) Detect EDR-based breath times
    in_times_edr, ex_times_edr = auto_detect_edr_breaths(
        edr_time, edr_signal, 
        init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
        adjust_factor=0.6
    )

    # 3) Segment start/end from exercise timestamps
    seg_start = float(np.min(exercise.timestamp))
    seg_end   = float(np.max(exercise.timestamp))

    # 4a) Gold arrays
    seg_edges_gold, gold_bpm, gold_insp, gold_exp = gold_breath_metrics(
        in_times, ex_times,
        seg_start=seg_start,
        seg_end=seg_end,
        interval_size=interval_size
    )

    # 4b) EDR arrays
    seg_edges_edr, edr_bpm, edr_insp, edr_exp = edr_breath_metrics(
        in_times_edr, ex_times_edr,
        compute_bpm_from_inspiration_times,
        compute_mean_insp_exp_edr,
        seg_start=seg_start,
        seg_end=seg_end,
        interval_size=interval_size
    )

    # 5) Bland–Altman plots
    bland_altman_plot(gold_bpm, edr_bpm, title="Bland–Altman: Gold BPM vs EDR BPM")
    bland_altman_plot(gold_insp, edr_insp, title="Bland–Altman: Gold Insp Duration")
    bland_altman_plot(gold_exp, edr_exp, title="Bland–Altman: Gold Exp Duration")


# Optionally, if you want to run directly:
if __name__ == "__main__":
    print("This file is meant to be imported. Please call run_bland_altman_for_breaths(...) from your code.")


In [ ]:
# find peaks peak detection:

In [ ]:
import plotly.graph_objects as go 
import plotly.express as px
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

# Convert to numpy array for convenience
timestamps = ecg_iso.samples_df["Timestamp"].values
inverted_amp = -ecg_iso.samples_df["Amplitude"].values  # Invert amplitude

# Use find_peaks to locate R-waves
# Tune 'distance' (minimum spacing) or 'height' based on your signal
peak_indices, properties = find_peaks(
    inverted_amp,
    distance=150,      # minimum number of samples between peaks; adjust per your sampling rate
    height=350       # minimum height of the peak; adjust based on typical amplitude
)

# peak_indices are the indices where an R-peak is detected
# properties contains additional info (e.g., the actual height at each peak)


# Plot ECGIsolated
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ecg_iso.samples_df["Timestamp"],
    y=-ecg_iso.samples_df["Amplitude"],
    mode='lines',
    name="ECGIsolated"
))

# Add the R-peaks as scatter markers
fig.add_trace(go.Scatter(
    x=timestamps[peak_indices],
    y=inverted_amp[peak_indices],
    mode='markers',
    marker=dict(symbol='circle', size=8),
    name="R-peaks"
))

# Shade exercise intervals and add split lines as before
for i, label in enumerate(exercise.ann):
    start = exercise.timestamp[i]
    end = exercise.timestamp[i + 1] if i < len(exercise.timestamp) - 1 else ecg_iso.samples_df["Timestamp"].iloc[-1]

    fig.add_vrect(
        x0=start,
        x1=end,
        fillcolor="red",
        opacity=0.2,
        line_width=0,
        annotation_text=label,
        annotation_position="top left"
    )

    if i < len(exercise.timestamp) - 1:
        fig.add_trace(go.Scatter(
            x=[end, end],
            y=[min(ecg_iso.samples_df["Amplitude"]), max(ecg_iso.samples_df["Amplitude"])],
            mode="lines",
            line=dict(color="black", dash="solid"),
            name="Exercise Split",
            showlegend=(i == 0)
        ))

# Final layout
fig.update_layout(
    title="ECGIsolated Signal with Exercise Intervals and Detected R-peaks",
    xaxis_title="Time (s)",
    yaxis_title="Amplitude",
    legend=dict(itemsizing="constant")
)

# If you want to zoom in on a range:
fig.update_xaxes(range=[200, 250])

fig.show()



# preprocessing.py før 27-03-2025, kl. 22:44:

In [ ]:


import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, iirnotch, find_peaks
from scipy.interpolate import interp1d
from scipy.fft import fft, fftfreq
import sys
import os
from sklearn.decomposition import PCA, KernelPCA
from pan_tompkins import detect_ecg_peaks


def preprocess_signal(signal, fs): 
    b, a = butter(N=2, Wn=1, btype='high', fs=fs)
    filt_signal = filtfilt(b, a, signal)

    notch_cut_off = 50
    for co in np.arange(notch_cut_off, fs / 2, notch_cut_off):
        b, a = iirnotch(co / (fs / 2), 10)
        filt_signal = filtfilt(b, a, filt_signal)

    return filt_signal


def interpolate_from_R_peaks(loc, val, fs,  
                             resampled_fs=10,
                             interp_method="linear",
                             filter_type="band"):

    sig_time = loc / fs
    interp_func = interp1d(sig_time, val, kind=interp_method, fill_value="extrapolate")

    new_time = np.arange(sig_time[0], sig_time[-1], 1.0 / resampled_fs)
    resampled_sig = interp_func(new_time)


    if filter_type == "band":
        b, a = butter(N=2, Wn=[0.1, 0.5], btype='band', fs=resampled_fs)
    elif filter_type == "low":
        b, a = butter(N=2, Wn=0.4, btype='low', fs=resampled_fs)
    else:
        raise ValueError("Invalid filter_type. Choose 'band' or 'low'.")

    filtered_sig = filtfilt(b, a, resampled_sig)

    return (new_time, resampled_sig, filtered_sig)



def get_QRS_amplitude(ecg, R_peaks, fs, default_window_len=0.1):

    if len(R_peaks) < 2:
        # Not enough peaks to do anything
        return np.array([]), (np.array([]), None, np.array([]))

    QRS_amplitude = np.zeros(len(R_peaks))


    RR_samples = np.diff(R_peaks)  

    for i in range(len(R_peaks)):
       
        if i == 0:
          
            half_win = int(default_window_len * fs)
        elif i == len(R_peaks) - 1:
           
            half_win = int(default_window_len * fs)
        else:
            
            local_RR = 0.5 * (RR_samples[i-1] + RR_samples[i])

            adaptive_len_sec = min(default_window_len, 0.15 * (local_RR / fs))
            half_win = int(adaptive_len_sec * fs)

  
        center = R_peaks[i]
        start = max(center - half_win, 0)
        end   = min(center + half_win, len(ecg))
        window = ecg[start:end]

   
        baseline_win = int(0.02 * fs) 
        baseline_start = max(center - baseline_win, start)
        baseline_segment = ecg[baseline_start:center]
        if len(baseline_segment) < 1:
            baseline_value = 0.0
        else:
            baseline_value = np.mean(baseline_segment)

        window_corrected = window - baseline_value


        QRS_amplitude[i] = window_corrected.max() - window_corrected.min()

 
    amplitude_resampled = interpolate_from_R_peaks(
        R_peaks, QRS_amplitude, fs,
        resampled_fs=10,
        interp_method="linear",
        filter_type="band"
    )

    return QRS_amplitude, amplitude_resampled


def get_RRI_from_signal(signal, fs, RRI_fs=10):
  
    R_peaks = detect_ecg_peaks(signal, fs)
    RRI = np.diff(R_peaks) / fs  
    

    RRI_resampled = interpolate_from_R_peaks(R_peaks[:-1], RRI, fs, RRI_fs, interp_method="linear", filter_type="low")

    return R_peaks, RRI, RRI_resampled


def get_QRS_effects(ecg, R_peaks, fs, search_window_len=0.06):
    BW_effect = np.zeros(len(R_peaks))
    AM_effect = np.zeros(len(R_peaks))
    half_win = int(search_window_len * fs)

    for i in range(len(R_peaks)):
        start = R_peaks[i] - half_win
        end = R_peaks[i] + half_win
        if start < 0 or end >= len(ecg):
            continue
        window = ecg[start:end]
        BW_effect[i] = np.mean([window.max(), window.min()])
        AM_effect[i] = window.max() - window.min()

    return BW_effect, AM_effect


def get_R_peak_amplitude(ecg, R_peaks):
    return ecg[R_peaks]


def get_EDR_from_ecg(ecg_signal, R_peaks, fs, method):
    if method == 'R_peak':
        edr_raw = ecg_signal[R_peaks]  
    else:
        search_window_len = 0.01
        edr_raw = np.zeros(len(R_peaks))
        for i, peak in enumerate(R_peaks):
            start_idx = peak - int(search_window_len * fs)
            end_idx = peak + int(search_window_len * fs)
            if start_idx < 0 or end_idx >= len(ecg_signal):
                continue
            window = ecg_signal[start_idx:end_idx]
            edr_raw[i] = window.max() - window.min()

    rpeak_times = R_peaks / fs
    interp_func = interp1d(rpeak_times, edr_raw, kind='linear', fill_value='extrapolate')
    resampled_fs = 10
    new_time = np.arange(rpeak_times[0], rpeak_times[-1], 1/resampled_fs)
    edr_interpolated = interp_func(new_time)

    b, a = butter(2, 0.5, btype='low', fs=resampled_fs)
    edr_smooth = filtfilt(b, a, edr_interpolated)

    return new_time, edr_smooth


def get_EDR_from_RRI(R_peaks, fs, resampled_fs=10, lp_cutoff=1.0):
    RR_intervals = np.diff(R_peaks) / fs
    if len(RR_intervals) < 2:
        return np.array([]), np.array([])

    rr_times = R_peaks[1:] / fs
    rr_interp = interp1d(rr_times, RR_intervals, kind='linear', fill_value="extrapolate")

    t_min, t_max = rr_times[0], rr_times[-1]
    edr_time = np.arange(t_min, t_max, 1/resampled_fs)
    edr_raw = rr_interp(edr_time)

    b, a = butter(2, lp_cutoff/(resampled_fs/2), btype='low')
    edr_smooth = filtfilt(b, a, edr_raw)

    return edr_time, edr_smooth


def annotate_axes(axes, record):
    exercises = [ann for ann in record.annotations if ann.name == 'EXERCISE' and ann.configuration == 'SEATED'][0]
    breaths = [ann for ann in record.annotations if ann.name == 'BREATHS' and ann.configuration == 'SEATED'][0]
    
    for ax in axes:
        # exercise intervals
        for j in range(0, len(exercises.ann), 2):
            if j + 1 < len(exercises.ann):
                start = exercises.timestamp[j]
                end = exercises.timestamp[j+1]
                ax.axvspan(start, end, color='red', alpha=0.2)
                
        #  exercises
        for j in range(len(exercises.ann) - 1):
            if exercises.timestamp[j] < ax.get_xlim()[1] and exercises.timestamp[j] > ax.get_xlim()[0]:
                ax.text(exercises.timestamp[j+1], ax.get_ylim()[0], exercises.ann[j+1], ha='left', va='bottom')
        
        # inspiration and expiration
        inspiration_idx = [k for k, breath in enumerate(breaths.ann) if breath == '(In']
        expiration_idx = [k for k, breath in enumerate(breaths.ann) if breath == '(Ex']
        
        for j in inspiration_idx:
            ax.axvline(breaths.timestamp[j], color='green', alpha=0.3, label='Inspiration' if j == inspiration_idx[0] else "")
        for j in expiration_idx:
            ax.axvline(breaths.timestamp[j], linestyle='--', color='blue', alpha=0.3, label='Expiration' if j == expiration_idx[0] else "")



def get_EDR_signal(ecg, fs, resampled_fs=10):
    """
    combining QRS amplitude and RRI signals - look at it later v
    
    Parameters:
        ecg (np.array): The raw (or preprocessed) ECG signal.
        fs (float): The sampling frequency of the ECG signal.
        resampled_fs (float): The target sampling frequency for resampled features.
        
    Returns:
        t_common (np.array): The common time axis for the EDR signal.
        edr_filtered (np.array): The combined, filtered EDR signal.
    """
    # --- Step 1: Compute QRS Amplitude signal ---
    R_peaks, _, _ = get_RRI_from_signal(ecg, fs, RRI_fs=resampled_fs)
    QRS_amp, QRS_amp_resampled = get_QRS_amplitude(ecg, R_peaks, fs)
    t_qrs, qrs_resampled, _ = QRS_amp_resampled  # time axis and resampled QRS amplitude signal

    # --- Step 2: Compute RRI signal ---
    _, _, RRI_resampled = get_RRI_from_signal(ecg, fs, RRI_fs=resampled_fs)
    t_rri, rri_resampled, _ = RRI_resampled  # time axis and resampled RRI signal

    # --- Step 3: Define a common time axis ---
    # Determine the overlapping time range between the two signals.
    t_start = max(t_qrs[0], t_rri[0])
    t_end = min(t_qrs[-1], t_rri[-1])
    t_common = np.arange(t_start, t_end, 1.0/resampled_fs)

    # --- Step 4: Interpolate both signals onto the common time axis ---
    qrs_common = np.interp(t_common, t_qrs, qrs_resampled)
    rri_common = np.interp(t_common, t_rri, rri_resampled)

    # --- Step 5: normalizing both signals ---
    norm_qrs = (qrs_common - np.mean(qrs_common)) / np.std(qrs_common)
    norm_rri = (rri_common - np.mean(rri_common)) / np.std(rri_common)

    # --- Step 6: Combining the normalized signals ---
    edr_raw = 0.5 * (norm_qrs + norm_rri)

    # --- Step 7: Smoothh the combined EDR signal ---
    cutoff = 0.6  # Hz
    nyquist = 0.5 * resampled_fs
    b, a = butter(N=2, Wn=cutoff/nyquist, btype='low')
    edr_filtered = filtfilt(b, a, edr_raw)

    # returning the common time axis and the filtered EDR signal.
    return t_common, edr_filtered

In [ ]:


from features import build_feature_table
import pandas as pd
import diamonds.data as dt
from diamonds import load_patients


ptt = load_patients(show_progress=True)
session_type = dt.SeatedSession  

feature_list = []

for pt in ptt:
    print(f"Processing patient: {pt.id}")
    
    try:
        session = pt[session_type]
        
        
        EMG, ECG = pt[session_type, dt.EMG].decompose()
        ecg_signal = -ECG.samples[:, 1]  # use channel 1 and flip signal
        fs = ECG.fs

        
        filtered_ecg = preprocess_signal(ecg_signal, fs)

        
        R_peaks, RRI, RRI_resampled = get_RRI_from_signal(filtered_ecg, fs)

        
        QRS_amplitude, QRS_amplitude_resampled = get_QRS_amplitude(filtered_ecg, R_peaks, fs)

        
        _df = build_feature_table(
            patient=pt,
            session=session,
            ecg_signal=filtered_ecg,
            fs=fs,
            QRS_amplitude_resampled=QRS_amplitude_resampled
        )

        feature_list.append(_df)

    except Exception as e:
        print(f"⚠️ Failed for patient {pt.id}: {e}")


features_df = pd.concat(feature_list, ignore_index=True)


print("✅ Combined feature dataframe:")
print(features_df)
features_df.to_excel("all_patient_features.xlsx", index=False)



In [ ]:
# features.py

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch
from scipy.interpolate import interp1d
import sys
import os


project_root = "C:/Users/Visnu/DIAMONDS"  

if project_root not in sys.path:
    sys.path.append(project_root)

import diamonds.data as dt

from pan_tompkins import detect_ecg_peaks
from breath_detection import auto_detect_edr_breaths, DEFAULT_MIN_BREATH_SEC
from preprocessing import preprocess_signal, get_QRS_amplitude, get_RRI_from_signal, get_EDR_signal
from diamonds_definitions import pt, session, exercise, ecg_signal, fs

record = pt

filtered_ecg = preprocess_signal(ecg_signal, fs)
R_peaks, RRI, RRI_resampled =  get_RRI_from_signal(filtered_ecg, fs) 
QRS_amplitude, QRS_amplitude_resampled = get_QRS_amplitude(filtered_ecg, R_peaks, fs, default_window_len=0.1)


# --- 2.1 EDR-derived Breathing Features ---
def compute_mean_insp_exp_edr(in_times, ex_times, t_start, t_end):
    """
    Compute the mean inspiration and expiration intervals within a window.
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    valid_ex = ex_times[(ex_times >= t_start) & (ex_times <= t_end)]
    
    if len(valid_in) < 2:
        mean_insp_interval = np.nan
    else:
        mean_insp_interval = np.mean(np.diff(valid_in))
    
    if len(valid_ex) < 2:
        mean_exp_interval = np.nan
    else:
        mean_exp_interval = np.mean(np.diff(valid_ex))
        
    return mean_insp_interval, mean_exp_interval

def compute_breath_features(in_times, t_start, t_end):
    """
    Compute additional breath features within a window:
      - Number of breaths (count of valid inspirations)
      - Mean breath duration (average time between consecutive inspirations)
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    n_breaths = len(valid_in)
    if n_breaths >= 2:
        mean_breath_duration = np.mean(np.diff(valid_in))
    else:
        mean_breath_duration = np.nan
    return n_breaths, mean_breath_duration

def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Compute the breathing rate (BPM) from inspiration times within a window.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    mean_interval = np.mean(np.diff(valid))
    return 60.0 / mean_interval

# -- 2.2 ECG Heart Rate Analysis --

def compute_ecg_heart_rate(ecg_signal, fs):
    """
    Compute ECG heart rate from the filtered ECG signal.
    Requires preprocessing.detect_ecg_peaks to be defined.
    """
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        print("⚠ Not enough R-peaks to compute heart rate.")
        return np.array([]), np.array([]), np.nan

    rr_intervals = np.diff(R_peaks) / fs  
    inst_hr = 60.0 / rr_intervals 
    hr_time = 0.5 * (R_peaks[:-1] + R_peaks[1:]) / fs
    mean_hr = np.mean(inst_hr)
    return hr_time, inst_hr, mean_hr

def segment_ecg_hr(ecg_signal, fs, start_time, end_time, interval_size=10):
    """
    Segment ECG heart rate in intervals.
    """
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        return np.array([]), np.array([])

    seg_times = np.arange(start_time, end_time, interval_size)
    hr_values = []
    for t0 in seg_times:
        t1 = t0 + interval_size
        segment_peaks = R_peaks[(R_peaks/fs >= t0) & (R_peaks/fs < t1)]
        if len(segment_peaks) < 2:
            hr_values.append(np.nan)
        else:
            rr = np.diff(segment_peaks) / fs
            hr_values.append(60.0 / np.mean(rr))
    return seg_times, np.array(hr_values)

# -- 2.3 QRS Amplitude Statistics per Breath --

def compute_qrs_amp_per_breath(edr_time, edr_signal, in_times):
    """
    Compute QRS amplitude statistics for each breath cycle.
    Returns breath_times, mean QRS, standard deviation of QRS,
    and beat-to-beat variability (std of consecutive differences).
    """
    breath_times = []
    mean_qrs = []
    std_qrs = []
    btb_var = []
    for i in range(len(in_times) - 1):
        start = in_times[i]
        end = in_times[i+1]
        mask = (edr_time >= start) & (edr_time < end)
        segment = edr_signal[mask]
        breath_time = 0.5 * (start + end)
        if len(segment) < 2:
            breath_times.append(breath_time)
            mean_qrs.append(np.nan)
            std_qrs.append(np.nan)
            btb_var.append(np.nan)
        else:
            mean_val = np.mean(segment)
            std_val = np.std(segment)
            diffs = np.diff(segment)
            btb_val = np.std(diffs) if len(diffs) > 0 else np.nan
            breath_times.append(breath_time)
            mean_qrs.append(mean_val)
            std_qrs.append(std_val)
            btb_var.append(btb_val)
    return (np.array(breath_times),
            np.array(mean_qrs),
            np.array(std_qrs),
            np.array(btb_var))

# -- 2.4 Plotting Function for EDR Breath Detection --

def plot_edr_breaths(edr_time, edr_signal, in_times, ex_times):
    """
    Plot the EDR signal with detected inspiration and expiration points.
    """
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal (QRS Amplitude)')
    plt.plot(in_times, np.interp(in_times, edr_time, edr_signal), 'o', label='Inspirations')
    plt.plot(ex_times, np.interp(ex_times, edr_time, edr_signal), 'o', label='Expirations')
    plt.title('ECG-Derived Respiration with Detected Breaths')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

#########################################
# SECTION 3: FEATURE TABLE CONSTRUCTION
#########################################

def build_feature_table(patient, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10):
    exercises = session[dt.Exercise]  # ✅ This is the correct way to access Exercise annotations



    all_ex_timestamps = exercises.timestamp
    full_start = float(np.min(all_ex_timestamps))
    full_end = float(np.max(all_ex_timestamps))
    print(f"build_feature_table: Using exercise interval {full_start:.1f}–{full_end:.1f} s")

    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]

    # (A) Breath detection on the EDR signal
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    # (B) Detect ECG R-peaks (ensure preprocessing.detect_ecg_peaks is defined)
    R_peaks = detect_ecg_peaks(ecg_signal, fs)

    feature_rows = []
    seg_times = np.arange(full_start, full_end, interval_size)
    
    # Pre-compute QRS amplitude per breath stats (used for BTB variability)
    btqrs_times, mean_qrs_breath, std_qrs_breath, btb_var_breath = compute_qrs_amp_per_breath(edr_time_qrs, edr_signal_qrs, in_times)
    
    for window_start in seg_times:
        window_end = window_start + interval_size

        # 1. EDR-derived breathing rate (BPM)
        edr_bpm = compute_bpm_from_inspiration_times(in_times, window_start, window_end)
        cycle_duration = 60.0 / edr_bpm if not np.isnan(edr_bpm) else np.nan

        # 2. Mean inspiration and expiration durations
        mean_insp, mean_exp = compute_mean_insp_exp_edr(in_times, ex_times, window_start, window_end)

        # 3. Breath count and mean breath duration (from inspirations)
        n_breaths, mean_breath_duration = compute_breath_features(in_times, window_start, window_end)

        # 4. ECG-derived heart rate (BPM) from R-peaks in the window
        window_peaks = R_peaks[(R_peaks/fs >= window_start) & (R_peaks/fs < window_end)]
        if len(window_peaks) < 2:
            ecg_bpm = np.nan
        else:
            rr = np.diff(window_peaks) / fs
            ecg_bpm = 60.0 / np.mean(rr)

        # 5. QRS amplitude statistics from the EDR signal in the window
        mask_qrs = (edr_time_qrs >= window_start) & (edr_time_qrs < window_end)
        qrs_window = edr_signal_qrs[mask_qrs]
        if len(qrs_window) < 1:
            qrs_mean = np.nan
            qrs_std = np.nan
        else:
            qrs_mean = np.mean(qrs_window)
            qrs_std = np.std(qrs_window)
        
        # 6. Beat-to-beat variability (BTB) of QRS amplitude for breaths in the window
        mask_btb = (btqrs_times >= window_start) & (btqrs_times < window_end)
        if np.any(mask_btb):
            qrs_btb = np.nanmean(btb_var_breath[mask_btb])
        else:
            qrs_btb = np.nan

        row = {
            "subject_id": patient.id,
            "window_start": window_start,
            "window_end": window_end,
            "EDR_BPM": edr_bpm,
            "BreathingCycleDuration": cycle_duration,
            "MeanInsp": mean_insp,
            "MeanExp": mean_exp,
            "n_breaths": n_breaths,
            "MeanBreathDuration": mean_breath_duration,
            "ECG_BPM": ecg_bpm,
            "QRS_mean": qrs_mean,
            "QRS_std": qrs_std,
            "QRS_BTB": qrs_btb
        }
        feature_rows.append(row)
    
    feature_df = pd.DataFrame(feature_rows)
    return feature_df


def main():

    # --- ECG Heart Rate Analysis ---
    hr_time, inst_hr, mean_hr = compute_ecg_heart_rate(filtered_ecg, fs)
    print(f"Heart Rate from ECG: {mean_hr:.2f} BPM (average)")
    
    plt.figure(figsize=(8, 4))
    plt.plot(hr_time, inst_hr, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    session = pt[dt.SeatedSession]  # or SupineSession depending on your use case
    exercise_ann = session[dt.Exercise]

    seg_times_hr, hr_10s = segment_ecg_hr(
        filtered_ecg,
        fs,
        start_time=np.min(exercise_ann.timestamp),
        end_time=np.max(exercise_ann.timestamp),
        interval_size=10
        )

    print("Segmented HR (10s intervals):", hr_10s)
    
    plt.figure(figsize=(8, 4))
    plt.plot(seg_times_hr, hr_10s, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate (10s Intervals)')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # --- SECTION 1: EDR Breath Detection ---
    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]
    
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    
    interval_size = 10  # 10-second samples for segmentation here
    seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio = [], [], [], []
    seg_times = np.arange(np.min(exercise.timestamp), np.max(exercise.timestamp), interval_size)

    for t0 in seg_times:
        t1 = t0 + interval_size
        mi, me = compute_mean_insp_exp_edr(in_times, ex_times, t0, t1)
        seg_times_edr.append(t0)
        edr_mean_insp.append(mi)
        edr_mean_exp.append(me)
        ratio = mi/me if (not np.isnan(mi) and not np.isnan(me) and me > 0) else np.nan
        edr_ie_ratio.append(ratio)
    
    print("=== EDR-derived Mean Intervals (10s windows) ===")
    for t0, mi, me, ratio in zip(seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio):
        print(f"[{t0:.0f}-{t0+interval_size:.0f}s]: Insp={mi:.2f}s, Exp={me:.2f}s, I:E={ratio:.2f}")
    
    plot_edr_breaths(edr_time_qrs, edr_signal_qrs, in_times, ex_times)
    
    # --- Build Feature Table ---
    feature_df = build_feature_table(record, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10)
    print("Feature Table:")
    print(feature_df)
    
    # 2) Create a label for each time window (e.g., "0–30s", "30–60s", etc.)
    labels = [
        f"{int(ws)}–{int(we)}s"
        for ws, we in zip(feature_df["window_start"], feature_df["window_end"])
    ]

    
    # --- Additional Plots for Feature Analysis ---
    # Calculate mid-time for each window from the feature table (using window length 30s)
    mid_times = feature_df['window_start'] + (30 / 2)

    # Plot: Mean Inspiration and Expiration Durations
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['MeanInsp'], marker='o', linestyle='-', label='Mean Inspiration Duration')
    plt.plot(mid_times, feature_df['MeanExp'], marker='o', linestyle='-', label='Mean Expiration Duration')
    plt.title('Mean Inspiration and Expiration Durations')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot: Number of Breaths per Window
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['n_breaths'], marker='o', linestyle='-', label='Number of Breaths')
    plt.title('Number of Breaths (per 10s window)')
    plt.xlabel('Time (s)')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot: Mean Breath Duration vs. Breathing Cycle Duration
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['MeanBreathDuration'], marker='o', linestyle='-', label='Mean Breath Duration')
    plt.title('Mean Breath Duration')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # Plot: Breathing Rate per Minute
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['EDR_BPM'], marker='o', linestyle='-', label='Breathing Rate (BPM)')
    plt.title('Breathing Rate per Minute')
    plt.xlabel('Time (s)')
    plt.ylabel('BPM')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 3) Bar Plot: Breathing Rate per Minute (EDR_BPM)
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["EDR_BPM"], color="royalblue", alpha=0.7)
    plt.title("Breathing Rate per Minute (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("BPM")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

    # 4) Bar Plot: Mean Breath Duration
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["MeanBreathDuration"], color="seagreen", alpha=0.7)
    plt.title("Mean Breath Duration (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("Duration (s)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()
    
    # --- Additional Plots for QRS Amplitude Statistics ---
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_mean'], marker='o', linestyle='-', label='Mean QRS per Breath')
    plt.title('Mean QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Mean QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_std'], marker='o', linestyle='-', color='orange', label='Std of QRS per Breath')
    plt.title('Std of QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Std QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_BTB'], marker='o', linestyle='-', color='green', label='Beat-to-beat Variability')
    plt.title('Beat-to-beat Variability of QRS Amplitude')
    plt.xlabel('Time (s)')
    plt.ylabel('Std of Consecutive Differences')
    plt.grid(True)
    plt.legend()
    plt.show()


if __name__ == "__main__":
    main()


In [ ]:
import importlib
import preprocessing
import pan_tompkins
import engzee_tompkins
import new_rpeak_detector

importlib.reload(preprocessing)
importlib.reload(pan_tompkins)
importlib.reload(engzee_tompkins)
importlib.reload(new_rpeak_detector)

# Choose the method you want:
method_choice = "pan_tompkins"   # or "engzee" or "neurokit"

# Extract features:
results = preprocessing.extract_ecg_features(
    ecg_signal,
    fs,
    method=method_choice,
    RRI_fs=10,
    default_window_len=0.1,
    search_window_len=0.01,
    edr_method='R_peak'
)

filtered_ecg = results["filtered_ecg"]
R_peaks       = results["R_peaks"]
RRI           = results["RRI"]
RRI_resampled = results["RRI_resampled"]
QRS_amplitude = results["QRS_amp"]
QRS_amplitude_resampled = results["QRS_amp_resampled"]
BW_effect     = results["BW_effect"]
AM_effect     = results["AM_effect"]
R_peak_amplitude = results["R_peak_amplitude"]
R_peak_amplitude_resampled = results["R_peak_amplitude_resampled"]
edr_time_r_peak = results["edr_time_r_peak"]
edr_signal_r_peak = results["edr_signal_r_peak"]
edr_rri_time  = results["edr_rri_time"]
edr_rri_signal = results["edr_rri_signal"]

print(f"Method used: {method_choice}")
print(f"Detected {len(R_peaks)} R-peaks")

# Compute the combined EDR signal from the features:
from preprocessing import get_combined_signal_from_features
combined_edr_time, combined_edr_signal = get_combined_signal_from_features(results, resampled_fs=10)

# Then plot everything:
import interpolated_signals_view
importlib.reload(interpolated_signals_view)
from interpolated_signals_view import plot_interpolated_signals

plot_interpolated_signals(
    ecg_signal=ecg_signal,
    filtered_ecg=filtered_ecg,
    R_peaks=R_peaks,
    edr_time_r_peak=edr_time_r_peak,
    edr_signal_r_peak=edr_signal_r_peak,
    RRI=RRI,
    RRI_resampled=RRI_resampled,
    QRS_amplitude=QRS_amplitude,
    QRS_amplitude_resampled=QRS_amplitude_resampled,
    R_peak_amplitude=R_peak_amplitude,
    R_peak_amplitude_resampled=R_peak_amplitude_resampled,
    edr_rri_time=edr_rri_time,
    edr_rri_signal=edr_rri_signal,
    BW_effect=BW_effect,
    AM_effect=AM_effect,
    combined_edr_time=combined_edr_time,     
    combined_edr_signal=combined_edr_signal,    
    fs=fs
) 




# breath dteetction and feature code with old meanisnp, meanexp and rreathing cycle duration:

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import find_peaks, butter, filtfilt, iirnotch
from scipy.stats import skew, kurtosis
from scipy.interpolate import interp1d
from pan_tompkins import detect_ecg_peaks

# Global constant for minimum breath separation (in seconds)
DEFAULT_MIN_BREATH_SEC = 2.5
record = pt

#########################################
# SECTION 1: BREATH DETECTION FUNCTIONS
#########################################

def detect_edr_breaths(edr_time, edr_signal, min_breath_sec=DEFAULT_MIN_BREATH_SEC):
    """
    Detect inspiration and expiration breaths from the EDR signal.
    """
    min_distance_samples = int(10 * min_breath_sec)
    in_peaks, _ = find_peaks(-edr_signal, distance=min_distance_samples)
    ex_peaks, _ = find_peaks(edr_signal, distance=min_distance_samples)
    in_times = edr_time[in_peaks]
    ex_times = edr_time[ex_peaks]
    return in_times, ex_times

def auto_detect_edr_breaths(edr_time, edr_signal, init_min_breath_sec=2.5,
                            adjust_factor=0.5, max_iterations=5, tol=0.05,
                            min_allowed=0.5, max_allowed=10.0):
    """
    Automatically adapts min_breath_sec with multiple passes until stable.
    """
    current_min_breath_sec = init_min_breath_sec

    for iteration in range(max_iterations):
        in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
        all_breaths = np.sort(np.concatenate([in_times, ex_times]))
        if len(all_breaths) < 2:
            print(f"Not enough breath events (iteration {iteration+1}). Stopping.")
            return in_times, ex_times
        
        # Compute median interval (with outlier rejection)
        breath_intervals = np.diff(all_breaths)
        median_rough = np.median(breath_intervals)
        upper_cut = 2.0 * median_rough
        lower_cut = 0.2 * median_rough
        cleaned_intervals = breath_intervals[(breath_intervals <= upper_cut) & (breath_intervals >= lower_cut)]
        median_interval = np.median(cleaned_intervals) if len(cleaned_intervals) >= 2 else median_rough

        new_min_breath_sec = adjust_factor * median_interval
        new_min_breath_sec = np.clip(new_min_breath_sec, min_allowed, max_allowed)
        ratio = new_min_breath_sec / current_min_breath_sec
        print(f"Iteration {iteration+1}: old={current_min_breath_sec:.2f}, new={new_min_breath_sec:.2f} (ratio={ratio:.2f})")
        if abs(ratio - 1.0) < tol:
            current_min_breath_sec = new_min_breath_sec
            break
        else:
            current_min_breath_sec = new_min_breath_sec

    # Final detection pass
    in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
    print(f"Final min_breath_sec after {iteration+1} iteration(s): {current_min_breath_sec:.2f}")
    return in_times, ex_times

#########################################
# SECTION 2: FEATURE ANALYSIS FUNCTIONS
#########################################

# -- 2.1 EDR-derived Breathing Features --

def compute_mean_insp_exp_edr(in_times, ex_times, t_start, t_end):
    """
    Compute the mean inspiration and expiration intervals within a window.
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    valid_ex = ex_times[(ex_times >= t_start) & (ex_times <= t_end)]
    
    if len(valid_in) < 2:
        mean_insp_interval = np.nan
    else:
        mean_insp_interval = np.mean(np.diff(valid_in))
    
    if len(valid_ex) < 2:
        mean_exp_interval = np.nan
    else:
        mean_exp_interval = np.mean(np.diff(valid_ex))
        
    return mean_insp_interval, mean_exp_interval

def compute_true_insp_exp_durations(in_times, ex_times, t_start, t_end):
    insp_durations = []
    exp_durations = []

    # Ex → In = True Inspiration
    for e_time in ex_times:
        if e_time < t_start or e_time > t_end:
            continue
        next_in = in_times[in_times > e_time]
        if next_in.size > 0 and next_in[0] <= t_end:
            insp_durations.append(next_in[0] - e_time)

    # In → Ex = True Expiration
    for i_time in in_times:
        if i_time < t_start or i_time > t_end:
            continue
        next_ex = ex_times[ex_times > i_time]
        if next_ex.size > 0 and next_ex[0] <= t_end:
            exp_durations.append(next_ex[0] - i_time)

    return (
        np.mean(insp_durations) if insp_durations else np.nan,
        np.mean(exp_durations) if exp_durations else np.nan
    )



def compute_breath_features(in_times, t_start, t_end):
    """
    Compute additional breath features within a window:
      - Number of breaths (count of valid inspirations)
      - Mean breath duration (average time between consecutive inspirations)
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    n_breaths = len(valid_in)
    if n_breaths >= 2:
        mean_breath_duration = np.mean(np.diff(valid_in))
    else:
        mean_breath_duration = np.nan
    return n_breaths, mean_breath_duration

def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Compute the breathing rate (BPM) from inspiration times within a window.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    mean_interval = np.mean(np.diff(valid))
    return 60.0 / mean_interval

# -- 2.2 ECG Heart Rate Analysis --

def compute_ecg_heart_rate(ecg_signal, fs):
    """
    Compute ECG heart rate from the filtered ECG signal.
    Requires pan_tompkins.detect_ecg_peaks to be defined.
    """
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        print("⚠ Not enough R-peaks to compute heart rate.")
        return np.array([]), np.array([]), np.nan

    rr_intervals = np.diff(R_peaks) / fs  
    inst_hr = 60.0 / rr_intervals 
    hr_time = 0.5 * (R_peaks[:-1] + R_peaks[1:]) / fs
    mean_hr = np.mean(inst_hr)
    return hr_time, inst_hr, mean_hr

def segment_ecg_hr(ecg_signal, fs, start_time, end_time, interval_size=10):
    """
    Segment ECG heart rate in intervals.
    """
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        return np.array([]), np.array([])

    seg_times = np.arange(start_time, end_time, interval_size)
    hr_values = []
    for t0 in seg_times:
        t1 = t0 + interval_size
        segment_peaks = R_peaks[(R_peaks/fs >= t0) & (R_peaks/fs < t1)]
        if len(segment_peaks) < 2:
            hr_values.append(np.nan)
        else:
            rr = np.diff(segment_peaks) / fs
            hr_values.append(60.0 / np.mean(rr))
    return seg_times, np.array(hr_values)

# -- 2.3 QRS Amplitude Statistics per Breath --

def compute_qrs_amp_per_breath(edr_time, edr_signal, in_times):
    """
    Compute QRS amplitude statistics for each breath cycle.
    Returns breath_times, mean QRS, standard deviation of QRS,
    and beat-to-beat variability (std of consecutive differences).
    """
    breath_times = []
    mean_qrs = []
    std_qrs = []
    btb_var = []
    for i in range(len(in_times) - 1):
        start = in_times[i]
        end = in_times[i+1]
        mask = (edr_time >= start) & (edr_time < end)
        segment = edr_signal[mask]
        breath_time = 0.5 * (start + end)
        if len(segment) < 2:
            breath_times.append(breath_time)
            mean_qrs.append(np.nan)
            std_qrs.append(np.nan)
            btb_var.append(np.nan)
        else:
            mean_val = np.mean(segment)
            std_val = np.std(segment)
            diffs = np.diff(segment)
            btb_val = np.std(diffs) if len(diffs) > 0 else np.nan
            breath_times.append(breath_time)
            mean_qrs.append(mean_val)
            std_qrs.append(std_val)
            btb_var.append(btb_val)
    return (np.array(breath_times),
            np.array(mean_qrs),
            np.array(std_qrs),
            np.array(btb_var))

def compute_peak_to_trough(edr_time, edr_signal, t_start, t_end):
    """Compute the peak-to-trough difference in the EDR signal within [t_start, t_end]."""
    mask = (edr_time >= t_start) & (edr_time <= t_end)
    segment = edr_signal[mask]
    if segment.size == 0:
        return np.nan
    return np.max(segment) - np.min(segment)

def compute_rising_slope(edr_time, edr_signal, t_start, t_end):
    """
    Compute the rising slope of the EDR signal in the window.
    We estimate the slope from t_start to the time of the maximum amplitude in [t_start, t_end].
    """
    mask = (edr_time >= t_start) & (edr_time <= t_end)
    seg_time = edr_time[mask]
    seg_signal = edr_signal[mask]
    if seg_time.size < 2:
        return np.nan
    # Identify time of the peak (maximum)
    peak_index = np.argmax(seg_signal)
    t_peak = seg_time[peak_index]
    # Use the segment from t_start to t_peak for linear regression (polyfit of degree 1)
    mask_rise = (seg_time >= t_start) & (seg_time <= t_peak)
    if np.sum(mask_rise) < 2:
        return np.nan
    rise_time = seg_time[mask_rise]
    rise_signal = seg_signal[mask_rise]
    # Linear fit: slope is the first coefficient
    slope, _ = np.polyfit(rise_time, rise_signal, 1)
    return slope

def compute_area_under_curve(edr_time, edr_signal, t_start, t_end):
    """Compute the area under the EDR signal curve in the given window using the trapezoidal rule."""
    mask = (edr_time >= t_start) & (edr_time <= t_end)
    seg_time = edr_time[mask]
    seg_signal = edr_signal[mask]
    if seg_time.size < 2:
        return np.nan
    return np.trapz(seg_signal, seg_time)




# -- 2.4 Plotting Function for EDR Breath Detection --

def plot_edr_breaths(edr_time, edr_signal, in_times, ex_times):
    """
    Plot the EDR signal with detected inspiration and expiration points.
    """
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal (QRS Amplitude)')
    plt.plot(in_times, np.interp(in_times, edr_time, edr_signal), 'o', label='Inspirations')
    plt.plot(ex_times, np.interp(ex_times, edr_time, edr_signal), 'o', label='Expirations')
    plt.title('ECG-Derived Respiration with Detected Breaths')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

#########################################
# SECTION 3: FEATURE TABLE CONSTRUCTION
#########################################

def build_feature_table(patient, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10):
    exercises = session[dt.Exercise]
    all_ex_timestamps = exercises.timestamp
    full_start = float(np.min(all_ex_timestamps))
    full_end = float(np.max(all_ex_timestamps))
    print(f"build_feature_table: Using exercise interval {full_start:.1f}–{full_end:.1f} s")

    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]

    # Breath detection on the EDR signal
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    # Detect ECG R-peaks
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    
    # Compute ECG HR for RSA calculation (for the entire session)
    hr_time, inst_hr, _ = compute_ecg_heart_rate(ecg_signal, fs)

    feature_rows = []
    seg_times = np.arange(full_start, full_end, interval_size)
    
    # Pre-compute QRS amplitude per breath stats for BTB variability
    btqrs_times, mean_qrs_breath, std_qrs_breath, btb_var_breath = compute_qrs_amp_per_breath(edr_time_qrs, edr_signal_qrs, in_times)
    
    for window_start in seg_times:
        window_end = window_start + interval_size

        # Compute "true" inspiration and expiration durations (if available)
        true_insp_dur, true_exp_dur = compute_true_insp_exp_durations(in_times, ex_times, window_start, window_end)
        
        # EDR-derived breathing rate (BPM)
        edr_bpm = compute_bpm_from_inspiration_times(in_times, window_start, window_end)
        cycle_duration = 60.0 / edr_bpm if not np.isnan(edr_bpm) else np.nan

        # Mean inspiration and expiration durations (using existing method)
        mean_insp, mean_exp = compute_mean_insp_exp_edr(in_times, ex_times, window_start, window_end)
        
        # Breath count and mean breath duration
        n_breaths, mean_breath_duration = compute_breath_features(in_times, window_start, window_end)

        # ECG-derived heart rate (BPM) in the window
        window_peaks = R_peaks[(R_peaks/fs >= window_start) & (R_peaks/fs < window_end)]
        if len(window_peaks) < 2:
            ecg_bpm = np.nan
        else:
            rr = np.diff(window_peaks) / fs
            ecg_bpm = 60.0 / np.mean(rr)
        
        # QRS amplitude statistics in the window
        mask_qrs = (edr_time_qrs >= window_start) & (edr_time_qrs < window_end)
        qrs_window = edr_signal_qrs[mask_qrs]
        if len(qrs_window) < 1:
            qrs_mean = np.nan
            qrs_std = np.nan
        else:
            qrs_mean = np.mean(qrs_window)
            qrs_std = np.std(qrs_window)
        
        # Beat-to-beat variability (BTB) of QRS amplitude for breaths in the window
        mask_btb = (btqrs_times >= window_start) & (btqrs_times < window_end)
        qrs_btb = np.nanmean(btb_var_breath[mask_btb]) if np.any(mask_btb) else np.nan
        
        # ---- NEW METRICS ----
        # 1. Peak-to-Trough Difference of EDR signal
        pt_diff = compute_peak_to_trough(edr_time_qrs, edr_signal_qrs, window_start, window_end)
        
        # 2. Rising Slope: compute slope from window start to the time of maximum amplitude
        mask_window = (edr_time_qrs >= window_start) & (edr_time_qrs < window_end)
        if np.any(mask_window):
            seg_time = edr_time_qrs[mask_window]
            seg_signal = edr_signal_qrs[mask_window]
            peak_index = np.argmax(seg_signal)
            t_peak = seg_time[peak_index]
            rising_slope = compute_rising_slope(edr_time_qrs, edr_signal_qrs, window_start, t_peak)
        else:
            rising_slope = np.nan
        
        # 3. Area Under the EDR Curve in the window
        area = compute_area_under_curve(edr_time_qrs, edr_signal_qrs, window_start, window_end)
        
        row = {
            "subject_id": patient.id,
            "window_start": window_start,
            "window_end": window_end,
            "EDR_BPM": edr_bpm,
            "BreathingCycleDuration": cycle_duration,
            "MeanInsp": mean_insp,
            "MeanExp": mean_exp,
            "n_breaths": n_breaths,
            "MeanBreathDuration": mean_breath_duration,
            "ECG_BPM": ecg_bpm,
            "QRS_mean": qrs_mean,
            "QRS_std": qrs_std,
            "QRS_BTB": qrs_btb,
            "TrueInspDuration": true_insp_dur,
            "TrueExpDuration": true_exp_dur,
            "PeakToTrough": pt_diff,
            "RisingSlope": rising_slope,
            "AreaUnderCurve": area,
        }
        feature_rows.append(row)
    
    feature_df = pd.DataFrame(feature_rows)
    return feature_df



#########################################
# SECTION 4: MAIN EXECUTION & PLOTTING
#########################################

if __name__ == "__main__":
    # NOTE: Ensure that the following variables are defined in your environment:
    #   record, ecg_signal, filtered_ecg, fs, QRS_amplitude_resampled, preprocessing
    
    # --- ECG Heart Rate Analysis (for plotting) ---
    hr_time, inst_hr, mean_hr = compute_ecg_heart_rate(filtered_ecg, fs)
    print(f"Heart Rate from ECG: {mean_hr:.2f} BPM (average)")
    
    plt.figure(figsize=(8, 4))
    plt.plot(hr_time, inst_hr, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    session = pt[dt.SeatedSession]  # or SupineSession depending on your use case
    exercise_ann = session[dt.Exercise]

    seg_times_hr, hr_10s = segment_ecg_hr(
        filtered_ecg,
        fs,
        start_time=np.min(exercise_ann.timestamp),
        end_time=np.max(exercise_ann.timestamp),
        interval_size=10
        )

    print("Segmented HR (10s intervals):", hr_10s)
    
    plt.figure(figsize=(8, 4))
    plt.plot(seg_times_hr, hr_10s, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate (10s Intervals)')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # --- SECTION 1: EDR Breath Detection ---
    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]
    
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    
    interval_size = 10  # 10-second samples for segmentation here
    seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio = [], [], [], []
    seg_times = np.arange(np.min(exercise.timestamp), np.max(exercise.timestamp), interval_size)

    for t0 in seg_times:
        t1 = t0 + interval_size
        mi, me = compute_mean_insp_exp_edr(in_times, ex_times, t0, t1)
        seg_times_edr.append(t0)
        edr_mean_insp.append(mi)
        edr_mean_exp.append(me)
        ratio = mi/me if (not np.isnan(mi) and not np.isnan(me) and me > 0) else np.nan
        edr_ie_ratio.append(ratio)
    
    print("=== EDR-derived Mean Intervals (10s windows) ===")
    for t0, mi, me, ratio in zip(seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio):
        print(f"[{t0:.0f}-{t0+interval_size:.0f}s]: Insp={mi:.2f}s, Exp={me:.2f}s, I:E={ratio:.2f}")
    
    plot_edr_breaths(edr_time_qrs, edr_signal_qrs, in_times, ex_times)
    
    # --- Build Feature Table ---
    feature_df = build_feature_table(record, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10)
    print("Feature Table:")
    print(feature_df)
    
    # 2) Create a label for each time window (e.g., "0–30s", "30–60s", etc.)
    labels = [
        f"{int(ws)}–{int(we)}s"
        for ws, we in zip(feature_df["window_start"], feature_df["window_end"])
    ]

    
    # --- Additional Plots for Feature Analysis ---
    # Calculate mid-time for each window from the feature table (using window length 30s)
    mid_times = feature_df['window_start'] + (30 / 2)

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['TrueInspDuration'], marker='o', linestyle='-', label='True Insp Duration (In→Ex)')
    plt.plot(mid_times, feature_df['TrueExpDuration'], marker='o', linestyle='-', label='True Exp Duration (Ex→In)')
    plt.title('True Inspiration and Expiration Durations')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot: Mean Inspiration and Expiration Durations
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['MeanInsp'], marker='o', linestyle='-', label='Mean Inspiration Duration')
    plt.plot(mid_times, feature_df['MeanExp'], marker='o', linestyle='-', label='Mean Expiration Duration')
    plt.title('Mean Inspiration and Expiration Durations')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot: Number of Breaths per Window
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['n_breaths'], marker='o', linestyle='-', label='Number of Breaths')
    plt.title('Number of Breaths (per 10s window)')
    plt.xlabel('Time (s)')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot: Mean Breath Duration vs. Breathing Cycle Duration
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['MeanBreathDuration'], marker='o', linestyle='-', label='Mean Breath Duration')
    plt.title('Mean Breath Duration')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # Plot: Breathing Rate per Minute
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['EDR_BPM'], marker='o', linestyle='-', label='Breathing Rate (BPM)')
    plt.title('Breathing Rate per Minute')
    plt.xlabel('Time (s)')
    plt.ylabel('BPM')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 3) Bar Plot: Breathing Rate per Minute (EDR_BPM)
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["EDR_BPM"], color="royalblue", alpha=0.7)
    plt.title("Breathing Rate per Minute (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("BPM")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

    # 4) Bar Plot: Mean Breath Duration
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["MeanBreathDuration"], color="seagreen", alpha=0.7)
    plt.title("Mean Breath Duration (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("Duration (s)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()
    
    # --- Additional Plots for QRS Amplitude Statistics ---
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_mean'], marker='o', linestyle='-', label='Mean QRS per Breath')
    plt.title('Mean QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Mean QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_std'], marker='o', linestyle='-', color='orange', label='Std of QRS per Breath')
    plt.title('Std of QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Std QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_BTB'], marker='o', linestyle='-', color='green', label='Beat-to-beat Variability')
    plt.title('Beat-to-beat Variability of QRS Amplitude')
    plt.xlabel('Time (s)')
    plt.ylabel('Std of Consecutive Differences')
    plt.grid(True)
    plt.legend()
    plt.show()

    import matplotlib.pyplot as plt

    # Suppose feature_df is already built
    # Plot Peak-to-Trough
    plt.figure(figsize=(10, 6))
    plt.plot(feature_df['window_start'], feature_df['PeakToTrough'], '-o', label='Peak-to-Trough')
    plt.title("EDR Signal Peak-to-Trough Difference")
    plt.xlabel("Window Start Time (s)")
    plt.ylabel("Amplitude Difference")
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot Rising Slope
    plt.figure(figsize=(10, 6))
    plt.plot(feature_df['window_start'], feature_df['RisingSlope'], '-o', color='orange', label='Rising Slope')
    plt.title("EDR Signal Rising Slope")
    plt.xlabel("Window Start Time (s)")
    plt.ylabel("Slope (Amplitude/sec)")
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot Area Under Curve
    plt.figure(figsize=(10, 6))
    plt.plot(feature_df['window_start'], feature_df['AreaUnderCurve'], '-o', color='green', label='Area Under Curve')
    plt.title("Area Under the EDR Curve")
    plt.xlabel("Window Start Time (s)")
    plt.ylabel("Area (integral units)")
    plt.grid(True)
    plt.legend()
    plt.show()




# breah segmentation and featrue code for combined signal:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import find_peaks, butter, filtfilt, iirnotch
from scipy.stats import skew, kurtosis
from scipy.interpolate import interp1d
from pan_tompkins import detect_ecg_peaks
import diamonds.data as dt
from diamonds import load_patients

# Global constant for minimum breath separation (in seconds)
DEFAULT_MIN_BREATH_SEC = 2.5

# Assume 'pt' is defined (e.g., the first patient) or use record = pt
record = pt  # <-- Make sure that 'pt' is defined in your environment

#########################################
# SECTION 1: BREATH DETECTION FUNCTIONS
#########################################

def detect_edr_breaths(edr_time, edr_signal, min_breath_sec=DEFAULT_MIN_BREATH_SEC):
    """
    Detect inspiration and expiration breaths from the EDR signal.
    """
    min_distance_samples = int(10 * min_breath_sec)
    in_peaks, _ = find_peaks(-edr_signal, distance=min_distance_samples)
    ex_peaks, _ = find_peaks(edr_signal, distance=min_distance_samples)
    in_times = edr_time[in_peaks]
    ex_times = edr_time[ex_peaks]
    return in_times, ex_times

def auto_detect_edr_breaths(edr_time, edr_signal, init_min_breath_sec=2.5,
                            adjust_factor=0.5, max_iterations=5, tol=0.05,
                            min_allowed=0.5, max_allowed=10.0):
    """
    Automatically adapts min_breath_sec with multiple passes until stable.
    """
    current_min_breath_sec = init_min_breath_sec

    for iteration in range(max_iterations):
        in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
        all_breaths = np.sort(np.concatenate([in_times, ex_times]))
        if len(all_breaths) < 2:
            print(f"Not enough breath events (iteration {iteration+1}). Stopping.")
            return in_times, ex_times
        
        # Compute median interval (with outlier rejection)
        breath_intervals = np.diff(all_breaths)
        median_rough = np.median(breath_intervals)
        upper_cut = 2.0 * median_rough
        lower_cut = 0.2 * median_rough
        cleaned_intervals = breath_intervals[(breath_intervals <= upper_cut) & (breath_intervals >= lower_cut)]
        median_interval = np.median(cleaned_intervals) if len(cleaned_intervals) >= 2 else median_rough

        new_min_breath_sec = adjust_factor * median_interval
        new_min_breath_sec = np.clip(new_min_breath_sec, min_allowed, max_allowed)
        ratio = new_min_breath_sec / current_min_breath_sec
        print(f"Iteration {iteration+1}: old={current_min_breath_sec:.2f}, new={new_min_breath_sec:.2f} (ratio={ratio:.2f})")
        if abs(ratio - 1.0) < tol:
            current_min_breath_sec = new_min_breath_sec
            break
        else:
            current_min_breath_sec = new_min_breath_sec

    # Final detection pass
    in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
    print(f"Final min_breath_sec after {iteration+1} iteration(s): {current_min_breath_sec:.2f}")
    return in_times, ex_times

#########################################
# SECTION 2: FEATURE ANALYSIS FUNCTIONS
#########################################

# -- 2.1 EDR-derived Breathing Features --

def compute_mean_insp_exp_edr(in_times, ex_times, t_start, t_end):
    """
    Compute the mean inspiration and expiration intervals within a window.
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    valid_ex = ex_times[(ex_times >= t_start) & (ex_times <= t_end)]
    
    if len(valid_in) < 2:
        mean_insp_interval = np.nan
    else:
        mean_insp_interval = np.mean(np.diff(valid_in))
    
    if len(valid_ex) < 2:
        mean_exp_interval = np.nan
    else:
        mean_exp_interval = np.mean(np.diff(valid_ex))
        
    return mean_insp_interval, mean_exp_interval

def compute_true_insp_exp_durations(in_times, ex_times, t_start, t_end):
    insp_durations = []
    exp_durations = []

    # Ex → In = True Inspiration
    for e_time in ex_times:
        if e_time < t_start or e_time > t_end:
            continue
        next_in = in_times[in_times > e_time]
        if next_in.size > 0 and next_in[0] <= t_end:
            insp_durations.append(next_in[0] - e_time)

    # In → Ex = True Expiration
    for i_time in in_times:
        if i_time < t_start or i_time > t_end:
            continue
        next_ex = ex_times[ex_times > i_time]
        if next_ex.size > 0 and next_ex[0] <= t_end:
            exp_durations.append(next_ex[0] - i_time)

    return (
        np.mean(insp_durations) if insp_durations else np.nan,
        np.mean(exp_durations) if exp_durations else np.nan
    )



def compute_breath_features(in_times, t_start, t_end):
    """
    Compute additional breath features within a window:
      - Number of breaths (count of valid inspirations)
      - Mean breath duration (average time between consecutive inspirations)
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    n_breaths = len(valid_in)
    if n_breaths >= 2:
        mean_breath_duration = np.mean(np.diff(valid_in))
    else:
        mean_breath_duration = np.nan
    return n_breaths, mean_breath_duration

def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Compute the breathing rate (BPM) from inspiration times within a window.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    mean_interval = np.mean(np.diff(valid))
    return 60.0 / mean_interval

# -- 2.2 ECG Heart Rate Analysis --

def compute_ecg_heart_rate(ecg_signal, fs):
    """
    Compute ECG heart rate from the filtered ECG signal.
    Requires pan_tompkins.detect_ecg_peaks to be defined.
    """
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        print("⚠ Not enough R-peaks to compute heart rate.")
        return np.array([]), np.array([]), np.nan

    rr_intervals = np.diff(R_peaks) / fs  
    inst_hr = 60.0 / rr_intervals 
    hr_time = 0.5 * (R_peaks[:-1] + R_peaks[1:]) / fs
    mean_hr = np.mean(inst_hr)
    return hr_time, inst_hr, mean_hr

def segment_ecg_hr(ecg_signal, fs, start_time, end_time, interval_size=10):
    """
    Segment ECG heart rate in intervals.
    """
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        return np.array([]), np.array([])

    seg_times = np.arange(start_time, end_time, interval_size)
    hr_values = []
    for t0 in seg_times:
        t1 = t0 + interval_size
        segment_peaks = R_peaks[(R_peaks/fs >= t0) & (R_peaks/fs < t1)]
        if len(segment_peaks) < 2:
            hr_values.append(np.nan)
        else:
            rr = np.diff(segment_peaks) / fs
            hr_values.append(60.0 / np.mean(rr))
    return seg_times, np.array(hr_values)

# -- 2.3 QRS Amplitude Statistics per Breath --

def compute_qrs_amp_per_breath(edr_time, edr_signal, in_times):
    """
    Compute QRS amplitude statistics for each breath cycle.
    Returns breath_times, mean QRS, standard deviation of QRS,
    and beat-to-beat variability (std of consecutive differences).
    """
    breath_times = []
    mean_qrs = []
    std_qrs = []
    btb_var = []
    for i in range(len(in_times) - 1):
        start = in_times[i]
        end = in_times[i+1]
        mask = (edr_time >= start) & (edr_time < end)
        segment = edr_signal[mask]
        breath_time = 0.5 * (start + end)
        if len(segment) < 2:
            breath_times.append(breath_time)
            mean_qrs.append(np.nan)
            std_qrs.append(np.nan)
            btb_var.append(np.nan)
        else:
            mean_val = np.mean(segment)
            std_val = np.std(segment)
            diffs = np.diff(segment)
            btb_val = np.std(diffs) if len(diffs) > 0 else np.nan
            breath_times.append(breath_time)
            mean_qrs.append(mean_val)
            std_qrs.append(std_val)
            btb_var.append(btb_val)
    return (np.array(breath_times),
            np.array(mean_qrs),
            np.array(std_qrs),
            np.array(btb_var))

# -- 2.4 Plotting Function for EDR Breath Detection --

def plot_edr_breaths(edr_time, edr_signal, in_times, ex_times):
    """
    Plot the EDR signal with detected inspiration and expiration points.
    """
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal (QRS Amplitude)')
    plt.plot(in_times, np.interp(in_times, edr_time, edr_signal), 'o', label='Inspirations')
    plt.plot(ex_times, np.interp(ex_times, edr_time, edr_signal), 'o', label='Expirations')
    plt.title('ECG-Derived Respiration with Detected Breaths')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

#########################################
# SECTION 3: FEATURE TABLE CONSTRUCTION
#########################################

def build_feature_table(patient, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10):
    # Access the exercise annotations
    exercises = session[dt.Exercise]
    all_ex_timestamps = exercises.timestamp
    full_start = float(np.min(all_ex_timestamps))
    full_end = float(np.max(all_ex_timestamps))
    print(f"build_feature_table: Using exercise interval {full_start:.1f}–{full_end:.1f} s")

    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]

    # Breath detection on the EDR signal
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    

    
    # Detect ECG R-peaks (using pan_tompkins here)
    R_peaks = detect_ecg_peaks(ecg_signal, fs)

    feature_rows = []
    seg_times = np.arange(full_start, full_end, interval_size)
    
    # Pre-compute QRS amplitude per breath stats (for beat-to-beat variability)
    btqrs_times, mean_qrs_breath, std_qrs_breath, btb_var_breath = compute_qrs_amp_per_breath(edr_time_qrs, edr_signal_qrs, in_times)
    
    for window_start in seg_times:
        window_end = window_start + interval_size
        true_insp_dur, true_exp_dur = compute_true_insp_exp_durations(in_times, ex_times, window_start, window_end)


        # EDR-derived breathing rate (BPM)
        edr_bpm = compute_bpm_from_inspiration_times(in_times, window_start, window_end)

        # Breath count and mean breath duration (from inspirations)
        n_breaths, mean_breath_duration = compute_breath_features(in_times, window_start, window_end)

        # ECG-derived heart rate (BPM) from R-peaks in the window
        window_peaks = R_peaks[(R_peaks/fs >= window_start) & (R_peaks/fs < window_end)]
        if len(window_peaks) < 2:
            ecg_bpm = np.nan
        else:
            rr = np.diff(window_peaks) / fs
            ecg_bpm = 60.0 / np.mean(rr)

        # QRS amplitude statistics in the window
        mask_qrs = (edr_time_qrs >= window_start) & (edr_time_qrs < window_end)
        qrs_window = edr_signal_qrs[mask_qrs]
        if len(qrs_window) < 1:
            qrs_mean = np.nan
            qrs_std = np.nan
        else:
            qrs_mean = np.mean(qrs_window)
            qrs_std = np.std(qrs_window)
        
        # Beat-to-beat variability (BTB) of QRS amplitude for breaths in the window
        mask_btb = (btqrs_times >= window_start) & (btqrs_times < window_end)
        if np.any(mask_btb):
            qrs_btb = np.nanmean(btb_var_breath[mask_btb])
        else:
            qrs_btb = np.nan

        row = {
            "subject_id": patient.id,
            "window_start": window_start,
            "window_end": window_end,
            "EDR_BPM": edr_bpm,
            "n_breaths": n_breaths,
            "MeanBreathDuration": mean_breath_duration,
            "ECG_BPM": ecg_bpm,
            "QRS_mean": qrs_mean,
            "QRS_std": qrs_std,
            "QRS_BTB": qrs_btb,
            "TrueInspDuration": true_insp_dur,
            "TrueExpDuration": true_exp_dur

        }
        feature_rows.append(row)
    
    feature_df = pd.DataFrame(feature_rows)
    return feature_df

#########################################
# SECTION 4: MAIN EXECUTION & PLOTTING
#########################################

if __name__ == "__main__":
    # Ensure that record, ecg_signal, filtered_ecg, fs, etc. are defined.
    # For example, you might have:
    #   pt = load_patients(show_progress=True)[0]
    #   record = pt
    #   ecg_signal = -pt[dt.SeatedSession, dt.ECG].samples[:, 1]
    #   fs = pt[dt.SeatedSession, dt.ECG].fs
    #
    # For this example, we assume these variables are already set.
    
    # ECG Heart Rate Analysis (for plotting)
    hr_time, inst_hr, mean_hr = compute_ecg_heart_rate(filtered_ecg, fs)
    print(f"Heart Rate from ECG: {mean_hr:.2f} BPM (average)")
    plt.figure(figsize=(8, 4))
    plt.plot(hr_time, inst_hr, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    session = pt[dt.SeatedSession]  # Use appropriate session
    exercise_ann = session[dt.Exercise]
    seg_times_hr, hr_10s = segment_ecg_hr(filtered_ecg, fs,
                                          start_time=np.min(exercise_ann.timestamp),
                                          end_time=np.max(exercise_ann.timestamp),
                                          interval_size=10)
    print("Segmented HR (10s intervals):", hr_10s)
    plt.figure(figsize=(8, 4))
    plt.plot(seg_times_hr, hr_10s, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate (10s Intervals)')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # Compute combined EDR signal from features (using extract_ecg_features)
    from preprocessing import extract_ecg_features, get_combined_signal_from_features

    features = extract_ecg_features(
        ecg_signal,
        fs,
        method="pan_tompkins",
        RRI_fs=10,
        default_window_len=0.1,
        search_window_len=0.01,
        edr_method='R_peak'
    )

    # Compute combined EDR signal with equal weighting and a 0.1–0.5 Hz bandpass:
    combined_edr_time, combined_edr_signal = get_combined_signal_from_features(
        features,
        resampled_fs=10,
        freq_band=(0.1, 0.5),
        weighting="equal"
    )

    
    # Breath detection on the combined EDR signal
    in_times, ex_times = auto_detect_edr_breaths(combined_edr_time, combined_edr_signal,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    interval_size = 10
    seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio = [], [], [], []
    seg_times = np.arange(np.min(exercise_ann.timestamp), np.max(exercise_ann.timestamp), interval_size)
    for t0 in seg_times:
        t1 = t0 + interval_size
        mi, me = compute_mean_insp_exp_edr(in_times, ex_times, t0, t1)
        seg_times_edr.append(t0)
        edr_mean_insp.append(mi)
        edr_mean_exp.append(me)
        ratio = mi/me if (not np.isnan(mi) and not np.isnan(me) and me > 0) else np.nan
        edr_ie_ratio.append(ratio)
    print("=== EDR-derived Mean Intervals (10s windows) ===")
    for t0, mi, me, ratio in zip(seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio):
        print(f"[{t0:.0f}-{t0+interval_size:.0f}s]: Insp={mi:.2f}s, Exp={me:.2f}s, I:E={ratio:.2f}")
    plot_edr_breaths(combined_edr_time, combined_edr_signal, in_times, ex_times)
    
    # Build feature table using combined EDR signal
    feature_df = build_feature_table(record, session, ecg_signal, fs, 
                                     QRS_amplitude_resampled=(combined_edr_time, None, combined_edr_signal),
                                     interval_size=10)
    print("Feature Table:")
    print(feature_df)
    
    labels = [f"{int(ws)}–{int(we)}s" for ws, we in zip(feature_df["window_start"], feature_df["window_end"])]
    
    # Additional feature analysis plots:
    mid_times = feature_df['window_start'] + (30 / 2)
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['TrueInspDuration'], marker='o', linestyle='-', label='True Insp Duration (In→Ex)')
    plt.plot(mid_times, feature_df['TrueExpDuration'], marker='o', linestyle='-', label='True Exp Duration (Ex→In)')
    plt.title('True Inspiration and Expiration Durations')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['n_breaths'], marker='o', linestyle='-', label='Number of Breaths')
    plt.title('Number of Breaths (per 10s window)')
    plt.xlabel('Time (s)')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['MeanBreathDuration'], marker='o', linestyle='-', label='Mean Breath Duration')
    plt.title('Mean Breath Duration')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['EDR_BPM'], marker='o', linestyle='-', label='Breathing Rate (BPM)')
    plt.title('Breathing Rate per Minute')
    plt.xlabel('Time (s)')
    plt.ylabel('BPM')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["EDR_BPM"], color="royalblue", alpha=0.7)
    plt.title("Breathing Rate per Minute (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("BPM")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["MeanBreathDuration"], color="seagreen", alpha=0.7)
    plt.title("Mean Breath Duration (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("Duration (s)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_mean'], marker='o', linestyle='-', label='Mean QRS per Breath')
    plt.title('Mean QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Mean QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_std'], marker='o', linestyle='-', color='orange', label='Std of QRS per Breath')
    plt.title('Std of QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Std QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_BTB'], marker='o', linestyle='-', color='green', label='Beat-to-beat Variability')
    plt.title('Beat-to-beat Variability of QRS Amplitude')
    plt.xlabel('Time (s)')
    plt.ylabel('Std of Consecutive Differences')
    plt.grid(True)
    plt.legend()
    plt.show()


# breath segmentation and feature code for QRS amplitdue:

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import find_peaks, butter, filtfilt, iirnotch
from scipy.stats import skew, kurtosis
from scipy.interpolate import interp1d
from pan_tompkins import detect_ecg_peaks

# Global constant for minimum breath separation (in seconds)
DEFAULT_MIN_BREATH_SEC = 2.5
record = pt

#########################################
# SECTION 1: BREATH DETECTION FUNCTIONS
#########################################

def detect_edr_breaths(edr_time, edr_signal, min_breath_sec=DEFAULT_MIN_BREATH_SEC):
    """
    Detect inspiration and expiration breaths from the EDR signal.
    """
    min_distance_samples = int(10 * min_breath_sec)
    in_peaks, _ = find_peaks(-edr_signal, distance=min_distance_samples)
    ex_peaks, _ = find_peaks(edr_signal, distance=min_distance_samples)
    in_times = edr_time[in_peaks]
    ex_times = edr_time[ex_peaks]
    return in_times, ex_times

def auto_detect_edr_breaths(edr_time, edr_signal, init_min_breath_sec=2.5,
                            adjust_factor=0.5, max_iterations=5, tol=0.05,
                            min_allowed=0.5, max_allowed=10.0):
    """
    Automatically adapts min_breath_sec with multiple passes until stable.
    """
    current_min_breath_sec = init_min_breath_sec

    for iteration in range(max_iterations):
        in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
        all_breaths = np.sort(np.concatenate([in_times, ex_times]))
        if len(all_breaths) < 2:
            print(f"Not enough breath events (iteration {iteration+1}). Stopping.")
            return in_times, ex_times
        
        # Compute median interval (with outlier rejection)
        breath_intervals = np.diff(all_breaths)
        median_rough = np.median(breath_intervals)
        upper_cut = 2.0 * median_rough
        lower_cut = 0.2 * median_rough
        cleaned_intervals = breath_intervals[(breath_intervals <= upper_cut) & (breath_intervals >= lower_cut)]
        median_interval = np.median(cleaned_intervals) if len(cleaned_intervals) >= 2 else median_rough

        new_min_breath_sec = adjust_factor * median_interval
        new_min_breath_sec = np.clip(new_min_breath_sec, min_allowed, max_allowed)
        ratio = new_min_breath_sec / current_min_breath_sec
        print(f"Iteration {iteration+1}: old={current_min_breath_sec:.2f}, new={new_min_breath_sec:.2f} (ratio={ratio:.2f})")
        if abs(ratio - 1.0) < tol:
            current_min_breath_sec = new_min_breath_sec
            break
        else:
            current_min_breath_sec = new_min_breath_sec

    # Final detection pass
    in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=current_min_breath_sec)
    print(f"Final min_breath_sec after {iteration+1} iteration(s): {current_min_breath_sec:.2f}")
    return in_times, ex_times

#########################################
# SECTION 2: FEATURE ANALYSIS FUNCTIONS
#########################################

# -- 2.1 EDR-derived Breathing Features --

# --- 2.1 EDR-derived Breathing Features ---
def compute_mean_insp_exp_edr(in_times, ex_times, t_start, t_end):
    """
    Compute the mean inspiration and expiration intervals within a window.
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    valid_ex = ex_times[(ex_times >= t_start) & (ex_times <= t_end)]
    
    if len(valid_in) < 2:
        mean_insp_interval = np.nan
    else:
        mean_insp_interval = np.mean(np.diff(valid_in))
    
    if len(valid_ex) < 2:
        mean_exp_interval = np.nan
    else:
        mean_exp_interval = np.mean(np.diff(valid_ex))
        
    return mean_insp_interval, mean_exp_interval

def compute_true_insp_exp_durations(in_times, ex_times, t_start, t_end):
    insp_durations = []
    exp_durations = []

    # Ex → In = True Inspiration
    for e_time in ex_times:
        if e_time < t_start or e_time > t_end:
            continue
        next_in = in_times[in_times > e_time]
        if next_in.size > 0 and next_in[0] <= t_end:
            insp_durations.append(next_in[0] - e_time)

    # In → Ex = True Expiration
    for i_time in in_times:
        if i_time < t_start or i_time > t_end:
            continue
        next_ex = ex_times[ex_times > i_time]
        if next_ex.size > 0 and next_ex[0] <= t_end:
            exp_durations.append(next_ex[0] - i_time)

    return (
        np.mean(insp_durations) if insp_durations else np.nan,
        np.mean(exp_durations) if exp_durations else np.nan
    )



def compute_breath_features(in_times, t_start, t_end):
    """
    Compute additional breath features within a window:
      - Number of breaths (count of valid inspirations)
      - Mean breath duration (average time between consecutive inspirations)
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    n_breaths = len(valid_in)
    if n_breaths >= 2:
        mean_breath_duration = np.mean(np.diff(valid_in))
    else:
        mean_breath_duration = np.nan
    return n_breaths, mean_breath_duration

def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Compute the breathing rate (BPM) from inspiration times within a window.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    mean_interval = np.mean(np.diff(valid))
    return 60.0 / mean_interval

# -- 2.2 ECG Heart Rate Analysis --

def compute_ecg_heart_rate(ecg_signal, fs):
    """
    Compute ECG heart rate from the filtered ECG signal.
    Requires pan_tompkins.detect_ecg_peaks to be defined.
    """
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        print("⚠ Not enough R-peaks to compute heart rate.")
        return np.array([]), np.array([]), np.nan

    rr_intervals = np.diff(R_peaks) / fs  
    inst_hr = 60.0 / rr_intervals 
    hr_time = 0.5 * (R_peaks[:-1] + R_peaks[1:]) / fs
    mean_hr = np.mean(inst_hr)
    return hr_time, inst_hr, mean_hr

def segment_ecg_hr(ecg_signal, fs, start_time, end_time, interval_size=10):
    """
    Segment ECG heart rate in intervals.
    """
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        return np.array([]), np.array([])

    seg_times = np.arange(start_time, end_time, interval_size)
    hr_values = []
    for t0 in seg_times:
        t1 = t0 + interval_size
        segment_peaks = R_peaks[(R_peaks/fs >= t0) & (R_peaks/fs < t1)]
        if len(segment_peaks) < 2:
            hr_values.append(np.nan)
        else:
            rr = np.diff(segment_peaks) / fs
            hr_values.append(60.0 / np.mean(rr))
    return seg_times, np.array(hr_values)

# -- 2.3 QRS Amplitude Statistics per Breath --

def compute_qrs_amp_per_breath(edr_time, edr_signal, in_times):
    """
    Compute QRS amplitude statistics for each breath cycle.
    Returns breath_times, mean QRS, standard deviation of QRS,
    and beat-to-beat variability (std of consecutive differences).
    """
    breath_times = []
    mean_qrs = []
    std_qrs = []
    btb_var = []
    for i in range(len(in_times) - 1):
        start = in_times[i]
        end = in_times[i+1]
        mask = (edr_time >= start) & (edr_time < end)
        segment = edr_signal[mask]
        breath_time = 0.5 * (start + end)
        if len(segment) < 2:
            breath_times.append(breath_time)
            mean_qrs.append(np.nan)
            std_qrs.append(np.nan)
            btb_var.append(np.nan)
        else:
            mean_val = np.mean(segment)
            std_val = np.std(segment)
            diffs = np.diff(segment)
            btb_val = np.std(diffs) if len(diffs) > 0 else np.nan
            breath_times.append(breath_time)
            mean_qrs.append(mean_val)
            std_qrs.append(std_val)
            btb_var.append(btb_val)
    return (np.array(breath_times),
            np.array(mean_qrs),
            np.array(std_qrs),
            np.array(btb_var))

def compute_peak_to_trough(edr_time, edr_signal, t_start, t_end):
    mask = (edr_time >= t_start) & (edr_time < t_end)
    segment = edr_signal[mask]
    if segment.size == 0:
        return np.nan
    return np.max(segment) - np.min(segment)

def compute_rising_slope(edr_time, edr_signal, t_start, t_peak):
    mask = (edr_time >= t_start) & (edr_time <= t_peak)
    seg_time = edr_time[mask]
    seg_signal = edr_signal[mask]
    if seg_time.size < 2:
        return np.nan
    # Slope can be estimated by the linear fit (slope)
    p = np.polyfit(seg_time, seg_signal, 1)
    return p[0]  # slope

def compute_area_under_curve(edr_time, edr_signal, t_start, t_end):
    mask = (edr_time >= t_start) & (edr_time <= t_end)
    seg_time = edr_time[mask]
    seg_signal = edr_signal[mask]
    if seg_time.size < 2:
        return np.nan
    return np.trapz(seg_signal, seg_time)


# -- 2.4 Plotting Function for EDR Breath Detection --

def plot_edr_breaths(edr_time, edr_signal, in_times, ex_times):
    """
    Plot the EDR signal with detected inspiration and expiration points.
    """
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal (QRS Amplitude)')
    plt.plot(in_times, np.interp(in_times, edr_time, edr_signal), 'o', label='Inspirations')
    plt.plot(ex_times, np.interp(ex_times, edr_time, edr_signal), 'o', label='Expirations')
    plt.title('ECG-Derived Respiration with Detected Breaths')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

#########################################
# SECTION 3: FEATURE TABLE CONSTRUCTION
#########################################

def build_feature_table(patient, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10):
    # Access the exercise annotations
    exercises = session[dt.Exercise]
    all_ex_timestamps = exercises.timestamp
    full_start = float(np.min(all_ex_timestamps))
    full_end = float(np.max(all_ex_timestamps))
    print(f"build_feature_table: Using exercise interval {full_start:.1f}–{full_end:.1f} s")

    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]

    # Breath detection on the EDR signal
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    

    
    # Detect ECG R-peaks (using pan_tompkins here)
    R_peaks = detect_ecg_peaks(ecg_signal, fs)

    feature_rows = []
    seg_times = np.arange(full_start, full_end, interval_size)
    
    # Pre-compute QRS amplitude per breath stats (for beat-to-beat variability)
    btqrs_times, mean_qrs_breath, std_qrs_breath, btb_var_breath = compute_qrs_amp_per_breath(edr_time_qrs, edr_signal_qrs, in_times)
    
    for window_start in seg_times:
        window_end = window_start + interval_size
        true_insp_dur, true_exp_dur = compute_true_insp_exp_durations(in_times, ex_times, window_start, window_end)


        # EDR-derived breathing rate (BPM)
        edr_bpm = compute_bpm_from_inspiration_times(in_times, window_start, window_end)


        # Breath count and mean breath duration (from inspirations)
        n_breaths, mean_breath_duration = compute_breath_features(in_times, window_start, window_end)

        # ECG-derived heart rate (BPM) from R-peaks in the window
        window_peaks = R_peaks[(R_peaks/fs >= window_start) & (R_peaks/fs < window_end)]
        if len(window_peaks) < 2:
            ecg_bpm = np.nan
        else:
            rr = np.diff(window_peaks) / fs
            ecg_bpm = 60.0 / np.mean(rr)

        # QRS amplitude statistics in the window
        mask_qrs = (edr_time_qrs >= window_start) & (edr_time_qrs < window_end)
        qrs_window = edr_signal_qrs[mask_qrs]
        if len(qrs_window) < 1:
            qrs_mean = np.nan
            qrs_std = np.nan
        else:
            qrs_mean = np.mean(qrs_window)
            qrs_std = np.std(qrs_window)
        
        # Beat-to-beat variability (BTB) of QRS amplitude for breaths in the window
        mask_btb = (btqrs_times >= window_start) & (btqrs_times < window_end)
        if np.any(mask_btb):
            qrs_btb = np.nanmean(btb_var_breath[mask_btb])
        else:
            qrs_btb = np.nan
        
        # New metric: Peak-to-Trough of EDR signal in this window
        pt_diff = compute_peak_to_trough(edr_time_qrs, edr_signal_qrs, window_start, window_end)
        # New metric: Rising slope (for example, from window start to the first peak)
        # You would need to identify the peak within this window:
        mask = (edr_time_qrs >= window_start) & (edr_time_qrs < window_end)
        if np.any(mask):
            seg_time = edr_time_qrs[mask]
            seg_signal = edr_signal_qrs[mask]
            # Find the index of the maximum in the segment (peak)
            peak_idx = np.argmax(seg_signal)
            t_peak = seg_time[peak_idx]
            rising_slope = compute_rising_slope(edr_time_qrs, edr_signal_qrs, window_start, t_peak)
        else:
            rising_slope = np.nan

        # New metric: Area under the EDR curve in this window
        area = compute_area_under_curve(edr_time_qrs, edr_signal_qrs, window_start, window_end)

        row = {
            "subject_id": patient.id,
            "window_start": window_start,
            "window_end": window_end,
            "EDR_BPM": edr_bpm,
            "n_breaths": n_breaths,
            "MeanBreathDuration": mean_breath_duration,
            "ECG_BPM": ecg_bpm,
            "QRS_mean": qrs_mean,
            "QRS_std": qrs_std,
            "QRS_BTB": qrs_btb,
            "TrueInspDuration": true_insp_dur,
            "TrueExpDuration": true_exp_dur,
            "PeakToTrough": pt_diff,
            "RisingSlope": rising_slope,
            "AreaUnderCurve": area

        }
        feature_rows.append(row)
    
    feature_df = pd.DataFrame(feature_rows)
    return feature_df



#########################################
# SECTION 4: MAIN EXECUTION & PLOTTING
#########################################

if __name__ == "__main__":
    # NOTE: Ensure that the following variables are defined in your environment:
    #   record, ecg_signal, filtered_ecg, fs, QRS_amplitude_resampled, preprocessing
    
    # --- ECG Heart Rate Analysis (for plotting) ---
    hr_time, inst_hr, mean_hr = compute_ecg_heart_rate(filtered_ecg, fs)
    print(f"Heart Rate from ECG: {mean_hr:.2f} BPM (average)")
    
    plt.figure(figsize=(8, 4))
    plt.plot(hr_time, inst_hr, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    session = pt[dt.SeatedSession]  # or SupineSession depending on your use case
    exercise_ann = session[dt.Exercise]

    seg_times_hr, hr_10s = segment_ecg_hr(
        filtered_ecg,
        fs,
        start_time=np.min(exercise_ann.timestamp),
        end_time=np.max(exercise_ann.timestamp),
        interval_size=10
        )

    print("Segmented HR (10s intervals):", hr_10s)
    
    plt.figure(figsize=(8, 4))
    plt.plot(seg_times_hr, hr_10s, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate (10s Intervals)')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # --- SECTION 1: EDR Breath Detection ---
    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]
    
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    
    interval_size = 10  # 10-second samples for segmentation here
    seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio = [], [], [], []
    seg_times = np.arange(np.min(exercise.timestamp), np.max(exercise.timestamp), interval_size)

    for t0 in seg_times:
        t1 = t0 + interval_size
        mi, me = compute_mean_insp_exp_edr(in_times, ex_times, t0, t1)
        seg_times_edr.append(t0)
        edr_mean_insp.append(mi)
        edr_mean_exp.append(me)
        ratio = mi/me if (not np.isnan(mi) and not np.isnan(me) and me > 0) else np.nan
        edr_ie_ratio.append(ratio)
    
    print("=== EDR-derived Mean Intervals (10s windows) ===")
    for t0, mi, me, ratio in zip(seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio):
        print(f"[{t0:.0f}-{t0+interval_size:.0f}s]: Insp={mi:.2f}s, Exp={me:.2f}s, I:E={ratio:.2f}")
    
    plot_edr_breaths(edr_time_qrs, edr_signal_qrs, in_times, ex_times)
    
    # --- Build Feature Table ---
    feature_df = build_feature_table(record, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10)
    print("Feature Table:")
    print(feature_df)
    
    # 2) Create a label for each time window (e.g., "0–30s", "30–60s", etc.)
    labels = [
        f"{int(ws)}–{int(we)}s"
        for ws, we in zip(feature_df["window_start"], feature_df["window_end"])
    ]

    
    # --- Additional Plots for Feature Analysis ---
    # Calculate mid-time for each window from the feature table (using window length 30s)
    mid_times = feature_df['window_start'] + (30 / 2)

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['TrueInspDuration'], marker='o', linestyle='-', label='True Insp Duration (In→Ex)')
    plt.plot(mid_times, feature_df['TrueExpDuration'], marker='o', linestyle='-', label='True Exp Duration (Ex→In)')
    plt.title('True Inspiration and Expiration Durations')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot: Number of Breaths per Window
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['n_breaths'], marker='o', linestyle='-', label='Number of Breaths')
    plt.title('Number of Breaths (per 10s window)')
    plt.xlabel('Time (s)')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot: Mean Breath Duration vs. Breathing Cycle Duration
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['MeanBreathDuration'], marker='o', linestyle='-', label='Mean Breath Duration')
    plt.title('Mean Breath Duration')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # Plot: Breathing Rate per Minute
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['EDR_BPM'], marker='o', linestyle='-', label='Breathing Rate (BPM)')
    plt.title('Breathing Rate per Minute')
    plt.xlabel('Time (s)')
    plt.ylabel('BPM')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 3) Bar Plot: Breathing Rate per Minute (EDR_BPM)
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["EDR_BPM"], color="royalblue", alpha=0.7)
    plt.title("Breathing Rate per Minute (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("BPM")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

    # 4) Bar Plot: Mean Breath Duration
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["MeanBreathDuration"], color="seagreen", alpha=0.7)
    plt.title("Mean Breath Duration (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("Duration (s)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()
    
    # --- Additional Plots for QRS Amplitude Statistics ---
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_mean'], marker='o', linestyle='-', label='Mean QRS per Breath')
    plt.title('Mean QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Mean QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_std'], marker='o', linestyle='-', color='orange', label='Std of QRS per Breath')
    plt.title('Std of QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Std QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_BTB'], marker='o', linestyle='-', color='green', label='Beat-to-beat Variability')
    plt.title('Beat-to-beat Variability of QRS Amplitude')
    plt.xlabel('Time (s)')
    plt.ylabel('Std of Consecutive Differences')
    plt.grid(True)
    plt.legend()
    plt.show()

    import matplotlib.pyplot as plt

    # Suppose feature_df is already built
    # Plot Peak-to-Trough
    plt.figure(figsize=(10, 6))
    plt.plot(feature_df['window_start'], feature_df['PeakToTrough'], '-o', label='Peak-to-Trough')
    plt.title("EDR Signal Peak-to-Trough Difference")
    plt.xlabel("Window Start Time (s)")
    plt.ylabel("Amplitude Difference")
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot Rising Slope
    plt.figure(figsize=(10, 6))
    plt.plot(feature_df['window_start'], feature_df['RisingSlope'], '-o', color='orange', label='Rising Slope')
    plt.title("EDR Signal Rising Slope")
    plt.xlabel("Window Start Time (s)")
    plt.ylabel("Slope (Amplitude/sec)")
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot Area Under Curve
    plt.figure(figsize=(10, 6))
    plt.plot(feature_df['window_start'], feature_df['AreaUnderCurve'], '-o', color='green', label='Area Under Curve')
    plt.title("Area Under the EDR Curve")
    plt.xlabel("Window Start Time (s)")
    plt.ylabel("Area (integral units)")
    plt.grid(True)
    plt.legend()
    plt.show()







# making feature dataframe where you can choose which method:

In [ ]:
""" from features import build_feature_table
import pandas as pd
import diamonds.data as dt
from diamonds import load_patients
from preprocessing import extract_ecg_features  # unified method


ptt = load_patients(show_progress=True)
session_type = dt.SeatedSession  

feature_list = []

# ⬅️ Pick your R-peak detection method here:
method_choice = "pan_tompkins"  # or "engzee", or "pan_tompkins"

for pt in ptt:
    print(f"Processing patient: {pt.id}")
    
    try:
        session = pt[session_type]
        
        EMG, ECG = pt[session_type, dt.EMG].decompose()
        ecg_signal = -ECG.samples[:, 1]  # use channel 1 and flip signal
        fs = ECG.fs

        # 🔄 Extract all features using the selected method
        results = extract_ecg_features(
            ecg_signal=ecg_signal,
            fs=fs,
            method=method_choice,
            RRI_fs=10,
            default_window_len=0.1,
            search_window_len=0.01,
            edr_method='R_peak'
        )

        # 👇 Feed required values into your table builder
        filtered_ecg = results["filtered_ecg"]
        QRS_amplitude_resampled = results["QRS_amp_resampled"]

        _df = build_feature_table(
            patient=pt,
            session=session,
            ecg_signal=filtered_ecg,
            fs=fs,
            QRS_amplitude_resampled=QRS_amplitude_resampled
        )

        feature_list.append(_df)

    except Exception as e:
        print(f"⚠️ Failed for patient {pt.id}: {e}")

# 📦 Combine results
features_df = pd.concat(feature_list, ignore_index=True)

print("✅ Combined feature dataframe:")
print(features_df)
features_df.to_excel("all_patient_features_v2.xlsx", index=False)  """


# features.py when freature table is pr breath:

In [ ]:
# features.py

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch
from scipy.interpolate import interp1d
import sys
import os


project_root = "C:/Users/Visnu/DIAMONDS"  

if project_root not in sys.path:
    sys.path.append(project_root)

import diamonds.data as dt

from pan_tompkins import detect_ecg_peaks
from breath_detection import auto_detect_edr_breaths, define_breaths_insp_exp, DEFAULT_MIN_BREATH_SEC
from diamonds_definitions import pt, session, exercise, ecg_signal, fs
import importlib
import preprocessing
importlib.reload(preprocessing)
import importlib
import pan_tompkins
import engzee_tompkins
import new_rpeak_detector

importlib.reload(pan_tompkins)
importlib.reload(engzee_tompkins)
importlib.reload(new_rpeak_detector)

record = pt

# Then pick the method you want:
method_choice = "pan_tompkins"   # or "engzee" or "pan_tompkins"

# Now call the unified function:
results = preprocessing.extract_ecg_features(
    ecg_signal,
    fs,
    method=method_choice,
    RRI_fs=10,
    default_window_len=0.1,
    search_window_len=0.01,
    edr_method='R_peak'
)

# The dictionary 'results' now has everything:
filtered_ecg = results["filtered_ecg"]
R_peaks       = results["R_peaks"]
RRI           = results["RRI"]
RRI_resampled = results["RRI_resampled"]
QRS_amplitude = results["QRS_amp"]
QRS_amplitude_resampled = results["QRS_amp_resampled"]
BW_effect     = results["BW_effect"]
AM_effect     = results["AM_effect"]
R_peak_amplitude             = results["R_peak_amplitude"]
R_peak_amplitude_resampled   = results["R_peak_amplitude_resampled"]
edr_time_r_peak              = results["edr_time_r_peak"]
edr_signal_r_peak            = results["edr_signal_r_peak"]
edr_rri_time                 = results["edr_rri_time"]
edr_rri_signal               = results["edr_rri_signal"]

print(f"Method used: {method_choice}")
print(f"Detected {len(R_peaks)} R-peaks")



# --- 2.1 EDR-derived Breathing Features ---
def compute_mean_insp_exp_edr(in_times, ex_times, t_start, t_end):
    """
    Compute the mean inspiration and expiration intervals within a window.
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    valid_ex = ex_times[(ex_times >= t_start) & (ex_times <= t_end)]
    
    if len(valid_in) < 2:
        mean_insp_interval = np.nan
    else:
        mean_insp_interval = np.mean(np.diff(valid_in))
    
    if len(valid_ex) < 2:
        mean_exp_interval = np.nan
    else:
        mean_exp_interval = np.mean(np.diff(valid_ex))
        
    return mean_insp_interval, mean_exp_interval

def compute_true_insp_exp_durations(in_times, ex_times, t_start, t_end):
    insp_durations = []
    exp_durations = []

    # Ex → In = True Inspiration
    for e_time in ex_times:
        if e_time < t_start or e_time > t_end:
            continue
        next_in = in_times[in_times > e_time]
        if next_in.size > 0 and next_in[0] <= t_end:
            insp_durations.append(next_in[0] - e_time)

    # In → Ex = True Expiration
    for i_time in in_times:
        if i_time < t_start or i_time > t_end:
            continue
        next_ex = ex_times[ex_times > i_time]
        if next_ex.size > 0 and next_ex[0] <= t_end:
            exp_durations.append(next_ex[0] - i_time)

    return (
        np.mean(insp_durations) if insp_durations else np.nan,
        np.mean(exp_durations) if exp_durations else np.nan
    )



def compute_breath_features(in_times, t_start, t_end):
    """
    Compute additional breath features within a window:
      - Number of breaths (count of valid inspirations)
      - Mean breath duration (average time between consecutive inspirations)
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    n_breaths = len(valid_in)
    if n_breaths >= 2:
        mean_breath_duration = np.mean(np.diff(valid_in))
    else:
        mean_breath_duration = np.nan
    return n_breaths, mean_breath_duration

def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Compute the breathing rate (BPM) from inspiration times within a window.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    mean_interval = np.mean(np.diff(valid))
    return 60.0 / mean_interval

# -- 2.2 ECG Heart Rate Analysis --

def compute_ecg_heart_rate(ecg_signal, fs):
    """
    Compute ECG heart rate from the filtered ECG signal.
    Requires pan_tompkins.detect_ecg_peaks to be defined.
    """
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        print("⚠ Not enough R-peaks to compute heart rate.")
        return np.array([]), np.array([]), np.nan

    rr_intervals = np.diff(R_peaks) / fs  
    inst_hr = 60.0 / rr_intervals 
    hr_time = 0.5 * (R_peaks[:-1] + R_peaks[1:]) / fs
    mean_hr = np.mean(inst_hr)
    return hr_time, inst_hr, mean_hr

def segment_ecg_hr(ecg_signal, fs, start_time, end_time, interval_size=10):
    """
    Segment ECG heart rate in intervals.
    """
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        return np.array([]), np.array([])

    seg_times = np.arange(start_time, end_time, interval_size)
    hr_values = []
    for t0 in seg_times:
        t1 = t0 + interval_size
        segment_peaks = R_peaks[(R_peaks/fs >= t0) & (R_peaks/fs < t1)]
        if len(segment_peaks) < 2:
            hr_values.append(np.nan)
        else:
            rr = np.diff(segment_peaks) / fs
            hr_values.append(60.0 / np.mean(rr))
    return seg_times, np.array(hr_values)

# -- 2.3 QRS Amplitude Statistics per Breath --

def compute_qrs_amp_per_breath(edr_time, edr_signal, in_times):
    """
    Compute QRS amplitude statistics for each breath cycle.
    Returns breath_times, mean QRS, standard deviation of QRS,
    and beat-to-beat variability (std of consecutive differences).
    """
    breath_times = []
    mean_qrs = []
    std_qrs = []
    btb_var = []
    for i in range(len(in_times) - 1):
        start = in_times[i]
        end = in_times[i+1]
        mask = (edr_time >= start) & (edr_time < end)
        segment = edr_signal[mask]
        breath_time = 0.5 * (start + end)
        if len(segment) < 2:
            breath_times.append(breath_time)
            mean_qrs.append(np.nan)
            std_qrs.append(np.nan)
            btb_var.append(np.nan)
        else:
            mean_val = np.mean(segment)
            std_val = np.std(segment)
            diffs = np.diff(segment)
            btb_val = np.std(diffs) if len(diffs) > 0 else np.nan
            breath_times.append(breath_time)
            mean_qrs.append(mean_val)
            std_qrs.append(std_val)
            btb_var.append(btb_val)
    return (np.array(breath_times),
            np.array(mean_qrs),
            np.array(std_qrs),
            np.array(btb_var))

def compute_peak_to_trough(edr_time, edr_signal, t_start, t_end):
    mask = (edr_time >= t_start) & (edr_time < t_end)
    segment = edr_signal[mask]
    if segment.size == 0:
        return np.nan
    return np.max(segment) - np.min(segment)

def compute_rising_slope(edr_time, edr_signal, t_start, t_peak):
    mask = (edr_time >= t_start) & (edr_time <= t_peak)
    seg_time = edr_time[mask]
    seg_signal = edr_signal[mask]
    if seg_time.size < 2:
        return np.nan
    # Slope can be estimated by the linear fit (slope)
    p = np.polyfit(seg_time, seg_signal, 1)
    return p[0]  # slope

def compute_area_under_curve(edr_time, edr_signal, t_start, t_end):
    mask = (edr_time >= t_start) & (edr_time <= t_end)
    seg_time = edr_time[mask]
    seg_signal = edr_signal[mask]
    if seg_time.size < 2:
        return np.nan
    return np.trapz(seg_signal, seg_time)


# -- 2.4 Plotting Function for EDR Breath Detection --

def plot_breath_cycles(edr_time, edr_signal, in_times, ex_times):
    """
    Plot the EDR signal and overlay the defined breath cycles,
    where each breath is defined as starting at an inspiration and ending at the subsequent expiration.
    """
    # Compute breath boundaries using your definition function.
    from breath_detection import define_breaths_insp_exp  # ensure it's imported
    breath_starts, breath_ends = define_breaths_insp_exp(in_times, ex_times)
    
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal')
    
    # Plot breath start markers (e.g., green circles)
    plt.plot(breath_starts, np.interp(breath_starts, edr_time, edr_signal),
             'go', label='Breath Start')
    # Plot breath end markers (e.g., red circles)
    plt.plot(breath_ends, np.interp(breath_ends, edr_time, edr_signal),
             'ro', label='Breath End')
    
    # Optionally, draw vertical lines at each boundary
    for t in breath_starts:
        plt.axvline(t, color='green', linestyle='--', alpha=0.5)
    for t in breath_ends:
        plt.axvline(t, color='red', linestyle='--', alpha=0.5)
    
    plt.title('Breath Cycles (Inspiration to Expiration)')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_edr_breaths(edr_time, edr_signal, in_times, ex_times):
    """
    Plot the EDR signal with detected inspiration and expiration points.
    """
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal (QRS Amplitude)')
    plt.plot(in_times, np.interp(in_times, edr_time, edr_signal), 'o', label='Inspirations')
    plt.plot(ex_times, np.interp(ex_times, edr_time, edr_signal), 'o', label='Expirations')
    plt.title('ECG-Derived Respiration with Detected Breaths')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

#########################################
# SECTION 3: FEATURE TABLE CONSTRUCTION
#########################################

def build_feature_table_by_breath(patient, session, ecg_signal, fs, QRS_amplitude_resampled):
    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]

    # --- Detect Breaths ---
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)

    breath_starts, breath_ends = define_breaths_insp_exp(in_times, ex_times)

    # --- ECG R-peaks ---
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    R_times = R_peaks / fs

    feature_rows = []

    for i in range(len(in_times) - 1):  # Loop through breaths based on inspirations
        start = in_times[i]
        end = ex_times[ex_times > start][0] if np.any(ex_times > start) else np.nan

        # True Inspiration: time from previous Ex → current In
        true_insp = np.nan
        if i > 0 and ex_times[ex_times < start].size > 0:
            last_ex = ex_times[ex_times < start][-1]
            true_insp = start - last_ex

        # True Expiration: In → Ex
        true_exp = end - start if not np.isnan(end) else np.nan


        # --- QRS Stats within Breath ---
        mask_qrs = (edr_time_qrs >= start) & (edr_time_qrs < end)
        qrs_segment = edr_signal_qrs[mask_qrs]
        qrs_mean = np.mean(qrs_segment) if qrs_segment.size > 0 else np.nan
        qrs_std = np.std(qrs_segment) if qrs_segment.size > 0 else np.nan
        qrs_btb = np.std(np.diff(qrs_segment)) if len(qrs_segment) > 2 else np.nan

        # --- ECG BPM within Breath (if ≥2 R-peaks) ---
        r_mask = (R_times >= start) & (R_times < end)
        r_within = R_times[r_mask]
        if len(r_within) >= 2:
            rr = np.diff(r_within)
            ecg_bpm = 60.0 / np.mean(rr)
        else:
            ecg_bpm = np.nan

        # --- EDR Peak-to-Trough ---
        pt_diff = compute_peak_to_trough(edr_time_qrs, edr_signal_qrs, start, end)

        # --- Rising Slope (start → peak) ---
        seg_time = edr_time_qrs[mask_qrs]
        seg_signal = edr_signal_qrs[mask_qrs]
        if seg_signal.size > 1:
            peak_idx = np.argmax(seg_signal)
            t_peak = seg_time[peak_idx]
            rising_slope = compute_rising_slope(edr_time_qrs, edr_signal_qrs, start, t_peak)
        else:
            rising_slope = np.nan

        # --- Area Under Curve ---
        area = compute_area_under_curve(edr_time_qrs, edr_signal_qrs, start, end)

        row = {
            "subject_id": patient.id,
            "breath_start": start,
            "breath_end": end,
            "true_insp_duration": true_insp,
            "true_exp_duration": true_exp,
            "QRS_mean": qrs_mean,
            "QRS_std": qrs_std,
            "QRS_BTB": qrs_btb,
            "ECG_BPM": ecg_bpm,
            "EDR_peak_to_trough": pt_diff,
            "EDR_rising_slope": rising_slope,
            "EDR_area_under_curve": area
        }

        feature_rows.append(row)

    feature_df = pd.DataFrame(feature_rows)
    return feature_df




def main():
        # --- SECTION 1: EDR Breath Detection ---
    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]
    edr_time = QRS_amplitude_resampled[0]
    edr_signal = QRS_amplitude_resampled[2]

    
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    breath_starts, breath_ends = define_breaths_insp_exp(in_times, ex_times)

    plot_breath_cycles(edr_time, edr_signal, in_times, ex_times)


    plot_edr_breaths(edr_time_qrs, edr_signal_qrs, in_times, ex_times)
    


    # --- ECG Heart Rate Analysis ---
    hr_time, inst_hr, mean_hr = compute_ecg_heart_rate(filtered_ecg, fs)
    print(f"Heart Rate from ECG: {mean_hr:.2f} BPM (average)")
    
    plt.figure(figsize=(8, 4))
    plt.plot(hr_time, inst_hr, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    session = pt[dt.SeatedSession]  # or SupineSession depending on your use case
    exercise_ann = session[dt.Exercise]

    seg_times_hr, hr_10s = segment_ecg_hr(
        filtered_ecg,
        fs,
        start_time=np.min(exercise_ann.timestamp),
        end_time=np.max(exercise_ann.timestamp),
        interval_size=10
        )

    print("Segmented HR (10s intervals):", hr_10s)
    
    plt.figure(figsize=(8, 4))
    plt.plot(seg_times_hr, hr_10s, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate (10s Intervals)')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    
    interval_size = 10  # 10-second samples for segmentation here
    seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio = [], [], [], []
    seg_times = np.arange(np.min(exercise.timestamp), np.max(exercise.timestamp), interval_size)

    for t0 in seg_times:
        t1 = t0 + interval_size
        mi, me = compute_mean_insp_exp_edr(in_times, ex_times, t0, t1)
        seg_times_edr.append(t0)
        edr_mean_insp.append(mi)
        edr_mean_exp.append(me)
        ratio = mi/me if (not np.isnan(mi) and not np.isnan(me) and me > 0) else np.nan
        edr_ie_ratio.append(ratio)
    
    print("=== EDR-derived Mean Intervals (10s windows) ===")
    for t0, mi, me, ratio in zip(seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio):
        print(f"[{t0:.0f}-{t0+interval_size:.0f}s]: Insp={mi:.2f}s, Exp={me:.2f}s, I:E={ratio:.2f}")
    
    
    # --- Build Feature Table ---
    # --- Build Per-Breath Feature Table ---
    feature_df = build_feature_table_by_breath(record, session, ecg_signal, fs, QRS_amplitude_resampled)
    print("Per-Breath Feature Table:")
    print(feature_df)

    # Optional: Create breath labels like "Breath 1", "Breath 2", ...
    labels = [f"Breath {i+1}" for i in range(len(feature_df))]

    # Breath midpoints (for plotting vs time)
    mid_times = (feature_df["breath_start"] + feature_df["breath_end"]) / 2


    # Plot: True Inspiration and Expiration Durations (per breath)
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['true_insp_duration'], marker='o', linestyle='-', label='True Inspiration (Ex→In)')
    plt.plot(mid_times, feature_df['true_exp_duration'], marker='o', linestyle='-', label='True Expiration (In→Ex)')
    plt.title('Inspiration and Expiration Durations (Per Breath)')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()



    """ # Plot: Number of Breaths per Window
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['n_breaths'], marker='o', linestyle='-', label='Number of Breaths')
    plt.title('Number of Breaths (per 10s window)')
    plt.xlabel('Time (s)')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot: Mean Breath Duration vs. Breathing Cycle Duration
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['MeanBreathDuration'], marker='o', linestyle='-', label='Mean Breath Duration')
    plt.title('Mean Breath Duration')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # Plot: Breathing Rate per Minute
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['EDR_BPM'], marker='o', linestyle='-', label='Breathing Rate (BPM)')
    plt.title('Breathing Rate per Minute')
    plt.xlabel('Time (s)')
    plt.ylabel('BPM')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 3) Bar Plot: Breathing Rate per Minute (EDR_BPM)
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["EDR_BPM"], color="royalblue", alpha=0.7)
    plt.title("Breathing Rate per Minute (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("BPM")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

    # 4) Bar Plot: Mean Breath Duration
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["MeanBreathDuration"], color="seagreen", alpha=0.7)
    plt.title("Mean Breath Duration (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("Duration (s)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show() """
    
    # --- Additional Plots for QRS Amplitude Statistics ---
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_mean'], marker='o', linestyle='-', label='Mean QRS per Breath')
    plt.title('Mean QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Mean QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_std'], marker='o', linestyle='-', color='orange', label='Std of QRS per Breath')
    plt.title('Std of QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Std QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_BTB'], marker='o', linestyle='-', color='green', label='Beat-to-beat Variability')
    plt.title('Beat-to-beat Variability of QRS Amplitude')
    plt.xlabel('Time (s)')
    plt.ylabel('Std of Consecutive Differences')
    plt.grid(True)
    plt.legend()
    plt.show()

    """ # Suppose feature_df is already built
    # Plot Peak-to-Trough
    plt.figure(figsize=(10, 6))
    plt.plot(feature_df['window_start'], feature_df['PeakToTrough'], '-o', label='Peak-to-Trough')
    plt.title("EDR Signal Peak-to-Trough Difference")
    plt.xlabel("Window Start Time (s)")
    plt.ylabel("Amplitude Difference")
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot Rising Slope
    plt.figure(figsize=(10, 6))
    plt.plot(feature_df['window_start'], feature_df['RisingSlope'], '-o', color='orange', label='Rising Slope')
    plt.title("EDR Signal Rising Slope")
    plt.xlabel("Window Start Time (s)")
    plt.ylabel("Slope (Amplitude/sec)")
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot Area Under Curve
    plt.figure(figsize=(10, 6))
    plt.plot(feature_df['window_start'], feature_df['AreaUnderCurve'], '-o', color='green', label='Area Under Curve')
    plt.title("Area Under the EDR Curve")
    plt.xlabel("Window Start Time (s)")
    plt.ylabel("Area (integral units)")
    plt.grid(True)
    plt.legend()
    plt.show() """


if __name__ == "__main__":
    main()


In [ ]:
# features.py

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch
from scipy.interpolate import interp1d
import sys
import os


project_root = "C:/Users/Visnu/DIAMONDS"  

if project_root not in sys.path:
    sys.path.append(project_root)

import diamonds.data as dt

from pan_tompkins import detect_ecg_peaks
from breath_detection import auto_detect_edr_breaths, define_breaths_insp_exp, DEFAULT_MIN_BREATH_SEC
from diamonds_definitions import pt, session, exercise, ecg_signal, fs
import importlib
import preprocessing
importlib.reload(preprocessing)
import importlib
import pan_tompkins
import engzee_tompkins
import new_rpeak_detector

importlib.reload(pan_tompkins)
importlib.reload(engzee_tompkins)
importlib.reload(new_rpeak_detector)

record = pt

# Then pick the method you want:
method_choice = "pan_tompkins"   # or "engzee" or "pan_tompkins"

# Now call the unified function:
results = preprocessing.extract_ecg_features(
    ecg_signal,
    fs,
    method=method_choice,
    RRI_fs=10,
    default_window_len=0.1,
    search_window_len=0.01,
    edr_method='R_peak'
)

# The dictionary 'results' now has everything:
filtered_ecg = results["filtered_ecg"]
R_peaks       = results["R_peaks"]
RRI           = results["RRI"]
RRI_resampled = results["RRI_resampled"]
QRS_amplitude = results["QRS_amp"]
QRS_amplitude_resampled = results["QRS_amp_resampled"]
BW_effect     = results["BW_effect"]
AM_effect     = results["AM_effect"]
R_peak_amplitude             = results["R_peak_amplitude"]
R_peak_amplitude_resampled   = results["R_peak_amplitude_resampled"]
edr_time_r_peak              = results["edr_time_r_peak"]
edr_signal_r_peak            = results["edr_signal_r_peak"]
edr_rri_time                 = results["edr_rri_time"]
edr_rri_signal               = results["edr_rri_signal"]

print(f"Method used: {method_choice}")
print(f"Detected {len(R_peaks)} R-peaks")



# --- 2.1 EDR-derived Breathing Features ---
def compute_mean_insp_exp_edr(in_times, ex_times, t_start, t_end):
    """
    Compute the mean inspiration and expiration intervals within a window.
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    valid_ex = ex_times[(ex_times >= t_start) & (ex_times <= t_end)]
    
    if len(valid_in) < 2:
        mean_insp_interval = np.nan
    else:
        mean_insp_interval = np.mean(np.diff(valid_in))
    
    if len(valid_ex) < 2:
        mean_exp_interval = np.nan
    else:
        mean_exp_interval = np.mean(np.diff(valid_ex))
        
    return mean_insp_interval, mean_exp_interval

def compute_true_insp_exp_durations(in_times, ex_times, t_start, t_end):
    insp_durations = []
    exp_durations = []

    # Ex → In = True Inspiration
    for e_time in ex_times:
        if e_time < t_start or e_time > t_end:
            continue
        next_in = in_times[in_times > e_time]
        if next_in.size > 0 and next_in[0] <= t_end:
            insp_durations.append(next_in[0] - e_time)

    # In → Ex = True Expiration
    for i_time in in_times:
        if i_time < t_start or i_time > t_end:
            continue
        next_ex = ex_times[ex_times > i_time]
        if next_ex.size > 0 and next_ex[0] <= t_end:
            exp_durations.append(next_ex[0] - i_time)

    return (
        np.mean(insp_durations) if insp_durations else np.nan,
        np.mean(exp_durations) if exp_durations else np.nan
    )



def compute_breath_features(in_times, t_start, t_end):
    """
    Compute additional breath features within a window:
      - Number of breaths (count of valid inspirations)
      - Mean breath duration (average time between consecutive inspirations)
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    n_breaths = len(valid_in)
    if n_breaths >= 2:
        mean_breath_duration = np.mean(np.diff(valid_in))
    else:
        mean_breath_duration = np.nan
    return n_breaths, mean_breath_duration

def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Compute the breathing rate (BPM) from inspiration times within a window.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    mean_interval = np.mean(np.diff(valid))
    return 60.0 / mean_interval

# -- 2.2 ECG Heart Rate Analysis --

def compute_ecg_heart_rate(ecg_signal, fs):
    """
    Compute ECG heart rate from the filtered ECG signal.
    Requires pan_tompkins.detect_ecg_peaks to be defined.
    """
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        print("⚠ Not enough R-peaks to compute heart rate.")
        return np.array([]), np.array([]), np.nan

    rr_intervals = np.diff(R_peaks) / fs  
    inst_hr = 60.0 / rr_intervals 
    hr_time = 0.5 * (R_peaks[:-1] + R_peaks[1:]) / fs
    mean_hr = np.mean(inst_hr)
    return hr_time, inst_hr, mean_hr

def segment_ecg_hr(ecg_signal, fs, start_time, end_time, interval_size=10):
    """
    Segment ECG heart rate in intervals.
    """
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    if len(R_peaks) < 2:
        return np.array([]), np.array([])

    seg_times = np.arange(start_time, end_time, interval_size)
    hr_values = []
    for t0 in seg_times:
        t1 = t0 + interval_size
        segment_peaks = R_peaks[(R_peaks/fs >= t0) & (R_peaks/fs < t1)]
        if len(segment_peaks) < 2:
            hr_values.append(np.nan)
        else:
            rr = np.diff(segment_peaks) / fs
            hr_values.append(60.0 / np.mean(rr))
    return seg_times, np.array(hr_values)

# -- 2.3 QRS Amplitude Statistics per Breath --

def compute_qrs_amp_per_breath(edr_time, edr_signal, in_times):
    """
    Compute QRS amplitude statistics for each breath cycle.
    Returns breath_times, mean QRS, standard deviation of QRS,
    and beat-to-beat variability (std of consecutive differences).
    """
    breath_times = []
    mean_qrs = []
    std_qrs = []
    btb_var = []
    for i in range(len(in_times) - 1):
        start = in_times[i]
        end = in_times[i+1]
        mask = (edr_time >= start) & (edr_time < end)
        segment = edr_signal[mask]
        breath_time = 0.5 * (start + end)
        if len(segment) < 2:
            breath_times.append(breath_time)
            mean_qrs.append(np.nan)
            std_qrs.append(np.nan)
            btb_var.append(np.nan)
        else:
            mean_val = np.mean(segment)
            std_val = np.std(segment)
            diffs = np.diff(segment)
            btb_val = np.std(diffs) if len(diffs) > 0 else np.nan
            breath_times.append(breath_time)
            mean_qrs.append(mean_val)
            std_qrs.append(std_val)
            btb_var.append(btb_val)
    return (np.array(breath_times),
            np.array(mean_qrs),
            np.array(std_qrs),
            np.array(btb_var))

def compute_peak_to_trough(edr_time, edr_signal, t_start, t_end):
    mask = (edr_time >= t_start) & (edr_time < t_end)
    segment = edr_signal[mask]
    if segment.size == 0:
        return np.nan
    return np.max(segment) - np.min(segment)

def compute_rising_slope(edr_time, edr_signal, t_start, t_peak):
    mask = (edr_time >= t_start) & (edr_time <= t_peak)
    seg_time = edr_time[mask]
    seg_signal = edr_signal[mask]
    if seg_time.size < 2:
        return np.nan
    # Slope can be estimated by the linear fit (slope)
    p = np.polyfit(seg_time, seg_signal, 1)
    return p[0]  # slope

def compute_area_under_curve(edr_time, edr_signal, t_start, t_end):
    mask = (edr_time >= t_start) & (edr_time <= t_end)
    seg_time = edr_time[mask]
    seg_signal = edr_signal[mask]
    if seg_time.size < 2:
        return np.nan
    return np.trapz(seg_signal, seg_time)


# -- 2.4 Plotting Function for EDR Breath Detection --

def plot_breath_cycles(edr_time, edr_signal, in_times, ex_times):
    """
    Plot the EDR signal and overlay the defined breath cycles,
    where each breath is defined as starting at an inspiration and ending at the subsequent expiration.
    """
    # Compute breath boundaries using your definition function.
    from breath_detection import define_breaths_insp_exp  # ensure it's imported
    breath_starts, breath_ends = define_breaths_insp_exp(in_times, ex_times)
    
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal')
    
    # Plot breath start markers (e.g., green circles)
    plt.plot(breath_starts, np.interp(breath_starts, edr_time, edr_signal),
             'go', label='Breath Start')
    # Plot breath end markers (e.g., red circles)
    plt.plot(breath_ends, np.interp(breath_ends, edr_time, edr_signal),
             'ro', label='Breath End')
    
    # Optionally, draw vertical lines at each boundary
    for t in breath_starts:
        plt.axvline(t, color='green', linestyle='--', alpha=0.5)
    for t in breath_ends:
        plt.axvline(t, color='red', linestyle='--', alpha=0.5)
    
    plt.title('Breath Cycles (Inspiration to Expiration)')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_edr_breaths(edr_time, edr_signal, in_times, ex_times):
    """
    Plot the EDR signal with detected inspiration and expiration points.
    """
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal (QRS Amplitude)')
    plt.plot(in_times, np.interp(in_times, edr_time, edr_signal), 'o', label='Inspirations')
    plt.plot(ex_times, np.interp(ex_times, edr_time, edr_signal), 'o', label='Expirations')
    plt.title('ECG-Derived Respiration with Detected Breaths')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

#########################################
# SECTION 3: FEATURE TABLE CONSTRUCTION
#########################################

def build_feature_table(patient, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10):
    # Access the exercise annotations
    exercises = session[dt.Exercise]
    all_ex_timestamps = exercises.timestamp
    full_start = float(np.min(all_ex_timestamps))
    full_end = float(np.max(all_ex_timestamps))
    print(f"build_feature_table: Using exercise interval {full_start:.1f}–{full_end:.1f} s")

    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]

    # Breath detection on the EDR signal
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    

    
    # Detect ECG R-peaks (using pan_tompkins here)
    R_peaks = detect_ecg_peaks(ecg_signal, fs)

    feature_rows = []
    seg_times = np.arange(full_start, full_end, interval_size)
    
    # Pre-compute QRS amplitude per breath stats (for beat-to-beat variability)
    btqrs_times, mean_qrs_breath, std_qrs_breath, btb_var_breath = compute_qrs_amp_per_breath(edr_time_qrs, edr_signal_qrs, in_times)
    
    for window_start in seg_times:
        window_end = window_start + interval_size
        true_insp_dur, true_exp_dur = compute_true_insp_exp_durations(in_times, ex_times, window_start, window_end)


        # EDR-derived breathing rate (BPM)
        edr_bpm = compute_bpm_from_inspiration_times(in_times, window_start, window_end)


        # Breath count and mean breath duration (from inspirations)
        n_breaths, mean_breath_duration = compute_breath_features(in_times, window_start, window_end)

        # ECG-derived heart rate (BPM) from R-peaks in the window
        window_peaks = R_peaks[(R_peaks/fs >= window_start) & (R_peaks/fs < window_end)]
        if len(window_peaks) < 2:
            ecg_bpm = np.nan
        else:
            rr = np.diff(window_peaks) / fs
            ecg_bpm = 60.0 / np.mean(rr)

        # QRS amplitude statistics in the window
        mask_qrs = (edr_time_qrs >= window_start) & (edr_time_qrs < window_end)
        qrs_window = edr_signal_qrs[mask_qrs]
        if len(qrs_window) < 1:
            qrs_mean = np.nan
            qrs_std = np.nan
        else:
            qrs_mean = np.mean(qrs_window)
            qrs_std = np.std(qrs_window)
        
        # Beat-to-beat variability (BTB) of QRS amplitude for breaths in the window
        mask_btb = (btqrs_times >= window_start) & (btqrs_times < window_end)
        if np.any(mask_btb):
            qrs_btb = np.nanmean(btb_var_breath[mask_btb])
        else:
            qrs_btb = np.nan
        
        # New metric: Peak-to-Trough of EDR signal in this window
        pt_diff = compute_peak_to_trough(edr_time_qrs, edr_signal_qrs, window_start, window_end)
        # New metric: Rising slope (for example, from window start to the first peak)
        # You would need to identify the peak within this window:
        mask = (edr_time_qrs >= window_start) & (edr_time_qrs < window_end)
        if np.any(mask):
            seg_time = edr_time_qrs[mask]
            seg_signal = edr_signal_qrs[mask]
            # Find the index of the maximum in the segment (peak)
            peak_idx = np.argmax(seg_signal)
            t_peak = seg_time[peak_idx]
            rising_slope = compute_rising_slope(edr_time_qrs, edr_signal_qrs, window_start, t_peak)
        else:
            rising_slope = np.nan

        # New metric: Area under the EDR curve in this window
        area = compute_area_under_curve(edr_time_qrs, edr_signal_qrs, window_start, window_end)

        row = {
            "subject_id": patient.id,
            "window_start": window_start,
            "window_end": window_end,
            "EDR_BPM": edr_bpm,
            "n_breaths": n_breaths,
            "MeanBreathDuration": mean_breath_duration,
            "ECG_BPM": ecg_bpm,
            "QRS_mean": qrs_mean,
            "QRS_std": qrs_std,
            "QRS_BTB": qrs_btb,
            "TrueInspDuration": true_insp_dur,
            "TrueExpDuration": true_exp_dur,
            "PeakToTrough": pt_diff,
            "RisingSlope": rising_slope,
            "AreaUnderCurve": area

        }
        feature_rows.append(row)
    
    feature_df = pd.DataFrame(feature_rows)
    return feature_df



def main():
        # --- SECTION 1: EDR Breath Detection ---
    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]
    edr_time = QRS_amplitude_resampled[0]
    edr_signal = QRS_amplitude_resampled[2]

    
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                                                 init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
                                                 adjust_factor=0.6)
    breath_starts, breath_ends = define_breaths_insp_exp(in_times, ex_times)

    plot_breath_cycles(edr_time, edr_signal, in_times, ex_times)


    plot_edr_breaths(edr_time_qrs, edr_signal_qrs, in_times, ex_times)
    


    # --- ECG Heart Rate Analysis ---
    hr_time, inst_hr, mean_hr = compute_ecg_heart_rate(filtered_ecg, fs)
    print(f"Heart Rate from ECG: {mean_hr:.2f} BPM (average)")
    
    plt.figure(figsize=(8, 4))
    plt.plot(hr_time, inst_hr, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    session = pt[dt.SeatedSession]  # or SupineSession depending on your use case
    exercise_ann = session[dt.Exercise]

    seg_times_hr, hr_10s = segment_ecg_hr(
        filtered_ecg,
        fs,
        start_time=np.min(exercise_ann.timestamp),
        end_time=np.max(exercise_ann.timestamp),
        interval_size=10
        )

    print("Segmented HR (10s intervals):", hr_10s)
    
    plt.figure(figsize=(8, 4))
    plt.plot(seg_times_hr, hr_10s, marker='o', linestyle='-', label='Instantaneous HR')
    plt.title('ECG-Derived Heart Rate (10s Intervals)')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (BPM)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    
    interval_size = 10  # 10-second samples for segmentation here
    seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio = [], [], [], []
    seg_times = np.arange(np.min(exercise.timestamp), np.max(exercise.timestamp), interval_size)

    for t0 in seg_times:
        t1 = t0 + interval_size
        mi, me = compute_mean_insp_exp_edr(in_times, ex_times, t0, t1)
        seg_times_edr.append(t0)
        edr_mean_insp.append(mi)
        edr_mean_exp.append(me)
        ratio = mi/me if (not np.isnan(mi) and not np.isnan(me) and me > 0) else np.nan
        edr_ie_ratio.append(ratio)
    
    print("=== EDR-derived Mean Intervals (10s windows) ===")
    for t0, mi, me, ratio in zip(seg_times_edr, edr_mean_insp, edr_mean_exp, edr_ie_ratio):
        print(f"[{t0:.0f}-{t0+interval_size:.0f}s]: Insp={mi:.2f}s, Exp={me:.2f}s, I:E={ratio:.2f}")
    
    
    # --- Build Feature Table ---
    feature_df = build_feature_table(record, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10)
    print("Feature Table:")
    print(feature_df)
    
    # 2) Create a label for each time window (e.g., "0–30s", "30–60s", etc.)
    labels = [
        f"{int(ws)}–{int(we)}s"
        for ws, we in zip(feature_df["window_start"], feature_df["window_end"])
    ]

    
    # --- Additional Plots for Feature Analysis ---
    # Calculate mid-time for each window from the feature table (using window length 30s)
    mid_times = feature_df['window_start'] + (30 / 2)

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['TrueInspDuration'], marker='o', linestyle='-', label='True Insp Duration (In→Ex)')
    plt.plot(mid_times, feature_df['TrueExpDuration'], marker='o', linestyle='-', label='True Exp Duration (Ex→In)')
    plt.title('True Inspiration and Expiration Durations')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()


    # Plot: Number of Breaths per Window
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['n_breaths'], marker='o', linestyle='-', label='Number of Breaths')
    plt.title('Number of Breaths (per 10s window)')
    plt.xlabel('Time (s)')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot: Mean Breath Duration vs. Breathing Cycle Duration
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['MeanBreathDuration'], marker='o', linestyle='-', label='Mean Breath Duration')
    plt.title('Mean Breath Duration')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # Plot: Breathing Rate per Minute
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['EDR_BPM'], marker='o', linestyle='-', label='Breathing Rate (BPM)')
    plt.title('Breathing Rate per Minute')
    plt.xlabel('Time (s)')
    plt.ylabel('BPM')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 3) Bar Plot: Breathing Rate per Minute (EDR_BPM)
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["EDR_BPM"], color="royalblue", alpha=0.7)
    plt.title("Breathing Rate per Minute (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("BPM")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

    # 4) Bar Plot: Mean Breath Duration
    plt.figure(figsize=(10, 6))
    plt.bar(labels, feature_df["MeanBreathDuration"], color="seagreen", alpha=0.7)
    plt.title("Mean Breath Duration (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("Duration (s)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()
    
    # --- Additional Plots for QRS Amplitude Statistics ---
    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_mean'], marker='o', linestyle='-', label='Mean QRS per Breath')
    plt.title('Mean QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Mean QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_std'], marker='o', linestyle='-', color='orange', label='Std of QRS per Breath')
    plt.title('Std of QRS Amplitude per Breath')
    plt.xlabel('Time (s)')
    plt.ylabel('Std QRS Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(mid_times, feature_df['QRS_BTB'], marker='o', linestyle='-', color='green', label='Beat-to-beat Variability')
    plt.title('Beat-to-beat Variability of QRS Amplitude')
    plt.xlabel('Time (s)')
    plt.ylabel('Std of Consecutive Differences')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Suppose feature_df is already built
    # Plot Peak-to-Trough
    plt.figure(figsize=(10, 6))
    plt.plot(feature_df['window_start'], feature_df['PeakToTrough'], '-o', label='Peak-to-Trough')
    plt.title("EDR Signal Peak-to-Trough Difference")
    plt.xlabel("Window Start Time (s)")
    plt.ylabel("Amplitude Difference")
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot Rising Slope
    plt.figure(figsize=(10, 6))
    plt.plot(feature_df['window_start'], feature_df['RisingSlope'], '-o', color='orange', label='Rising Slope')
    plt.title("EDR Signal Rising Slope")
    plt.xlabel("Window Start Time (s)")
    plt.ylabel("Slope (Amplitude/sec)")
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot Area Under Curve
    plt.figure(figsize=(10, 6))
    plt.plot(feature_df['window_start'], feature_df['AreaUnderCurve'], '-o', color='green', label='Area Under Curve')
    plt.title("Area Under the EDR Curve")
    plt.xlabel("Window Start Time (s)")
    plt.ylabel("Area (integral units)")
    plt.grid(True)
    plt.legend()
    plt.show()


if __name__ == "__main__":
    main()

# feature dataframe for the pr breath>

In [ ]:
import importlib
import features
importlib.reload(features)
from features import build_feature_table_by_breath
import pandas as pd
import diamonds.data as dt
from diamonds import load_patients
import preprocessing
importlib.reload(preprocessing)
import engzee_tompkins
importlib.reload(engzee_tompkins)
import new_rpeak_detector

ptt = load_patients(show_progress=True)


session_types = [dt.SeatedSession, dt.SupineSession]

feature_list = []

for pt in ptt:
    print(f"processing subject: {pt.id}")
    for sess_type in session_types:
        try:
            
            session = pt[sess_type]
        except KeyError:
            print(f"  subject {pt.id} does not have session type {sess_type.__name__}")
            continue

        try:
           
            EMG, ECG = pt[(sess_type, dt.EMG)].decompose()
            ecg_signal = -ECG.samples[:, 1]
            fs = ECG.fs

            
            filtered_ecg = preprocessing.preprocess_signal(ecg_signal, fs)
            R_peaks, RRI, RRI_resampled = preprocessing.get_RRI_from_signal(filtered_ecg, fs)
            QRS_amplitude, QRS_amplitude_resampled = preprocessing.get_QRS_amplitude(
                filtered_ecg, R_peaks, fs, default_window_len=0.1
            )

            _df = build_feature_table_by_breath(pt, session, ecg_signal, fs, QRS_amplitude_resampled)

            
            _df['session_type'] = sess_type.__name__
            feature_list.append(_df)
            print(f"  processed {sess_type.__name__} for subject {pt.id} with DataFrame shape {_df.shape}")
        except Exception as e:
            print(f" failed for subject {pt.id} for session {sess_type.__name__}: {e}")

if feature_list:
    features_df = pd.concat(feature_list, ignore_index=True)
    print(" Combined feature dataframe:")
    print(features_df)
    features_df.to_excel("all_patient_features_combined_breath2.xlsx", index=False)
else:
    print(" no feature data was generated!")


""" from build_feature_table import build_feature_table
import pandas as pd
import diamonds.data as dt
from diamonds import load_patients

ptt = load_patients(show_progress=True)
# Choose a session type (e.g., dt.SeatedSession)
session = ptt[0][dt.SeatedSession]  # example for the first patient

# Build feature table for a given patient/session
feature_df = build_feature_table(ptt[0], session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10)
feature_df.to_excel("all_patient_features.xlsx", index=False) """


In [ ]:




    def correlate_with_edr(self, edr_time, edr_signal, window_size=20.0, target_fs=10):
        from scipy.interpolate import interp1d
        from scipy.stats import pearsonr

        print("\n=== Correlation Analysis (EDR vs Gold) ===")

        breath_times = np.sort(np.concatenate([self.in_times, self.ex_times]))
        if len(breath_times) < 2:
            print("Not enough breath events for correlation.")
            return []

        # Create synthetic gold signal
        gold_signal = np.zeros_like(edr_time)
        gold_indices = np.searchsorted(edr_time, breath_times)
        gold_signal[gold_indices[gold_indices < len(gold_signal)]] = 1

        uniform_time = np.arange(self.start_time, self.end_time, 1.0 / target_fs)
        interp_edr = interp1d(edr_time, edr_signal, bounds_error=False, fill_value="extrapolate")
        interp_gold = interp1d(edr_time, gold_signal, bounds_error=False, fill_value=0)

        edr_resampled = interp_edr(uniform_time)
        gold_resampled = interp_gold(uniform_time)

        samples_per_win = int(window_size * target_fs)
        num_windows = len(uniform_time) // samples_per_win

        correlations = []
        for i in range(num_windows):
            start = i * samples_per_win
            end = start + samples_per_win
            edr_seg = edr_resampled[start:end]
            gold_seg = gold_resampled[start:end]

            if np.all(gold_seg == 0) or np.all(edr_seg == 0):
                correlations.append(np.nan)
            else:
                r, _ = pearsonr(edr_seg, gold_seg)
                correlations.append(r)

        correlations = np.array(correlations)
        median_corr = np.nanmedian(correlations)
        print(f"✅ Median correlation over {num_windows} windows: {median_corr:.3f}")

        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, 4))
        plt.plot(correlations, marker='o')
        plt.axhline(median_corr, color='red', linestyle='--', label=f"Median={median_corr:.2f}")
        plt.title("Windowed Correlation: EDR vs Gold")
        plt.xlabel("Window Index")
        plt.ylabel("Pearson Correlation")
        plt.grid(True)
        plt.legend()
        plt.show()

        return correlations, median_corr
    def compare_multiple_edr_signals(self, edr_list, window_size=20.0, target_fs=10):
        """
        Compare multiple EDR signals to gold standard in terms of windowed correlation.

        Parameters:
        - edr_list: list of tuples [(label, edr_time, edr_signal), ...]
        - window_size: size of sliding window in seconds
        - target_fs: resampling frequency (Hz)
        """

        from scipy.interpolate import interp1d
        from scipy.stats import pearsonr

        print("📊 Comparing multiple EDR signals to golden reference...")
        breath_times = np.sort(np.concatenate([self.in_times, self.ex_times]))

        if len(breath_times) < 2:
            print("Not enough gold events for correlation.")
            return

        # Step 1: Generate gold impulse signal
        gold_signal_template = None
        gold_corrs_all = {}

        for label, edr_time, edr_signal in edr_list:
            print(f"▶️ {label}...")

            # Create gold binary signal
            gold_signal = np.zeros_like(edr_time)
            gold_indices = np.searchsorted(edr_time, breath_times)
            gold_signal[gold_indices[gold_indices < len(gold_signal)]] = 1

            # Uniform resampling
            uniform_time = np.arange(self.start_time, self.end_time, 1.0 / target_fs)
            interp_edr = interp1d(edr_time, edr_signal, bounds_error=False, fill_value="extrapolate")
            interp_gold = interp1d(edr_time, gold_signal, bounds_error=False, fill_value=0)

            edr_resampled = interp_edr(uniform_time)
            gold_resampled = interp_gold(uniform_time)

            # Sliding window correlation
            samples_per_win = int(window_size * target_fs)
            num_windows = len(uniform_time) // samples_per_win
            correlations = []

            for i in range(num_windows):
                start = i * samples_per_win
                end = start + samples_per_win
                edr_seg = edr_resampled[start:end]
                gold_seg = gold_resampled[start:end]

                if np.all(gold_seg == 0) or np.all(edr_seg == 0):
                    correlations.append(np.nan)
                else:
                    r, _ = pearsonr(edr_seg, gold_seg)
                    correlations.append(r)

            correlations = np.array(correlations)
            gold_corrs_all[label] = correlations
            print(f"  → Median correlation: {np.nanmedian(correlations):.3f}")
""" 
        # Step 2: Boxplot
        plt.figure(figsize=(10, 5))
        data = [gold_corrs_all[label] for label in gold_corrs_all]
        labels = list(gold_corrs_all.keys())
        plt.boxplot(data, labels=labels, showmeans=True)
        plt.xticks(rotation=20)
        plt.ylabel("Windowed Pearson Correlation")
        plt.title("EDR vs Gold: Windowed Correlation Comparison")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        return gold_corrs_all """






In [ ]:
# features.py

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch
from scipy.interpolate import interp1d
import sys
import os

project_root = "C:/Users/Visnu/DIAMONDS"  
if project_root not in sys.path:
    sys.path.append(project_root)

import diamonds.data as dt

from pan_tompkins import detect_ecg_peaks
# We import define_complete_breath_cycles to get total breath duration from ex->ex
from breath_detection import (
    auto_detect_edr_breaths,
    define_breaths_insp_exp,
    define_complete_breath_cycles,
    compute_true_insp_exp_durations,
    compute_bpm_from_cycles,
    DEFAULT_MIN_BREATH_SEC
)
from diamonds_definitions import pt, session, exercise, ecg_signal, fs

import importlib
import preprocessing
importlib.reload(preprocessing)
import pan_tompkins
import engzee_tompkins
import new_rpeak_detector
importlib.reload(pan_tompkins)
importlib.reload(engzee_tompkins)
importlib.reload(new_rpeak_detector)

record = pt

# Then pick the method you want:
method_choice = "pan_tompkins"   # or "engzee" or "pan_tompkins"

# Now call the unified function:
results = preprocessing.extract_ecg_features(
    ecg_signal,
    fs,
    method=method_choice,
    RRI_fs=10,
    default_window_len=0.1,
    search_window_len=0.01,
    edr_method='R_peak'
)

# The dictionary 'results' now has everything:
filtered_ecg = results["filtered_ecg"]
R_peaks       = results["R_peaks"]
RRI           = results["RRI"]
RRI_resampled = results["RRI_resampled"]
QRS_amplitude = results["QRS_amp"]
QRS_amplitude_resampled = results["QRS_amp_resampled"]
BW_effect     = results["BW_effect"]
AM_effect     = results["AM_effect"]
R_peak_amplitude             = results["R_peak_amplitude"]
R_peak_amplitude_resampled   = results["R_peak_amplitude_resampled"]
edr_time_r_peak              = results["edr_time_r_peak"]
edr_signal_r_peak            = results["edr_signal_r_peak"]
edr_rri_time                 = results["edr_rri_time"]
edr_rri_signal               = results["edr_rri_signal"]

print(f"Method used: {method_choice}")
print(f"Detected {len(R_peaks)} R-peaks")


###############################################################
# Keep compute_true_insp_exp_durations (for I:E ratio)
###############################################################
def compute_true_insp_exp_durations(in_times, ex_times, t_start, t_end):
    insp_durations = []
    exp_durations = []

    # Ex → In = True Inspiration
    for e_time in ex_times:
        if e_time < t_start or e_time > t_end:
            continue
        next_in = in_times[in_times > e_time]
        if next_in.size > 0 and next_in[0] <= t_end:
            insp_durations.append(next_in[0] - e_time)

    # In → Ex = True Expiration
    for i_time in in_times:
        if i_time < t_start or i_time > t_end:
            continue
        next_ex = ex_times[ex_times > i_time]
        if next_ex.size > 0 and next_ex[0] <= t_end:
            exp_durations.append(next_ex[0] - i_time)

    return (
        np.mean(insp_durations) if insp_durations else np.nan,
        np.mean(exp_durations) if exp_durations else np.nan
    )

###############################################################
# Keep old compute_breath_features for n_breaths if you want,
# but user specifically said to remove the old mean breath
# duration from inspirations. So we remove references to it.
###############################################################
def compute_breath_count(in_times, t_start, t_end):
    """
    Compute how many inspirations (breaths) occur in [t_start, t_end].
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    return len(valid_in)


def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Compute the breathing rate (BPM) from inspiration times within a window.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    mean_interval = np.mean(np.diff(valid))
    return 60.0 / mean_interval


###############################################################
# We keep the QRS-related code for the user if needed but minimal changes
# are done. No changes to compute_qrs_amp_per_breath or compute_peak_to_trough
# because user only said to keep breath features and remove the old mean 
# breath duration from inspirations, which we did.
###############################################################
def compute_qrs_amp_per_breath(edr_time, edr_signal, in_times):
    breath_times = []
    mean_qrs = []
    std_qrs = []
    btb_var = []
    for i in range(len(in_times) - 1):
        start = in_times[i]
        end = in_times[i+1]
        mask = (edr_time >= start) & (edr_time < end)
        segment = edr_signal[mask]
        breath_time = 0.5 * (start + end)
        if len(segment) < 2:
            breath_times.append(breath_time)
            mean_qrs.append(np.nan)
            std_qrs.append(np.nan)
            btb_var.append(np.nan)
        else:
            mean_val = np.mean(segment)
            std_val = np.std(segment)
            diffs = np.diff(segment)
            btb_val = np.std(diffs) if len(diffs) > 0 else np.nan
            breath_times.append(breath_time)
            mean_qrs.append(mean_val)
            std_qrs.append(std_val)
            btb_var.append(btb_val)
    return (np.array(breath_times),
            np.array(mean_qrs),
            np.array(std_qrs),
            np.array(btb_var))


def compute_peak_to_trough(edr_time, edr_signal, t_start, t_end):
    mask = (edr_time >= t_start) & (edr_time < t_end)
    segment = edr_signal[mask]
    if segment.size == 0:
        return np.nan
    return np.max(segment) - np.min(segment)


###############################################################
# Plotting functions remain unchanged
###############################################################
def plot_breath_cycles(edr_time, edr_signal, in_times, ex_times):
    from breath_detection import define_breaths_insp_exp
    breath_starts, breath_ends = define_breaths_insp_exp(in_times, ex_times)
    
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal')
    
    plt.plot(breath_starts,
             np.interp(breath_starts, edr_time, edr_signal),
             'go', label='Breath Start')
    plt.plot(breath_ends,
             np.interp(breath_ends, edr_time, edr_signal),
             'ro', label='Breath End')

    for t in breath_starts:
        plt.axvline(t, color='green', linestyle='--', alpha=0.5)
    for t in breath_ends:
        plt.axvline(t, color='red', linestyle='--', alpha=0.5)
    
    plt.title('Breath Cycles (Inspiration to Expiration)')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_edr_breaths(edr_time, edr_signal, in_times, ex_times):
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal (QRS Amplitude)')
    plt.plot(in_times,
             np.interp(in_times, edr_time, edr_signal),
             'o', label='Inspirations')
    plt.plot(ex_times,
             np.interp(ex_times, edr_time, edr_signal),
             'o', label='Expirations')
    plt.title('ECG-Derived Respiration with Detected Breaths')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()


###############################################################
# SECTION 3: FEATURE TABLE CONSTRUCTION – minimal changes
#  - remove old "MeanBreathDuration" from inspirations
#  - get total breath durations from define_complete_breath_cycles
#  - compute I:E ratio from compute_true_insp_exp_durations
###############################################################
def build_feature_table(patient, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10):
    exercises = session[dt.Exercise]
    all_ex_timestamps = exercises.timestamp
    full_start = float(np.min(all_ex_timestamps))
    full_end = float(np.max(all_ex_timestamps))
    print(f"build_feature_table: Using exercise interval {full_start:.1f}–{full_end:.1f} s")

    # EDR from QRS_amplitude_resampled
    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]

    # 1) Breath detection
    in_times, ex_times = auto_detect_edr_breaths(
        edr_time_qrs, edr_signal_qrs,
        init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
        adjust_factor=0.6
    )

    feature_rows = []

    # Loop over each exercise segment
    for i in range(len(all_ex_timestamps) - 1):
        ex_start = all_ex_timestamps[i]
        ex_end = all_ex_timestamps[i + 1]

        # Subdivide this exercise period into 10-second windows
        sub_window_starts = np.arange(ex_start, ex_end, interval_size)

        for window_start in sub_window_starts:
            window_end = window_start + interval_size
            if window_end > ex_end:
                break  # Skip incomplete sub-window

            # (a) True Inspiration & Expiration durations in this window => I:E ratio
            true_insp_dur, true_exp_dur = compute_true_insp_exp_durations(
                in_times, ex_times, window_start, window_end
            )
            if (not np.isnan(true_insp_dur) and
                not np.isnan(true_exp_dur) and true_exp_dur != 0):
                ie_ratio = true_insp_dur / true_exp_dur
            else:
                ie_ratio = np.nan

            

            # (c) Local Ex/In times to define complete cycles
            local_ex = ex_times[(ex_times >= window_start) & (ex_times < window_end)]
            local_in = in_times[(in_times >= window_start) & (in_times < window_end)]
            b_starts, b_ends, i_durs, e_durs = define_complete_breath_cycles(local_in, local_ex)
            n_breaths = len(b_starts)

            # (b) EDR-derived breathing rate (BPM) from inspirations
            edr_bpm = compute_bpm_from_cycles(b_starts, b_ends, window_start, window_end)

            # => Average total breath duration from ex→ex
            if n_breaths > 0:
                total_durations = b_ends - b_starts
                avg_total_breath_duration = np.mean(total_durations)
            else:
                avg_total_breath_duration = np.nan

            row = {
                "subject_id": patient.id,
                "window_start": window_start,
                "window_end": window_end,
                "EDR_BPM": edr_bpm,
                "n_breaths": n_breaths,
                "AvgTotalBreathDuration": avg_total_breath_duration,
                "TrueInspDuration": true_insp_dur,
                "TrueExpDuration": true_exp_dur,
                "IEratio": ie_ratio
            }
            feature_rows.append(row)

    feature_df = pd.DataFrame(feature_rows)
    return feature_df



def main():
    # EDR data from QRS_amplitude_resampled
    edr_time = QRS_amplitude_resampled[0]
    edr_signal = QRS_amplitude_resampled[2]

    in_times, ex_times = auto_detect_edr_breaths(
        edr_time, edr_signal,
        init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
        adjust_factor=0.6
    )
    # for plotting
    plot_breath_cycles(edr_time, edr_signal, in_times, ex_times)
    plot_edr_breaths(edr_time, edr_signal, in_times, ex_times)

    session = pt[dt.SeatedSession]  # or SupineSession
    exercise_ann = session[dt.Exercise]

    interval_size = 10  # 10-second segments
    feature_df = build_feature_table(
        record, session, ecg_signal, fs,
        QRS_amplitude_resampled, interval_size=interval_size
    )
    print("Feature Table:")
    print(feature_df)

    # Create a label for each time window
    labels = [
        f"{int(ws)}–{int(we)}s"
        for ws, we in zip(feature_df["window_start"], feature_df["window_end"])
    ]
    # We'll plot using the midpoint of each window (30 used by original code, but now 10)
    mid_times = feature_df['window_start'] + (interval_size / 2)

    # 1) Plot True Insp/Exp
    plt.figure()
    plt.plot(mid_times, feature_df['TrueInspDuration'], 'o-', label='True Insp Duration')
    plt.plot(mid_times, feature_df['TrueExpDuration'], 'o-', label='True Exp Duration')
    plt.title('True Inspiration and Expiration Durations')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 2) Plot # of breaths
    plt.figure()
    plt.plot(mid_times, feature_df['n_breaths'], 'o-', label='Number of Breaths')
    plt.title('Number of Breaths (per 10s window)')
    plt.xlabel('Time (s)')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 3) Plot: AvgTotalBreathDuration
    plt.figure()
    plt.plot(mid_times, feature_df['AvgTotalBreathDuration'], 'o-', label='AvgTotalBreathDuration')
    plt.title('Average Total Breath Duration (Ex->Ex)')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 4) Plot: Breathing Rate per Minute
    plt.figure()
    plt.plot(mid_times, feature_df['EDR_BPM'], 'o-', label='Breathing Rate (BPM)')
    plt.title('Breathing Rate per Minute')
    plt.xlabel('Time (s)')
    plt.ylabel('BPM')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 5) Bar Plot: EDR_BPM
    plt.figure()
    plt.bar(labels, feature_df["EDR_BPM"], color="royalblue", alpha=0.7)
    plt.title("Breathing Rate per Minute (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("BPM")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

    # 6) Bar Plot: AvgTotalBreathDuration
    plt.figure()
    plt.bar(labels, feature_df["AvgTotalBreathDuration"], color="seagreen", alpha=0.7)
    plt.title("Average Total Breath Duration (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("Duration (s)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

DEFAULT_MIN_BREATH_SEC = 2.5

def detect_edr_breaths(edr_time, edr_signal, min_breath_sec=DEFAULT_MIN_BREATH_SEC):
    """
    Single-pass detection of inspiration and expiration events.
    Returns:
      - in_times: timestamps for inspirations (local minima of -edr_signal)
      - ex_times: timestamps for expirations (local maxima of edr_signal)
    """
    # Assuming a sampling rate of 10 Hz
    min_distance_samples = int(10 * min_breath_sec)
    in_peaks, _ = find_peaks(-edr_signal, distance=min_distance_samples)
    ex_peaks, _ = find_peaks(edr_signal, distance=min_distance_samples)
    in_times = edr_time[in_peaks]
    ex_times = edr_time[ex_peaks]
    return in_times, ex_times

def auto_detect_edr_breaths(edr_time, edr_signal, min_breath_sec=DEFAULT_MIN_BREATH_SEC):
    """
    Simplified version: just calls detect_edr_breaths once.
    """
    in_times, ex_times = detect_edr_breaths(edr_time, edr_signal, min_breath_sec=min_breath_sec)
    return in_times, ex_times

def compute_true_insp_exp_durations(in_times, ex_times, t_start, t_end):
    insp_durations = []
    exp_durations = []
    # Ex → In = True Inspiration durations
    for e_time in ex_times:
        if e_time < t_start or e_time > t_end:
            continue
        next_in = in_times[in_times > e_time]
        if next_in.size > 0 and next_in[0] <= t_end:
            insp_durations.append(next_in[0] - e_time)
    # In → Ex = True Expiration durations
    for i_time in in_times:
        if i_time < t_start or i_time > t_end:
            continue
        next_ex = ex_times[ex_times > i_time]
        if next_ex.size > 0 and next_ex[0] <= t_end:
            exp_durations.append(next_ex[0] - i_time)
    return (
        np.mean(insp_durations) if insp_durations else np.nan,
        np.mean(exp_durations) if exp_durations else np.nan
    )

def define_complete_breath_cycles(in_times, ex_times):
    """
    Define full breath cycles using Ex → In → Ex pattern.
    Returns:
      breath_starts: array of expiration timestamps (start of cycle)
      breath_ends:   array of expiration timestamps following an inspiration
      insp_durations: array of inspiration durations (time from expiration to inspiration)
      exp_durations: array of expiration durations (time from inspiration to next expiration)
    """
    breath_starts = []
    breath_ends = []
    insp_durations = []
    exp_durations = []
    for i in range(len(ex_times) - 1):
        start_ex = ex_times[i]
        next_in = in_times[in_times > start_ex]
        if next_in.size == 0:
            continue
        in_time = next_in[0]
        next_ex = ex_times[ex_times > in_time]
        if next_ex.size == 0:
            continue
        end_ex = next_ex[0]
        breath_starts.append(start_ex)
        breath_ends.append(end_ex)
        insp_durations.append(in_time - start_ex)
        exp_durations.append(end_ex - in_time)
    return (
        np.array(breath_starts),
        np.array(breath_ends),
        np.array(insp_durations),
        np.array(exp_durations)
    )

def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Compute the breathing rate (BPM) from inspiration times within a window.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    mean_interval = np.mean(np.diff(valid))
    return 60.0 / mean_interval







def define_breaths_insp_exp(in_times, ex_times):
    """
    Define each breath as starting at an inspiration event and ending at the subsequent expiration event.
    Returns:
      - breath_starts: array of inspiration times
      - breath_ends:   array of the next expiration times
    """
    breath_starts = []
    breath_ends = []
    
    in_times = np.sort(in_times)
    ex_times = np.sort(ex_times)
    
    for i_time in in_times:
        valid_ex = ex_times[ex_times > i_time]
        if valid_ex.size > 0:
            e_time = valid_ex[0]
            breath_starts.append(i_time)
            breath_ends.append(e_time)
    return np.array(breath_starts), np.array(breath_ends)

def define_breaths_insp_to_expend(edr_time, edr_signal, in_times, ex_times):
    """
    Define each full breath as starting at an inspiration event and ending at the expiration end.
    The expiration end is defined as the first local minimum in the EDR signal after the detected expiration.
    Returns:
      - breath_starts: array of inspiration times
      - breath_ends:   array of expiration end times
    """
    breath_starts = []
    breath_ends = []
    
    # Ensure the event arrays are sorted
    in_times = np.sort(in_times)
    ex_times = np.sort(ex_times)
    
    # For each inspiration, find the next expiration and then determine its "end"
    for i_time in in_times:
        valid_ex = ex_times[ex_times > i_time]
        if valid_ex.size == 0:
            continue
        # Use the first expiration after the inspiration as a starting point
        e_time = valid_ex[0]
        # Define a window from the expiration event to the next inspiration (or end of signal)
        next_in = in_times[in_times > e_time]
        if next_in.size > 0:
            window_end = next_in[0]
        else:
            window_end = edr_time[-1]
        # In the window [e_time, window_end], search for a local minimum of the EDR signal
        mask = (edr_time >= e_time) & (edr_time < window_end)
        if np.any(mask):
            sub_time = edr_time[mask]
            sub_signal = edr_signal[mask]
            # Local minima are peaks in the inverted signal
            minima_indices, _ = find_peaks(-sub_signal)
            if minima_indices.size > 0:
                exp_end = sub_time[minima_indices[0]]
            else:
                exp_end = e_time
        else:
            exp_end = e_time
        breath_starts.append(i_time)
        breath_ends.append(exp_end)
    
    return np.array(breath_starts), np.array(breath_ends)



In [ ]:
# features.py

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch
from scipy.interpolate import interp1d
import sys
import os

project_root = "C:/Users/Visnu/DIAMONDS"  
if project_root not in sys.path:
    sys.path.append(project_root)

import diamonds.data as dt

from pan_tompkins import detect_ecg_peaks
# We import define_complete_breath_cycles to get total breath duration from ex->ex
import importlib
import breath_detection
importlib.reload(breath_detection)
from breath_detection import (
    auto_detect_edr_breaths,
    define_breaths_insp_exp,
    define_complete_breath_cycles,
    compute_true_insp_exp_durations,
    DEFAULT_MIN_BREATH_SEC
)
from diamonds_definitions import pt, session, exercise, ecg_signal, fs

import importlib
import preprocessing
importlib.reload(preprocessing)
import pan_tompkins
import engzee_tompkins
import new_rpeak_detector
importlib.reload(pan_tompkins)
importlib.reload(engzee_tompkins)
importlib.reload(new_rpeak_detector)

record = pt

# Then pick the method you want:
method_choice = "pan_tompkins"   # or "engzee" or "pan_tompkins"

# Now call the unified function:
results = preprocessing.extract_ecg_features(
    ecg_signal,
    fs,
    method=method_choice,
    RRI_fs=10,
    default_window_len=0.1,
    search_window_len=0.01,
    edr_method='R_peak'
)

# The dictionary 'results' now has everything:
filtered_ecg = results["filtered_ecg"]
R_peaks       = results["R_peaks"]
RRI           = results["RRI"]
RRI_resampled = results["RRI_resampled"]
QRS_amplitude = results["QRS_amp"]
QRS_amplitude_resampled = results["QRS_amp_resampled"]
BW_effect     = results["BW_effect"]
AM_effect     = results["AM_effect"]
R_peak_amplitude             = results["R_peak_amplitude"]
R_peak_amplitude_resampled   = results["R_peak_amplitude_resampled"]
edr_time_r_peak              = results["edr_time_r_peak"]
edr_signal_r_peak            = results["edr_signal_r_peak"]
edr_rri_time                 = results["edr_rri_time"]
edr_rri_signal               = results["edr_rri_signal"]

print(f"Method used: {method_choice}")
print(f"Detected {len(R_peaks)} R-peaks")


###############################################################
# Keep compute_true_insp_exp_durations (for I:E ratio)
###############################################################
def compute_true_insp_exp_durations(in_times, ex_times, t_start, t_end):
    insp_durations = []
    exp_durations = []

    # Ex → In = True Inspiration
    for e_time in ex_times:
        if e_time < t_start or e_time > t_end:
            continue
        next_in = in_times[in_times > e_time]
        if next_in.size > 0 and next_in[0] <= t_end:
            insp_durations.append(next_in[0] - e_time)

    # In → Ex = True Expiration
    for i_time in in_times:
        if i_time < t_start or i_time > t_end:
            continue
        next_ex = ex_times[ex_times > i_time]
        if next_ex.size > 0 and next_ex[0] <= t_end:
            exp_durations.append(next_ex[0] - i_time)

    return (
        np.mean(insp_durations) if insp_durations else np.nan,
        np.mean(exp_durations) if exp_durations else np.nan
    )

###############################################################
# Keep old compute_breath_features for n_breaths if you want,
# but user specifically said to remove the old mean breath
# duration from inspirations. So we remove references to it.
###############################################################
def compute_breath_count(in_times, t_start, t_end):
    """
    Compute how many inspirations (breaths) occur in [t_start, t_end].
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    return len(valid_in)


def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Compute the breathing rate (BPM) from inspiration times within a window.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    mean_interval = np.mean(np.diff(valid))
    return 60.0 / mean_interval


###############################################################
# We keep the QRS-related code for the user if needed but minimal changes
# are done. No changes to compute_qrs_amp_per_breath or compute_peak_to_trough
# because user only said to keep breath features and remove the old mean 
# breath duration from inspirations, which we did.
###############################################################
def compute_qrs_amp_per_breath(edr_time, edr_signal, in_times):
    breath_times = []
    mean_qrs = []
    std_qrs = []
    btb_var = []
    for i in range(len(in_times) - 1):
        start = in_times[i]
        end = in_times[i+1]
        mask = (edr_time >= start) & (edr_time < end)
        segment = edr_signal[mask]
        breath_time = 0.5 * (start + end)
        if len(segment) < 2:
            breath_times.append(breath_time)
            mean_qrs.append(np.nan)
            std_qrs.append(np.nan)
            btb_var.append(np.nan)
        else:
            mean_val = np.mean(segment)
            std_val = np.std(segment)
            diffs = np.diff(segment)
            btb_val = np.std(diffs) if len(diffs) > 0 else np.nan
            breath_times.append(breath_time)
            mean_qrs.append(mean_val)
            std_qrs.append(std_val)
            btb_var.append(btb_val)
    return (np.array(breath_times),
            np.array(mean_qrs),
            np.array(std_qrs),
            np.array(btb_var))


def compute_peak_to_trough(edr_time, edr_signal, t_start, t_end):
    mask = (edr_time >= t_start) & (edr_time < t_end)
    segment = edr_signal[mask]
    if segment.size == 0:
        return np.nan
    return np.max(segment) - np.min(segment)


###############################################################
# Plotting functions remain unchanged
###############################################################
def plot_breath_cycles(edr_time, edr_signal, in_times, ex_times):
    from breath_detection import define_breaths_insp_exp
    breath_starts, breath_ends = define_breaths_insp_exp(in_times, ex_times)
    
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal')
    
    plt.plot(breath_starts,
             np.interp(breath_starts, edr_time, edr_signal),
             'go', label='Breath Start')
    plt.plot(breath_ends,
             np.interp(breath_ends, edr_time, edr_signal),
             'ro', label='Breath End')

    for t in breath_starts:
        plt.axvline(t, color='green', linestyle='--', alpha=0.5)
    for t in breath_ends:
        plt.axvline(t, color='red', linestyle='--', alpha=0.5)
    
    plt.title('Breath Cycles (Inspiration to Expiration)')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_edr_breaths(edr_time, edr_signal, in_times, ex_times):
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal (QRS Amplitude)')
    plt.plot(in_times,
             np.interp(in_times, edr_time, edr_signal),
             'o', label='Inspirations')
    plt.plot(ex_times,
             np.interp(ex_times, edr_time, edr_signal),
             'o', label='Expirations')
    plt.title('ECG-Derived Respiration with Detected Breaths')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()


###############################################################
# SECTION 3: FEATURE TABLE CONSTRUCTION – minimal changes
#  - remove old "MeanBreathDuration" from inspirations
#  - get total breath durations from define_complete_breath_cycles
#  - compute I:E ratio from compute_true_insp_exp_durations
###############################################################
def build_feature_table(patient, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10):
    exercises = session[dt.Exercise]
    all_ex_timestamps = exercises.timestamp
    full_start = float(np.min(all_ex_timestamps))
    full_end = float(np.max(all_ex_timestamps))
    print(f"build_feature_table: Using exercise interval {full_start:.1f}–{full_end:.1f} s")

    # EDR from QRS_amplitude_resampled
    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]

    # 1) Breath detection
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs, min_breath_sec=DEFAULT_MIN_BREATH_SEC)

    feature_rows = []

    # Loop over each exercise segment
    for i in range(len(all_ex_timestamps) - 1):
        ex_start = all_ex_timestamps[i]
        ex_end = all_ex_timestamps[i + 1]

        # Subdivide this exercise period into 10-second windows
        sub_window_starts = np.arange(ex_start, ex_end, interval_size)

        for window_start in sub_window_starts:
            window_end = window_start + interval_size
            if window_end > ex_end:
                break  # Skip incomplete sub-window

            # (a) True Inspiration & Expiration durations in this window => I:E ratio
            true_insp_dur, true_exp_dur = compute_true_insp_exp_durations(
                in_times, ex_times, window_start, window_end
            )
            if (not np.isnan(true_insp_dur) and
                not np.isnan(true_exp_dur) and true_exp_dur != 0):
                ie_ratio = true_insp_dur / true_exp_dur
            else:
                ie_ratio = np.nan

            

            # (c) Local Ex/In times to define complete cycles
            local_ex = ex_times[(ex_times >= window_start) & (ex_times < window_end)]
            local_in = in_times[(in_times >= window_start) & (in_times < window_end)]
            b_starts, b_ends, i_durs, e_durs = define_complete_breath_cycles(local_in, local_ex)
            n_breaths = len(b_starts)

            # (b) EDR-derived breathing rate (BPM) from inspirations
            edr_bpm = compute_bpm_from_inspiration_times(in_times, window_start, window_end)

            # => Average total breath duration from ex→ex
            if n_breaths > 0:
                total_durations = b_ends - b_starts
                avg_total_breath_duration = np.mean(total_durations)
            else:
                avg_total_breath_duration = np.nan

            row = {
                "subject_id": patient.id,
                "window_start": window_start,
                "window_end": window_end,
                "EDR_BPM": edr_bpm,
                "n_breaths": n_breaths,
                "AvgTotalBreathDuration": avg_total_breath_duration,
                "TrueInspDuration": true_insp_dur,
                "TrueExpDuration": true_exp_dur,
                "IEratio": ie_ratio
            }
            feature_rows.append(row)

    feature_df = pd.DataFrame(feature_rows)
    return feature_df



def main():
    # EDR data from QRS_amplitude_resampled
    edr_time = QRS_amplitude_resampled[0]
    edr_signal = QRS_amplitude_resampled[2]

    in_times, ex_times = auto_detect_edr_breaths(edr_time, edr_signal, min_breath_sec=DEFAULT_MIN_BREATH_SEC)
    # for plotting
    plot_breath_cycles(edr_time, edr_signal, in_times, ex_times)
    plot_edr_breaths(edr_time, edr_signal, in_times, ex_times)

    session = pt[dt.SeatedSession]  # or SupineSession
    exercise_ann = session[dt.Exercise]

    interval_size = 10  # 10-second segments
    feature_df = build_feature_table(
        record, session, ecg_signal, fs,
        QRS_amplitude_resampled, interval_size=interval_size
    )
    print("Feature Table:")
    print(feature_df)

    # Create a label for each time window
    labels = [
        f"{int(ws)}–{int(we)}s"
        for ws, we in zip(feature_df["window_start"], feature_df["window_end"])
    ]
    # We'll plot using the midpoint of each window (30 used by original code, but now 10)
    mid_times = feature_df['window_start'] + (interval_size / 2)

    # 1) Plot True Insp/Exp
    plt.figure()
    plt.plot(mid_times, feature_df['TrueInspDuration'], 'o-', label='True Insp Duration')
    plt.plot(mid_times, feature_df['TrueExpDuration'], 'o-', label='True Exp Duration')
    plt.title('True Inspiration and Expiration Durations')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 2) Plot # of breaths
    plt.figure()
    plt.plot(mid_times, feature_df['n_breaths'], 'o-', label='Number of Breaths')
    plt.title('Number of Breaths (per 10s window)')
    plt.xlabel('Time (s)')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 3) Plot: AvgTotalBreathDuration
    plt.figure()
    plt.plot(mid_times, feature_df['AvgTotalBreathDuration'], 'o-', label='AvgTotalBreathDuration')
    plt.title('Average Total Breath Duration (Ex->Ex)')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 4) Plot: Breathing Rate per Minute
    plt.figure()
    plt.plot(mid_times, feature_df['EDR_BPM'], 'o-', label='Breathing Rate (BPM)')
    plt.title('Breathing Rate per Minute')
    plt.xlabel('Time (s)')
    plt.ylabel('BPM')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 5) Bar Plot: EDR_BPM
    plt.figure()
    plt.bar(labels, feature_df["EDR_BPM"], color="royalblue", alpha=0.7)
    plt.title("Breathing Rate per Minute (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("BPM")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

    # 6) Bar Plot: AvgTotalBreathDuration
    plt.figure()
    plt.bar(labels, feature_df["AvgTotalBreathDuration"], color="seagreen", alpha=0.7)
    plt.title("Average Total Breath Duration (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("Duration (s)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()


# with step, all code nessacary:

In [ ]:
# features.py

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch
from scipy.interpolate import interp1d
import sys
import os

project_root = "C:/Users/Visnu/DIAMONDS"  
if project_root not in sys.path:
    sys.path.append(project_root)

import diamonds.data as dt

from pan_tompkins import detect_ecg_peaks
# We import define_complete_breath_cycles to get total breath duration from ex->ex
import importlib
import breath_detection
importlib.reload(breath_detection)
from breath_detection import (
    auto_detect_edr_breaths,
    define_breaths_insp_exp,
    define_complete_breath_cycles,
    compute_true_insp_exp_durations,
    DEFAULT_MIN_BREATH_SEC
)
from diamonds_definitions import pt, session, exercise, ecg_signal, fs

import importlib
import preprocessing
importlib.reload(preprocessing)
import pan_tompkins
import engzee_tompkins
import new_rpeak_detector
importlib.reload(pan_tompkins)
importlib.reload(engzee_tompkins)
importlib.reload(new_rpeak_detector)

record = pt

# Then pick the method you want:
method_choice = "pan_tompkins"   # or "engzee" or "pan_tompkins"

# Now call the unified function:
results = preprocessing.extract_ecg_features(
    ecg_signal,
    fs,
    method=method_choice,
    RRI_fs=10,
    default_window_len=0.1,
    search_window_len=0.01,
    edr_method='R_peak'
)

# The dictionary 'results' now has everything:
filtered_ecg = results["filtered_ecg"]
R_peaks       = results["R_peaks"]
RRI           = results["RRI"]
RRI_resampled = results["RRI_resampled"]
QRS_amplitude = results["QRS_amp"]
QRS_amplitude_resampled = results["QRS_amp_resampled"]
BW_effect     = results["BW_effect"]
AM_effect     = results["AM_effect"]
R_peak_amplitude             = results["R_peak_amplitude"]
R_peak_amplitude_resampled   = results["R_peak_amplitude_resampled"]
edr_time_r_peak              = results["edr_time_r_peak"]
edr_signal_r_peak            = results["edr_signal_r_peak"]
edr_rri_time                 = results["edr_rri_time"]
edr_rri_signal               = results["edr_rri_signal"]

print(f"Method used: {method_choice}")
print(f"Detected {len(R_peaks)} R-peaks")


###############################################################
# Keep compute_true_insp_exp_durations (for I:E ratio)
###############################################################
def compute_true_insp_exp_durations(in_times, ex_times, t_start, t_end):
    insp_durations = []
    exp_durations = []

    # Ex → In = True Inspiration
    for e_time in ex_times:
        if e_time < t_start or e_time > t_end:
            continue
        next_in = in_times[in_times > e_time]
        if next_in.size > 0 and next_in[0] <= t_end:
            insp_durations.append(next_in[0] - e_time)

    # In → Ex = True Expiration
    for i_time in in_times:
        if i_time < t_start or i_time > t_end:
            continue
        next_ex = ex_times[ex_times > i_time]
        if next_ex.size > 0 and next_ex[0] <= t_end:
            exp_durations.append(next_ex[0] - i_time)

    return (
        np.mean(insp_durations) if insp_durations else np.nan,
        np.mean(exp_durations) if exp_durations else np.nan
    )

###############################################################
# Keep old compute_breath_features for n_breaths if you want,
# but user specifically said to remove the old mean breath
# duration from inspirations. So we remove references to it.
###############################################################
def compute_breath_count(in_times, t_start, t_end):
    """
    Compute how many inspirations (breaths) occur in [t_start, t_end].
    """
    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    return len(valid_in)


def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Compute the breathing rate (BPM) from inspiration times within a window.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    mean_interval = np.mean(np.diff(valid))
    return 60.0 / mean_interval


###############################################################
# We keep the QRS-related code for the user if needed but minimal changes
# are done. No changes to compute_qrs_amp_per_breath or compute_peak_to_trough
# because user only said to keep breath features and remove the old mean 
# breath duration from inspirations, which we did.
###############################################################
def compute_qrs_amp_per_breath(edr_time, edr_signal, in_times):
    breath_times = []
    mean_qrs = []
    std_qrs = []
    btb_var = []
    for i in range(len(in_times) - 1):
        start = in_times[i]
        end = in_times[i+1]
        mask = (edr_time >= start) & (edr_time < end)
        segment = edr_signal[mask]
        breath_time = 0.5 * (start + end)
        if len(segment) < 2:
            breath_times.append(breath_time)
            mean_qrs.append(np.nan)
            std_qrs.append(np.nan)
            btb_var.append(np.nan)
        else:
            mean_val = np.mean(segment)
            std_val = np.std(segment)
            diffs = np.diff(segment)
            btb_val = np.std(diffs) if len(diffs) > 0 else np.nan
            breath_times.append(breath_time)
            mean_qrs.append(mean_val)
            std_qrs.append(std_val)
            btb_var.append(btb_val)
    return (np.array(breath_times),
            np.array(mean_qrs),
            np.array(std_qrs),
            np.array(btb_var))


def compute_peak_to_trough(edr_time, edr_signal, t_start, t_end):
    mask = (edr_time >= t_start) & (edr_time < t_end)
    segment = edr_signal[mask]
    if segment.size == 0:
        return np.nan
    return np.max(segment) - np.min(segment)


###############################################################
# Plotting functions remain unchanged
###############################################################
def plot_breath_cycles(edr_time, edr_signal, in_times, ex_times):
    from breath_detection import define_breaths_insp_exp
    breath_starts, breath_ends = define_breaths_insp_exp(in_times, ex_times)
    
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal')
    
    plt.plot(breath_starts,
             np.interp(breath_starts, edr_time, edr_signal),
             'go', label='Breath Start')
    plt.plot(breath_ends,
             np.interp(breath_ends, edr_time, edr_signal),
             'ro', label='Breath End')

    for t in breath_starts:
        plt.axvline(t, color='green', linestyle='--', alpha=0.5)
    for t in breath_ends:
        plt.axvline(t, color='red', linestyle='--', alpha=0.5)
    
    plt.title('Breath Cycles (Inspiration to Expiration)')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_edr_breaths(edr_time, edr_signal, in_times, ex_times):
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal (QRS Amplitude)')
    plt.plot(in_times,
             np.interp(in_times, edr_time, edr_signal),
             'o', label='Inspirations')
    plt.plot(ex_times,
             np.interp(ex_times, edr_time, edr_signal),
             'o', label='Expirations')
    plt.title('ECG-Derived Respiration with Detected Breaths')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()


###############################################################
# SECTION 3: FEATURE TABLE CONSTRUCTION – minimal changes
#  - remove old "MeanBreathDuration" from inspirations
#  - get total breath durations from define_complete_breath_cycles
#  - compute I:E ratio from compute_true_insp_exp_durations
###############################################################
def build_feature_table(patient, session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10, step_len=5):
    exercises = session[dt.Exercise]
    all_ex_timestamps = exercises.timestamp
    full_start = float(np.min(all_ex_timestamps))
    full_end = float(np.max(all_ex_timestamps))
    print(f"build_feature_table: Using exercise interval {full_start:.1f}–{full_end:.1f} s")

    # EDR from QRS_amplitude_resampled
    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]

    # 1) Breath detection
    in_times, ex_times = auto_detect_edr_breaths(edr_time_qrs, edr_signal_qrs,
                            init_min_breath_sec=3.704561483567398,
                            adjust_factor=0.5534217859265204,
                            max_iterations=5,
                            tol=0.08570017284990156,
                            min_allowed=0.5,
                            max_allowed=10.0,
                            adaptive=True)

    feature_rows = []

    # Loop over each exercise segment
    for i in range(len(all_ex_timestamps) - 1):
        ex_start = all_ex_timestamps[i]
        ex_end = all_ex_timestamps[i + 1]

        # Subdivide this exercise period into 10-second windows
        sub_window_starts = np.arange(ex_start, ex_end - interval_size, step_len)

        for window_start in sub_window_starts:
            window_end = window_start + interval_size
            if window_end > ex_end:
                break  # Skip incomplete sub-window

            # Safety check if it goes slightly over
            if window_end > ex_end:
                break

            # (a) True Inspiration & Expiration durations in this window => I:E ratio
            true_insp_dur, true_exp_dur = compute_true_insp_exp_durations(
                in_times, ex_times, window_start, window_end
            )
            if (not np.isnan(true_insp_dur) and
                not np.isnan(true_exp_dur) and true_exp_dur != 0):
                ie_ratio = true_insp_dur / true_exp_dur
            else:
                ie_ratio = np.nan

            

            # (c) Local Ex/In times to define complete cycles
            local_ex = ex_times[(ex_times >= window_start) & (ex_times < window_end)]
            local_in = in_times[(in_times >= window_start) & (in_times < window_end)]
            b_starts, b_ends, i_durs, e_durs = define_complete_breath_cycles(local_in, local_ex)
            n_breaths = len(b_starts)

            # (b) EDR-derived breathing rate (BPM) from inspirations
            edr_bpm = compute_bpm_from_inspiration_times(in_times, window_start, window_end)

            # => Average total breath duration from ex→ex
            if n_breaths > 0:
                total_durations = b_ends - b_starts
                avg_total_breath_duration = np.mean(total_durations)
            else:
                avg_total_breath_duration = np.nan

            row = {
                "subject_id": patient.id,
                "window_start": window_start,
                "window_end": window_end,
                "EDR_BPM": edr_bpm,
                "n_breaths": n_breaths,
                "AvgTotalBreathDuration": avg_total_breath_duration,
                "TrueInspDuration": true_insp_dur,
                "TrueExpDuration": true_exp_dur,
                "IEratio": ie_ratio
            }
            feature_rows.append(row)

    feature_df = pd.DataFrame(feature_rows)
    return feature_df



def main():
    # EDR data from QRS_amplitude_resampled
    edr_time = QRS_amplitude_resampled[0]
    edr_signal = QRS_amplitude_resampled[2]

    in_times, ex_times = auto_detect_edr_breaths(
        edr_time, edr_signal,
        init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
        adjust_factor=0.6
    )
    # for plotting
    plot_breath_cycles(edr_time, edr_signal, in_times, ex_times)
    plot_edr_breaths(edr_time, edr_signal, in_times, ex_times)

    session = pt[dt.SeatedSession]  # or SupineSession
    exercise_ann = session[dt.Exercise]

    interval_size = 10  # 10-second segments
    feature_df = build_feature_table(
        record, session, ecg_signal, fs,
        QRS_amplitude_resampled, interval_size=interval_size, step_len=5
    )
    print("Feature Table:")
    print(feature_df)

    # Create a label for each time window
    labels = [
        f"{int(ws)}–{int(we)}s"
        for ws, we in zip(feature_df["window_start"], feature_df["window_end"])
    ]
    # We'll plot using the midpoint of each window (30 used by original code, but now 10)
    mid_times = feature_df['window_start'] + (interval_size / 2)

    # 1) Plot True Insp/Exp
    plt.figure()
    plt.plot(mid_times, feature_df['TrueInspDuration'], 'o-', label='True Insp Duration')
    plt.plot(mid_times, feature_df['TrueExpDuration'], 'o-', label='True Exp Duration')
    plt.title('True Inspiration and Expiration Durations')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 2) Plot # of breaths
    plt.figure()
    plt.plot(mid_times, feature_df['n_breaths'], 'o-', label='Number of Breaths')
    plt.title('Number of Breaths (per 10s window)')
    plt.xlabel('Time (s)')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 3) Plot: AvgTotalBreathDuration
    plt.figure()
    plt.plot(mid_times, feature_df['AvgTotalBreathDuration'], 'o-', label='AvgTotalBreathDuration')
    plt.title('Average Total Breath Duration (Ex->Ex)')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 4) Plot: Breathing Rate per Minute
    plt.figure()
    plt.plot(mid_times, feature_df['EDR_BPM'], 'o-', label='Breathing Rate (BPM)')
    plt.title('Breathing Rate per Minute')
    plt.xlabel('Time (s)')
    plt.ylabel('BPM')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 5) Bar Plot: EDR_BPM
    plt.figure()
    plt.bar(labels, feature_df["EDR_BPM"], color="royalblue", alpha=0.7)
    plt.title("Breathing Rate per Minute (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("BPM")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

    # 6) Bar Plot: AvgTotalBreathDuration
    plt.figure()
    plt.bar(labels, feature_df["AvgTotalBreathDuration"], color="seagreen", alpha=0.7)
    plt.title("Average Total Breath Duration (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("Duration (s)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()


In [ ]:
import sys
import pandas as pd
project_root = "C:/Users/Visnu/DIAMONDS"  

if project_root not in sys.path:
    sys.path.append(project_root)
from diamonds import data as dt
import numpy as np
import matplotlib.pyplot as plt
from diamonds_definitions import pt

class GoldenBreathAnalyzer:
    def __init__(self, patient, session_type=dt.SeatedSession, 
                 interval_size=10, step_len=5):
        self.record = patient
        self.session = self.record[session_type]
        self.breaths = self.session[dt.Breaths]
        self.exercise = self.session[dt.Exercise]
        self.interval_size = interval_size
        self.step_len = step_len

        self.in_times, self.ex_times = self._get_golden_breath_events()
        self.start_time = np.min(self.exercise.timestamp)
        self.end_time = np.max(self.exercise.timestamp)

        print(f"Exercise interval: {self.start_time:.1f}s to {self.end_time:.1f}s")
    def _get_golden_breath_events(self):
        in_times = np.array([t for t, l in zip(self.breaths.timestamp, self.breaths.ann) if l == '(In'])
        ex_times = np.array([t for t, l in zip(self.breaths.timestamp, self.breaths.ann) if l == '(Ex'])
        return in_times, ex_times

    def _annotate_exercises(self, ax):
        for tstamp, label in zip(self.exercise.timestamp, self.exercise.ann):
            ax.axvline(tstamp, color='magenta', linestyle='--', alpha=0.5)
            ax.text(tstamp, ax.get_ylim()[1] * 0.95, label,
                    rotation=90, verticalalignment='top', color='magenta')

    def _compute_bpm(self, t_start, t_end):
        valid_in = self.in_times[(self.in_times >= t_start) & (self.in_times <= t_end)]
        if len(valid_in) < 2:
            return np.nan
        return 60.0 / np.mean(np.diff(valid_in))

    def segment_bpm(self):
        """
        Return arrays of (segment_starts, BPM_values) using overlapping windows.
        """
        seg_starts = np.arange(
            self.start_time,
            self.end_time - self.interval_size,
            self.step_len
        )
        bpm_vals = []
        for t_start in seg_starts:
            t_end = t_start + self.interval_size
            bpm_vals.append(self._compute_bpm(t_start, t_end))
        return seg_starts, np.array(bpm_vals)
    
    def segment_true_insp_exp_durations(self):
        """
        For each overlapping window, compute the mean inspiration and expiration durations
        (Ex→In = inspiration, In→Ex = expiration).
        Returns (segment_starts, array_of_insp_durs, array_of_exp_durs).
        """
        seg_starts = np.arange(
            self.start_time,
            self.end_time - self.interval_size,
            self.step_len
        )
        true_insp_durations = []
        true_exp_durations = []

        for t_start in seg_starts:
            t_end = t_start + self.interval_size

            # Collect all (Ex -> In) durations in [t_start, t_end]
            insp_durations = []
            for e_time in self.ex_times:
                if e_time < t_start or e_time > t_end:
                    continue
                next_in = self.in_times[self.in_times > e_time]
                if next_in.size > 0 and next_in[0] <= t_end:
                    insp_durations.append(next_in[0] - e_time)

            # Collect all (In -> Ex) durations in [t_start, t_end]
            exp_durations = []
            for i_time in self.in_times:
                if i_time < t_start or i_time > t_end:
                    continue
                next_ex = self.ex_times[self.ex_times > i_time]
                if next_ex.size > 0 and next_ex[0] <= t_end:
                    exp_durations.append(next_ex[0] - i_time)

            # Compute mean if they exist
            mean_insp = np.mean(insp_durations) if insp_durations else np.nan
            mean_exp = np.mean(exp_durations) if exp_durations else np.nan

            true_insp_durations.append(mean_insp)
            true_exp_durations.append(mean_exp)

        return seg_starts, np.array(true_insp_durations), np.array(true_exp_durations)

    def segment_mean_intervals(self):
        seg_times = np.arange(self.start_time, self.end_time, self.interval_size)
        mean_insp_durations = []
        mean_exp_durations = []
        ie_ratios = []

        for t_start in seg_times:
            t_end = t_start + self.interval_size

            # --- Ex → In (Inspiration) ---
            insp_durations = []
            for e_time in self.ex_times:
                if e_time < t_start or e_time > t_end:
                    continue
                next_in = self.in_times[self.in_times > e_time]
                if next_in.size > 0 and next_in[0] <= t_end:
                    insp_durations.append(next_in[0] - e_time)

            # --- In → Ex (Expiration) ---
            exp_durations = []
            for i_time in self.in_times:
                if i_time < t_start or i_time > t_end:
                    continue
                next_ex = self.ex_times[self.ex_times > i_time]
                if next_ex.size > 0 and next_ex[0] <= t_end:
                    exp_durations.append(next_ex[0] - i_time)

            # --- Compute Means and I:E Ratio ---
            mi = np.mean(insp_durations) if insp_durations else np.nan
            me = np.mean(exp_durations) if exp_durations else np.nan
            ratio = mi / me if not np.isnan(mi) and not np.isnan(me) and me > 0 else np.nan

            mean_insp_durations.append(mi)
            mean_exp_durations.append(me)
            ie_ratios.append(ratio)

        return seg_times, np.array(mean_insp_durations), np.array(mean_exp_durations), np.array(ie_ratios)

    def plot_bpm(self, seg_times, bpm_vals):
        plt.figure(figsize=(8, 4))
        ax = plt.gca()
        ax.plot(seg_times, bpm_vals, '-o', label='Golden BPM (Inspiration)')
        ax.set_title("Golden Standard: BPM in Exercise Intervals")
        ax.set_xlabel("Segment Start Time (s)")
        ax.set_ylabel("Breaths Per Minute")
        ax.grid(True)
        ax.legend()
        self._annotate_exercises(ax)
        plt.show()

    def plot_true_insp_exp_durations(self, seg_times, true_insp, true_exp):
        plt.figure(figsize=(10, 4))
        plt.plot(seg_times, true_insp, '-o', label="True Inspiration (Ex→In)")
        plt.plot(seg_times, true_exp, '-o', label="True Expiration (In→Ex)")
        plt.title("True Inspiration & Expiration Durations (Gold Standard)")
        plt.xlabel("Segment Start Time (s)")
        plt.ylabel("Duration (s)")
        plt.grid(True)
        plt.legend()
        self._annotate_exercises(plt.gca())
        plt.show()

    def plot_ie_ratio(self, seg_times, ie_ratios):
        plt.figure(figsize=(8, 4))
        plt.plot(seg_times, ie_ratios, '-o', color='orange', label="I:E Ratio")
        plt.title("Inspiration to Expiration Ratio (I:E)")
        plt.xlabel("Segment Start Time (s)")
        plt.ylabel("Ratio")
        plt.grid(True)
        plt.legend()
        self._annotate_exercises(plt.gca())
        plt.show()
    
    def define_complete_breath_cycles(self):
        """
        Extract full breath cycles from the entire recording using gold standard:
         Ex -> In -> Ex
        Returns arrays of breath_starts, breath_ends, inspiration_durations, expiration_durations.
        """
        breath_starts = []
        breath_ends = []
        insp_durations = []
        exp_durations = []

        for i in range(len(self.ex_times) - 1):
            start_ex = self.ex_times[i]
            next_in = self.in_times[self.in_times > start_ex]
            if next_in.size == 0:
                continue
            in_time = next_in[0]

            next_ex = self.ex_times[self.ex_times > in_time]
            if next_ex.size == 0:
                continue
            end_ex = next_ex[0]

            breath_starts.append(start_ex)
            breath_ends.append(end_ex)
            insp_durations.append(in_time - start_ex)
            exp_durations.append(end_ex - in_time)

        return (
            np.array(breath_starts),
            np.array(breath_ends),
            np.array(insp_durations),
            np.array(exp_durations)
        )

    def print_total_breath_count(self):
        total = len(self.breaths.ann)
        total_in = len(self.in_times)
        total_ex = len(self.ex_times)
        est_cycles = min(total_in, total_ex)
        print(f"Total breath events: {total}")
        print(f"Inspirations: {total_in}, Expirations: {total_ex}")
        print(f"Estimated breath cycles (In↔Ex pairs): {est_cycles}")
    
    def print_breath_counts_per_interval(self):
        seg_times = np.arange(self.start_time, self.end_time, self.interval_size)
        print(f"\nBreath event counts per {self.interval_size}-second interval:")
        for t0 in seg_times:
            t1 = t0 + self.interval_size
            mask = (np.array(self.breaths.timestamp) >= t0) & (np.array(self.breaths.timestamp) < t1)
            print(f"[{t0:.0f}-{t1:.0f}s]: {np.sum(mask)} events")

    def run_analysis(self):
        print("\n=== Running Golden Breath Analysis ===")

        # 1. BPM over time
        bpm_times, bpm_vals = self.segment_bpm()
        self.plot_bpm(bpm_times, bpm_vals)

        # 2. True Inspiration and Expiration Durations
        seg_times, true_insp, true_exp = self.segment_true_insp_exp_durations()
        self.plot_true_insp_exp_durations(seg_times, true_insp, true_exp)

        # 3. I:E Ratio Plot (from true durations)
        ie_ratios = true_insp / true_exp
        self.plot_ie_ratio(seg_times, ie_ratios)

        # 4. Print per-segment summary
        print("\n=== Segment-wise Duration Summary ===")
        for t0, insp, exp, ratio in zip(seg_times, true_insp, true_exp, ie_ratios):
            t1 = t0 + self.interval_size
            print(f"[{t0:.0f}-{t1:.0f}s] Insp={insp:.2f}s, Exp={exp:.2f}s, I:E={ratio:.2f}")

        # 5. BPM Summary
        print("\n=== BPM Summary ===")
        for t0, bpm in zip(bpm_times, bpm_vals):
            t1 = t0 + self.interval_size
            print(f"[{t0:.0f}-{t1:.0f}s]: BPM={bpm:.2f}")

        # 6. Total Gold Annotations
        self.print_total_breath_count()
        self.print_breath_counts_per_interval()

        # 7. Total valid breath cycles using Ex→In→Ex
        starts, ends, insp_durs, exp_durs = self.define_complete_breath_cycles()
        print(f"\n✅ Total valid breath cycles (Ex→In→Ex): {len(starts)}")

    def build_golden_table(self):
        """Build a golden standard metrics table with gold_ prefixes matching the image structure."""
        seg_times = np.arange(self.start_time, self.end_time, self.interval_size)
        rows = []
        
        # Get all required metrics
        bpm_times, bpm_vals = self.segment_bpm()
        seg_times, true_insp, true_exp = self.segment_true_insp_exp_durations()
        ie_ratios = true_insp / true_exp
        
        # Get number of complete breath cycles per interval
        starts, ends, insp_durs, exp_durs = self.define_complete_breath_cycles()
        cycle_counts = []
        for t_start in seg_times:
            t_end = t_start + self.interval_size
            count = np.sum((starts >= t_start) & (ends < t_end))
            cycle_counts.append(count)
        
        # Calculate average total breath duration (insp + exp)
        avg_total_duration = []
        for t_start in seg_times:
            t_end = t_start + self.interval_size
            mask = (starts >= t_start) & (ends < t_end)
            durations = insp_durs[mask] + exp_durs[mask]
            avg_total_duration.append(np.mean(durations) if len(durations) > 0 else np.nan)
        
        # Build the table with gold_ prefixes
        for t_start, bpm, n_breaths, total_dur, insp, exp, ratio in zip(
            seg_times, bpm_vals, cycle_counts, avg_total_duration, 
            true_insp, true_exp, ie_ratios):
            
            row = {
                "subject_id": pt.id,  # Assuming your patient object has subject_id
                "window_start": t_start,
                "window_end": t_start + self.interval_size,
                "gold_BPM": bpm,
                "gold_n_breaths": n_breaths,
                "gold_AvgTotalBreathDuration": total_dur,
                "gold_TrueInspDuration": insp,
                "gold_TrueExpDuration": exp,
                "gold_IEratio": ratio
            }
            rows.append(row)
        
        return pd.DataFrame(rows)






In [ ]:
import importlib
import golden_analysis
importlib.reload(golden_analysis)

from golden_analysis import GoldenBreathAnalyzer
session_types = [dt.SeatedSession, dt.SupineSession]

# Assuming you have edr_time and edr_signal (or any signal you'd like to visualize)
analyzer = GoldenBreathAnalyzer(pt, interval_size=10, step_len=5)
analyzer.run_analysis()





In [ ]:
import importlib
import golden_analysis
importlib.reload(golden_analysis)
from golden_analysis import GoldenBreathAnalyzer
import pandas as pd
import diamonds.data as dt
from diamonds import load_patients

# Load patients
ptt = load_patients(show_progress=True)
session_types = [dt.SeatedSession, dt.SupineSession]

golden_list = []
example_shown = False  # Flag to show only one patient's result

for pt in ptt:
    for sess_type in session_types:
        try:
            session = pt[sess_type]
        except KeyError:
            continue  # Skip if session type doesn't exist

        try:
            analyzer = GoldenBreathAnalyzer(pt, session_type=sess_type, interval_size=10, step_len=5)
            golden_df = analyzer.build_golden_table()

            golden_df['session_type'] = sess_type.__name__
            golden_df['subject_id'] = pt.id
            golden_list.append(golden_df)

            # Show preview for only one patient
            if not example_shown:
                print(f"\n📌 Example: Subject {pt.id} - {sess_type.__name__}")
                print(golden_df.head())  # Show first few rows
                example_shown = True

        except Exception as e:
            print(f"❌ Failed: {pt.id} - {sess_type.__name__}: {e}")

# Save full combined DataFrame to Excel
if golden_list:
    combined_df = pd.concat(golden_list, ignore_index=True)
    combined_df.to_excel("all_patient_golden_metrics_combined.xlsx", index=False)
    print("\n✅ Saved full results to 'all_patient_golden_metrics_combined.xlsx'")
else:
    print("⚠️ No golden data was generated.") 


In [ ]:
import importlib
import features_3
importlib.reload(features_3)
from features_3 import build_feature_table
import pandas as pd
import diamonds.data as dt
from diamonds import load_patients
import preprocessing
importlib.reload(preprocessing)
import engzee_tompkins
importlib.reload(engzee_tompkins)
import new_rpeak_detector

ptt = load_patients(show_progress=True)


session_types = [dt.SeatedSession, dt.SupineSession]

feature_list = []

for pt in ptt:
    print(f"processing subject: {pt.id}")
    for sess_type in session_types:
        try:
            
            session = pt[sess_type]
        except KeyError:
            print(f"  subject {pt.id} does not have session type {sess_type.__name__}")
            continue

        try:
           
            EMG, ECG = pt[(sess_type, dt.EMG)].decompose()
            ecg_signal = -ECG.samples[:, 1]
            fs = ECG.fs

            
            filtered_ecg = preprocessing.preprocess_signal(ecg_signal, fs)
            R_peaks, RRI, RRI_resampled = preprocessing.get_RRI_from_signal(filtered_ecg, fs)
            QRS_amplitude, QRS_amplitude_resampled = preprocessing.get_QRS_amplitude(
                filtered_ecg, R_peaks, fs, default_window_len=0.1
            )

            _df = build_feature_table(
                patient=pt,
                session=session,
                ecg_signal=filtered_ecg,
                fs=fs,
                QRS_amplitude_resampled=QRS_amplitude_resampled,
                interval_size=10,
                step_len = 5
            )
            
            _df['session_type'] = sess_type.__name__
            feature_list.append(_df)
            print(f"  processed {sess_type.__name__} for subject {pt.id} with DataFrame shape {_df.shape}")
        except Exception as e:
            print(f" failed for subject {pt.id} for session {sess_type.__name__}: {e}")

if feature_list:
    features_df = pd.concat(feature_list, ignore_index=True)
    print(" Combined feature dataframe:")
    print(features_df)
    features_df.to_excel("all_patient_features_combined_breath_segmentation.xlsx", index=False)
else:
    print(" no feature data was generated!")


""" from build_feature_table import build_feature_table
import pandas as pd
import diamonds.data as dt
from diamonds import load_patients

ptt = load_patients(show_progress=True)
# Choose a session type (e.g., dt.SeatedSession)
session = ptt[0][dt.SeatedSession]  # example for the first patient

# Build feature table for a given patient/session
feature_df = build_feature_table(ptt[0], session, ecg_signal, fs, QRS_amplitude_resampled, interval_size=10)
feature_df.to_excel("all_patient_features.xlsx", index=False) """


# before step:

In [ ]:
# features_3.py

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch
from scipy.interpolate import interp1d
import sys
import os

# Add project root to the Python path if not already included.
project_root = "C:/Users/Visnu/DIAMONDS"  
if project_root not in sys.path:
    sys.path.append(project_root)

import diamonds.data as dt

# Import the ECG peak detector.
from pan_tompkins import detect_ecg_peaks

import importlib
import breath_detection
importlib.reload(breath_detection)
from breath_detection import (
    auto_detect_edr_breaths,
    define_breaths_insp_exp,
    define_complete_breath_cycles,
    compute_true_insp_exp_durations,
    DEFAULT_MIN_BREATH_SEC
)

# These definitions (pt, session, ecg_signal, fs) are assumed to be provided by your diamonds_definitions.
from diamonds_definitions import pt, session, ecg_signal, fs

import importlib
import preprocessing
importlib.reload(preprocessing)
import pan_tompkins
import engzee_tompkins
import new_rpeak_detector
importlib.reload(pan_tompkins)
importlib.reload(engzee_tompkins)
importlib.reload(new_rpeak_detector)

# The variable 'record' here is set to pt.
record = pt

# Choose the method you want (e.g., "pan_tompkins", "engzee")
method_choice = "pan_tompkins"

# Call the unified function to extract ECG features.
results = preprocessing.extract_ecg_features(
    ecg_signal,
    fs,
    method=method_choice,
    RRI_fs=10,
    default_window_len=0.1,
    search_window_len=0.01,
    edr_method='R_peak'
)

# Unpack the results dictionary.
filtered_ecg = results["filtered_ecg"]
R_peaks       = results["R_peaks"]
RRI           = results["RRI"]
RRI_resampled = results["RRI_resampled"]
QRS_amplitude = results["QRS_amp"]
QRS_amplitude_resampled = results["QRS_amp_resampled"]
BW_effect     = results["BW_effect"]
AM_effect     = results["AM_effect"]
R_peak_amplitude             = results["R_peak_amplitude"]
R_peak_amplitude_resampled   = results["R_peak_amplitude_resampled"]
edr_time_r_peak              = results["edr_time_r_peak"]
edr_signal_r_peak            = results["edr_signal_r_peak"]
edr_rri_time                 = results["edr_rri_time"]
edr_rri_signal               = results["edr_rri_signal"]
combined_time                = results["combined_time"]
combined_signal              = results["combined_signal"]

print(f"Method used: {method_choice}")
print(f"Detected {len(R_peaks)} R-peaks")


###############################################################
# Feature functions (only breath_duration, true_insp_duration, and true_exp_duration)
###############################################################
def compute_true_insp_exp_durations(in_times, ex_times, t_start, t_end):
    """
    Compute the true inspiration duration (from expiration to next inspiration)
    and true expiration duration (from inspiration to following expiration)
    within the [t_start, t_end] window.
    """
    insp_durations = []
    exp_durations = []

    # True Inspiration: from an expiration to the next inspiration.
    for e_time in ex_times:
        if e_time < t_start or e_time > t_end:
            continue
        next_in = in_times[in_times > e_time]
        if next_in.size > 0 and next_in[0] <= t_end:
            insp_durations.append(next_in[0] - e_time)

    # True Expiration: from an inspiration to the next expiration.
    for i_time in in_times:
        if i_time < t_start or i_time > t_end:
            continue
        next_ex = ex_times[ex_times > i_time]
        if next_ex.size > 0 and next_ex[0] <= t_end:
            exp_durations.append(next_ex[0] - i_time)

    return (
        np.mean(insp_durations) if insp_durations else np.nan,
        np.mean(exp_durations) if exp_durations else np.nan
    )

def compute_bpm_from_inspiration_times(in_times, t_start, t_end):
    """
    Compute breathing rate (BPM) based on inspiration times within a given window.
    """
    valid = in_times[(in_times >= t_start) & (in_times <= t_end)]
    if valid.size < 2:
        return np.nan
    mean_interval = np.mean(np.diff(valid))
    return 60.0 / mean_interval

def compute_peak_to_trough(edr_time, edr_signal, t_start, t_end):
    """
    Compute the peak-to-trough difference of the EDR signal within a specified interval.
    """
    mask = (edr_time >= t_start) & (edr_time < t_end)
    segment = edr_signal[mask]
    if segment.size == 0:
        return np.nan
    return np.max(segment) - np.min(segment)

###############################################################
# Plotting functions
###############################################################
def plot_breath_cycles(edr_time, edr_signal, in_times, ex_times):
    """
    Plot the EDR signal with detected breath start (inspiration) and breath end (expiration) points.
    """
    from breath_detection import define_breaths_insp_exp
    breath_starts, breath_ends = define_breaths_insp_exp(in_times, ex_times)
    
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal')
    
    plt.plot(breath_starts,
             np.interp(breath_starts, edr_time, edr_signal),
             'go', label='Breath Start')
    plt.plot(breath_ends,
             np.interp(breath_ends, edr_time, edr_signal),
             'ro', label='Breath End')

    for t in breath_starts:
        plt.axvline(t, color='green', linestyle='--', alpha=0.5)
    for t in breath_ends:
        plt.axvline(t, color='red', linestyle='--', alpha=0.5)
    
    plt.title('Breath Cycles (Inspiration to Expiration)')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_edr_breaths(edr_time, edr_signal, in_times, ex_times):
    """
    Plot the EDR signal with detected inspiration and expiration points.
    """
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal (QRS Amplitude)')
    plt.plot(in_times,
             np.interp(in_times, edr_time, edr_signal),
             'o', label='Inspirations')
    plt.plot(ex_times,
             np.interp(ex_times, edr_time, edr_signal),
             'o', label='Expirations')
    plt.title('ECG-Derived Respiration with Detected Breaths')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()

###############################################################
# FEATURE TABLE CONSTRUCTION – one row per breath.
# Each row includes:
#   - breath_duration: Sum of true inspiration and true expiration durations (full cycle).
#   - true_insp_duration: From previous expiration to current inspiration.
#   - true_exp_duration: From current inspiration to current expiration.
###############################################################
def build_feature_table(patient,  ecg_signal, fs, QRS_amplitude_resampled):
    """
    Build a feature table (as a pandas DataFrame) in which each row corresponds
    to one complete breath.
    """
    # Extract EDR time and signal from the QRS_amplitude_resampled data.
    edr_time_qrs = QRS_amplitude_resampled[0]
    edr_signal_qrs = QRS_amplitude_resampled[2]

    # Detect breaths using the adaptive EDR detector.
    in_times, ex_times = auto_detect_edr_breaths(
        edr_time_qrs, edr_signal_qrs,
        init_min_breath_sec=2.0,
        adjust_factor=0.4,
        max_iterations=5,
        tol=0.05,
        min_allowed=0.5,
        max_allowed=10.0,
        adaptive=True
    )

    # ***Filter the detected breaths to only include those within the exercise interval.***
    # Use the exercise data from the provided session object.
    exercise_data = session_obj[dt.Exercise]
    exercise_start = np.min(exercise_data.timestamp)
    exercise_end = np.max(exercise_data.timestamp)
    in_times = in_times[(in_times >= exercise_start) & (in_times <= exercise_end)]
    ex_times = ex_times[(ex_times >= exercise_start) & (ex_times <= exercise_end)]

    # Define breaths using the inspiration-to-expiration pattern.
    breath_starts, breath_ends = define_breaths_insp_exp(in_times, ex_times)

    # (Optional) ECG R-peak detection – not used in the current feature computations.
    R_peaks = detect_ecg_peaks(ecg_signal, fs)
    R_times = R_peaks / fs  # Convert R-peak indices to time in seconds

    # Compute features per breath.
    feature_rows = []
    for i in range(len(breath_starts)):
        start = breath_starts[i]
        end = breath_ends[i]

        # Compute true inspiration duration: current inspiration minus previous expiration.
        if i > 0:
            true_insp = start - breath_ends[i - 1]
        else:
            true_insp = np.nan

        # Compute true expiration duration: current expiration minus current inspiration.
        true_exp = end - start if not np.isnan(end) else np.nan

        # Compute full breath duration (cycle): valid only when a previous expiration exists.
        if i > 0:
            breath_duration = true_insp + true_exp
        else:
            breath_duration = np.nan

        row = {
            "subject_id": patient.id,
            "breath_start": start,
            "breath_end": end,
            "breath_duration": breath_duration,
            "true_insp_duration": true_insp,
            "true_exp_duration": true_exp
        }
        feature_rows.append(row)
    feature_df = pd.DataFrame(feature_rows)
    return feature_df

###############################################################
# Main function for demonstration and plotting
###############################################################
def main():
    # Extract EDR data from QRS_amplitude_resampled.
    edr_time = QRS_amplitude_resampled[0]
    edr_signal = QRS_amplitude_resampled[2]

    # For demonstration, choose a session from pt (using dt.SeatedSession).
    session_selected = pt[dt.SeatedSession]  # You may also choose SupineSession if available.

    # Use the exercise data from the selected session to filter detected breaths.
    exercise_data = session_selected[dt.Exercise]
    exercise_start = np.min(exercise_data.timestamp)
    exercise_end = np.max(exercise_data.timestamp)

    in_times, ex_times = auto_detect_edr_breaths(
        edr_time, edr_signal,
        init_min_breath_sec=DEFAULT_MIN_BREATH_SEC,
        adjust_factor=0.6,
        max_iterations=5,
        tol=0.05,
        min_allowed=0.5,
        max_allowed=10.0,
        adaptive=True
    )
    in_times = in_times[(in_times >= exercise_start) & (in_times <= exercise_end)]
    ex_times = ex_times[(ex_times >= exercise_start) & (ex_times <= exercise_end)]
    
    # Plot the detected breath cycles.
    plot_breath_cycles(edr_time, edr_signal, in_times, ex_times)
    plot_edr_breaths(edr_time, edr_signal, in_times, ex_times)

    # Build the feature table where each row represents one complete breath.
    feature_df = build_feature_table(pt, ecg_signal, fs, QRS_amplitude_resampled)
    print("Feature Table:")
    print(feature_df)

    # Plot features per breath using the breath index as the x-axis.
    breath_indices = np.arange(len(feature_df))
    
    plt.figure()
    plt.plot(breath_indices, feature_df['breath_duration'], 'o-', label='Breath Duration')
    plt.xlabel('Breath Index')
    plt.ylabel('Breath Duration (s)')
    plt.title('Breath Duration per Breath')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure()
    plt.plot(breath_indices, feature_df['true_insp_duration'], 'o-', label='True Inspiration Duration')
    plt.xlabel('Breath Index')
    plt.ylabel('True Inspiration Duration (s)')
    plt.title('True Inspiration Duration per Breath')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure()
    plt.plot(breath_indices, feature_df['true_exp_duration'], 'o-', label='True Expiration Duration')
    plt.xlabel('Breath Index')
    plt.ylabel('True Expiration Duration (s)')
    plt.title('True Expiration Duration per Breath')
    plt.grid(True)
    plt.legend()
    plt.show()

if __name__ == "__main__":
    main()


In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set up project path.
project_root = "C:/Users/Visnu/DIAMONDS"
if project_root not in sys.path:
    sys.path.append(project_root)

from diamonds import data as dt
from diamonds_definitions import pt

class GoldenBreathAnalyzer:
    def __init__(self, patient, session_type=dt.SeatedSession, interval_size=15, step_len=5):
        self.record = patient
        self.session = self.record[session_type]
        self.breaths = self.session[dt.Breaths]
        self.exercise = self.session[dt.Exercise]
        self.interval_size = interval_size
        self.step_len = step_len

        self.in_times, self.ex_times = self._get_golden_breath_events()
        self.start_time = np.min(self.exercise.timestamp)
        self.end_time = np.max(self.exercise.timestamp)

        print(f"Exercise interval: {self.start_time:.1f}s to {self.end_time:.1f}s")
        
    def _get_golden_breath_events(self):
        # Extract inspiration and expiration times based on the golden annotation.
        in_times = np.array([t for t, l in zip(self.breaths.timestamp, self.breaths.ann) if l == '(In'])
        ex_times = np.array([t for t, l in zip(self.breaths.timestamp, self.breaths.ann) if l == '(Ex'])
        return in_times, ex_times

    def _annotate_exercises(self, ax):
        # Draw vertical lines and labels for each exercise event.
        for tstamp, label in zip(self.exercise.timestamp, self.exercise.ann):
            ax.axvline(tstamp, color='magenta', linestyle='--', alpha=0.5)
            ax.text(tstamp, ax.get_ylim()[1] * 0.95, label,
                    rotation=90, verticalalignment='top', color='magenta')

    def _compute_bpm(self, t_start, t_end):
        # Compute BPM based solely on inspirations within the window.
        valid_in = self.in_times[(self.in_times >= t_start) & (self.in_times <= t_end)]
        if len(valid_in) < 2:
            return np.nan
        return 60.0 / np.mean(np.diff(valid_in))

    def segment_bpm(self):
        """
        Return arrays of (segment_starts, BPM_values) using overlapping windows.
        """
        seg_starts = np.arange(
            self.start_time,
            self.end_time - self.interval_size,
            self.step_len
        )
        bpm_vals = []
        for t_start in seg_starts:
            t_end = t_start + self.interval_size
            bpm_vals.append(self._compute_bpm(t_start, t_end))
        return seg_starts, np.array(bpm_vals)
    
    def segment_true_insp_exp_durations(self):
        """
        For each overlapping window, compute the mean inspiration and expiration durations
        (Ex → In = inspiration, In → Ex = expiration).
        Returns (segment_starts, array_of_insp_durs, array_of_exp_durs).
        """
        seg_starts = np.arange(
            self.start_time,
            self.end_time - self.interval_size,
            self.step_len
        )
        true_insp_durations = []
        true_exp_durations = []

        for t_start in seg_starts:
            t_end = t_start + self.interval_size

            # Collect all (Ex → In) durations in [t_start, t_end]
            insp_durations = []
            for e_time in self.ex_times:
                if e_time < t_start or e_time > t_end:
                    continue
                next_in = self.in_times[self.in_times > e_time]
                if next_in.size > 0 and next_in[0] <= t_end:
                    insp_durations.append(next_in[0] - e_time)

            # Collect all (In → Ex) durations in [t_start, t_end]
            exp_durations = []
            for i_time in self.in_times:
                if i_time < t_start or i_time > t_end:
                    continue
                next_ex = self.ex_times[self.ex_times > i_time]
                if next_ex.size > 0 and next_ex[0] <= t_end:
                    exp_durations.append(next_ex[0] - i_time)

            mean_insp = np.mean(insp_durations) if insp_durations else np.nan
            mean_exp = np.mean(exp_durations) if exp_durations else np.nan

            true_insp_durations.append(mean_insp)
            true_exp_durations.append(mean_exp)

        return seg_starts, np.array(true_insp_durations), np.array(true_exp_durations)

    def segment_mean_intervals(self):
        seg_times = np.arange(self.start_time, self.end_time, self.interval_size)
        mean_insp_durations = []
        mean_exp_durations = []
        ie_ratios = []

        for t_start in seg_times:
            t_end = t_start + self.interval_size

            # --- Ex → In (Inspiration) ---
            insp_durations = []
            for e_time in self.ex_times:
                if e_time < t_start or e_time > t_end:
                    continue
                next_in = self.in_times[self.in_times > e_time]
                if next_in.size > 0 and next_in[0] <= t_end:
                    insp_durations.append(next_in[0] - e_time)

            # --- In → Ex (Expiration) ---
            exp_durations = []
            for i_time in self.in_times:
                if i_time < t_start or i_time > t_end:
                    continue
                next_ex = self.ex_times[self.ex_times > i_time]
                if next_ex.size > 0 and next_ex[0] <= t_end:
                    exp_durations.append(next_ex[0] - i_time)

            mi = np.mean(insp_durations) if insp_durations else np.nan
            me = np.mean(exp_durations) if exp_durations else np.nan
            ratio = mi / me if not np.isnan(mi) and not np.isnan(me) and me > 0 else np.nan

            mean_insp_durations.append(mi)
            mean_exp_durations.append(me)
            ie_ratios.append(ratio)

        return seg_times, np.array(mean_insp_durations), np.array(mean_exp_durations), np.array(ie_ratios)

    def plot_bpm(self, seg_times, bpm_vals):
        plt.figure(figsize=(8, 4))
        ax = plt.gca()
        ax.plot(seg_times, bpm_vals, '-o', label='Golden BPM (Inspiration)')
        ax.set_title("Golden Standard: BPM in Exercise Intervals")
        ax.set_xlabel("Segment Start Time (s)")
        ax.set_ylabel("Breaths Per Minute")
        ax.grid(True)
        ax.legend()
        self._annotate_exercises(ax)
        plt.show()

    def plot_true_insp_exp_durations(self, seg_times, true_insp, true_exp):
        plt.figure(figsize=(10, 4))
        plt.plot(seg_times, true_insp, '-o', label="True Inspiration (Ex→In)")
        plt.plot(seg_times, true_exp, '-o', label="True Expiration (In→Ex)")
        plt.title("True Inspiration & Expiration Durations (Gold Standard)")
        plt.xlabel("Segment Start Time (s)")
        plt.ylabel("Duration (s)")
        plt.grid(True)
        plt.legend()
        self._annotate_exercises(plt.gca())
        plt.show()

    def plot_ie_ratio(self, seg_times, ie_ratios):
        plt.figure(figsize=(8, 4))
        plt.plot(seg_times, ie_ratios, '-o', color='orange', label="I:E Ratio")
        plt.title("Inspiration to Expiration Ratio (I:E)")
        plt.xlabel("Segment Start Time (s)")
        plt.ylabel("Ratio")
        plt.grid(True)
        plt.legend()
        self._annotate_exercises(plt.gca())
        plt.show()
    
    def define_complete_breath_cycles(self):
        """
        Extract full breath cycles from the entire recording using the gold standard (Ex → In → Ex).
        Returns arrays of breath_starts, breath_ends, inspiration_durations, and expiration_durations.
        """
        breath_starts = []
        breath_ends = []
        insp_durations = []
        exp_durations = []

        for i in range(len(self.ex_times) - 1):
            start_ex = self.ex_times[i]
            next_in = self.in_times[self.in_times > start_ex]
            if next_in.size == 0:
                continue
            in_time = next_in[0]

            next_ex = self.ex_times[self.ex_times > in_time]
            if next_ex.size == 0:
                continue
            end_ex = next_ex[0]

            breath_starts.append(start_ex)
            breath_ends.append(end_ex)
            insp_durations.append(in_time - start_ex)
            exp_durations.append(end_ex - in_time)

        return (
            np.array(breath_starts),
            np.array(breath_ends),
            np.array(insp_durations),
            np.array(exp_durations)
        )

    def print_total_breath_count(self):
        total = len(self.breaths.ann)
        total_in = len(self.in_times)
        total_ex = len(self.ex_times)
        est_cycles = min(total_in, total_ex)
        print(f"Total breath events: {total}")
        print(f"Inspirations: {total_in}, Expirations: {total_ex}")
        print(f"Estimated breath cycles (In↔Ex pairs): {est_cycles}")
    
    def print_breath_counts_per_interval(self):
        seg_times = np.arange(self.start_time, self.end_time, self.interval_size)
        print(f"\nBreath event counts per {self.interval_size}-second interval:")
        for t0 in seg_times:
            t1 = t0 + self.interval_size
            mask = (np.array(self.breaths.timestamp) >= t0) & (np.array(self.breaths.timestamp) < t1)
            print(f"[{t0:.0f}-{t1:.0f}s]: {np.sum(mask)} events")

    def run_analysis(self):
        print("\n=== Running Golden Breath Analysis ===")

        # 1. BPM over time using overlapping windows.
        bpm_times, bpm_vals = self.segment_bpm()
        self.plot_bpm(bpm_times, bpm_vals)

        # 2. True Inspiration and Expiration Durations (overlapping windows).
        seg_times, true_insp, true_exp = self.segment_true_insp_exp_durations()
        self.plot_true_insp_exp_durations(seg_times, true_insp, true_exp)

        # 3. I:E Ratio Plot (from true durations).
        ie_ratios = true_insp / true_exp
        self.plot_ie_ratio(seg_times, ie_ratios)

        # 4. Print per-segment summary.
        print("\n=== Segment-wise Duration Summary ===")
        for t0, insp, exp, ratio in zip(seg_times, true_insp, true_exp, ie_ratios):
            t1 = t0 + self.interval_size
            print(f"[{t0:.0f}-{t1:.0f}s] Insp={insp:.2f}s, Exp={exp:.2f}s, I:E={ratio:.2f}")

        # 5. BPM Summary.
        print("\n=== BPM Summary ===")
        for t0, bpm in zip(bpm_times, bpm_vals):
            t1 = t0 + self.interval_size
            print(f"[{t0:.0f}-{t1:.0f}s]: BPM={bpm:.2f}")

        # 6. Total Gold Annotations.
        self.print_total_breath_count()
        self.print_breath_counts_per_interval()

        # 7. Total valid breath cycles using Ex→In→Ex.
        starts, ends, insp_durs, exp_durs = self.define_complete_breath_cycles()
        print(f"\n✅ Total valid breath cycles (Ex→In→Ex): {len(starts)}")

    def build_golden_table(self):
        breath_starts, breath_ends, insp_durs, exp_durs = self.define_complete_breath_cycles()
        rows = []
        for i in range(len(breath_starts)):
            start = breath_starts[i]
            end   = breath_ends[i]

            # If you want the i-th breath's actual inspiration duration from define_complete_breath_cycles:
            true_insp = insp_durs[i]
            true_exp  = exp_durs[i]
            breath_duration = true_insp + true_exp

            row = {
                "subject_id": self.record.id,
                "golden_breath_start": start,
                "golden_breath_end":   end,
                "true_insp_duration":   true_insp,
                "true_exp_duration":    true_exp,
                "golden_breath_duration": breath_duration
            }
            rows.append(row)
        return pd.DataFrame(rows)



In [ ]:
import golden_analysis
import importlib
importlib.reload(golden_analysis)
from golden_analysis import GoldenBreathAnalyzer
import diamonds.data as dt
from diamonds import load_patients
import pandas as pd

ptt = load_patients(show_progress=True)
session_types = [dt.SeatedSession, dt.SupineSession]

golden_list = []

for pt in ptt:
    for sess_type in session_types:
        try:
            _ = pt[sess_type]  # Verify that this session exists.
        except KeyError:
            continue

        try:
            # Create an instance of the GoldenBreathAnalyzer for the patient/session.
            analyzer = GoldenBreathAnalyzer(pt, session_type=sess_type,
                                            interval_size=15, step_len=5)
            # Use the per–breath table function.
            golden_df = analyzer.build_golden_table()
            # Optionally override/add metadata.
            golden_df['session_type'] = sess_type.__name__
            golden_df['subject_id']   = pt.id
            golden_list.append(golden_df)
        except Exception as e:
            print(f"Failed for {pt.id}, {sess_type.__name__}: {e}")

if golden_list:
    combined_golden_df = pd.concat(golden_list, ignore_index=True)
    combined_golden_df.to_excel("all_patient_golden_metrics_combined.xlsx", index=False)
    print("Saved gold data to 'all_patient_golden_metrics_combined.xlsx'")
else:
    print("No golden metrics data was generated!")


In [ ]:
import importlib
import features_3
importlib.reload(features_3)
from features_3 import build_feature_table  # This is now the per-breath version
import pandas as pd
import diamonds.data as dt
from diamonds import load_patients
import preprocessing
importlib.reload(preprocessing)
import engzee_tompkins
importlib.reload(engzee_tompkins)
import new_rpeak_detector

# Load patients
ptt = load_patients(show_progress=True)

# List the session types you want to process (e.g., Seated and Supine)
session_types = [dt.SeatedSession, dt.SupineSession]

feature_list = []

for pt in ptt:
    print(f"processing subject: {pt.id}")
    for sess_type in session_types:
        try:
            session = pt[sess_type]
        except KeyError:
            print(f"  subject {pt.id} does not have session type {sess_type.__name__}")
            continue

        try:
            # Extract ECG (and EMG) data from the patient/session record.
            EMG, ECG = pt[(sess_type, dt.EMG)].decompose()
            ecg_signal = -ECG.samples[:, 1]
            fs = ECG.fs

            # Preprocess the ECG signal and extract R-peaks
            filtered_ecg = preprocessing.preprocess_signal(ecg_signal, fs)
            R_peaks, RRI, RRI_resampled = preprocessing.get_RRI_from_signal(filtered_ecg, fs)
            QRS_amplitude, QRS_amplitude_resampled = preprocessing.get_QRS_amplitude(
                filtered_ecg, R_peaks, fs, default_window_len=0.1
            )

            # Build the feature table (one row per breath)
            _df = build_feature_table(
                patient=pt,
                session_type=sess_type,  # Pass the session type (e.g., dt.SeatedSession), not the session object.
                ecg_signal=filtered_ecg,
                fs=fs,
                QRS_amplitude_resampled=QRS_amplitude_resampled
            )


            # Tag the session type into the DataFrame
            _df['session_type'] = sess_type.__name__
            feature_list.append(_df)
            print(f"  processed {sess_type.__name__} for subject {pt.id} with DataFrame shape {_df.shape}")
        except Exception as e:
            print(f" failed for subject {pt.id} for session {sess_type.__name__}: {e}")

if feature_list:
    features_df = pd.concat(feature_list, ignore_index=True)
    print("Combined feature dataframe:")
    print(features_df)
    features_df.to_excel("all_patient_features_combined_breath_segmentation.xlsx", index=False)
else:
    print("No feature data was generated!")


# preprocessing before 18-04

In [ ]:
# preprocessing.py

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, iirnotch, find_peaks
from scipy.interpolate import interp1d
import importlib
import pan_tompkins
importlib.reload(pan_tompkins)
from pan_tompkins import detect_ecg_peaks as pan_detect
from engzee_tompkins import detect_ecg_peaks_engzee
from new_rpeak_detector import detect_r_peaks as neurokit_detect

def preprocess_signal(signal, fs): 
    b, a = butter(N=4, Wn=1, btype='high', fs=fs)
    filt_signal = filtfilt(b, a, signal)

    notch_cut_off = 50
    for co in np.arange(notch_cut_off, fs / 2, notch_cut_off):
        b, a = iirnotch(co / (fs / 2), 10)
        filt_signal = filtfilt(b, a, filt_signal)
    return filt_signal

def interpolate_from_R_peaks(loc, val, fs,
                             resampled_fs=10,
                             interp_method="linear",
                             filter_type="band"):
    import numpy as np
    from scipy.signal import butter, filtfilt
    from scipy.interpolate import interp1d

    # Convert lists to NumPy arrays
    loc = np.array(loc, dtype=float)
    val = np.array(val, dtype=float)

    if len(loc) < 2:
        # Not enough points to interpolate
        return ([], [], [])

    # Now we can do element-wise division
    sig_time = loc / fs

    # Next steps remain the same:
    f_interp = interp1d(sig_time, val, kind=interp_method, fill_value="extrapolate")
    new_time = np.arange(sig_time[0], sig_time[-1], 1.0 / resampled_fs)

    if len(new_time) < 2:
        return ([], [], [])

    resampled_sig = f_interp(new_time)

    if filter_type == "band":
        b, a = butter(N=2, Wn=[0.1, 0.5], btype='band', fs=resampled_fs)
    elif filter_type == "low":
        b, a = butter(N=2, Wn=0.4, btype='low', fs=resampled_fs)
    else:
        raise ValueError("Invalid filter_type. Choose 'band' or 'low'.")

    filtered_sig = filtfilt(b, a, resampled_sig)
    return (new_time, resampled_sig, filtered_sig)

def get_QRS_amplitude(ecg, R_peaks, fs, default_window_len=0.1):
    if len(R_peaks) < 2:
        return np.array([]), (np.array([]), None, np.array([]))
    QRS_amplitude = np.zeros(len(R_peaks))
    RR_samples = np.diff(R_peaks)
    for i in range(len(R_peaks)):
        if i == 0 or i == len(R_peaks) - 1:
            half_win = int(default_window_len * fs)
        else:
            # Adaptive window based on local RR
            local_RR = 0.5 * (RR_samples[i-1] + RR_samples[i])
            adaptive_len_sec = min(default_window_len, 0.15 * (local_RR / fs))
            half_win = int(adaptive_len_sec * fs)

        center = R_peaks[i]
        start = max(center - half_win, 0)
        end   = min(center + half_win, len(ecg))
        window = ecg[start:end]

        baseline_win = int(0.02 * fs) 
        baseline_start = max(center - baseline_win, start)
        baseline_segment = ecg[baseline_start:center]
        if len(baseline_segment) < 1:
            baseline_value = 0.0
        else:
            baseline_value = np.mean(baseline_segment)

        window_corrected = window - baseline_value
        QRS_amplitude[i] = window_corrected.max() - window_corrected.min()

    QRS_amplitude_resampled = interpolate_from_R_peaks(
        R_peaks, QRS_amplitude, fs,
        resampled_fs=10,
        interp_method="linear",
        filter_type="band"
    )
    return QRS_amplitude, QRS_amplitude_resampled


def get_combined_signal_from_features(
    features,
    resampled_fs=10,
    freq_band=(0.1, 0.5),
    weighting="equal",
    custom_weights=None
):
    """
    Create an ECG-derived respiration (EDR) signal by combining QRS amplitude and RRI signals,
    using the outputs from extract_ecg_features.
    
    Parameters
    ----------
    features : dict
        Dictionary returned by extract_ecg_features.
    resampled_fs : float, optional
        The target sampling frequency for resampled features (default=10).
    freq_band : tuple, optional
        The bandpass frequency range (in Hz) for final filtering (default=(0.1, 0.5)).
    weighting : str, optional
        Method to combine the normalized signals. Options:
          - "equal":  simple 50/50 average
          - "custom": use user-provided custom_weights (e.g., (0.7, 0.3))
          - "auto":   (example: inverse variance weighting)
    custom_weights : tuple or list, optional
        If weighting == "custom", pass a tuple like (w_qrs, w_rri) that sums to 1.
        
    Returns
    -------
    t_common : np.array
        The common time axis for the EDR signal.
    edr_filtered : np.array
        The combined and filtered EDR signal.
    """
    from scipy.signal import butter, filtfilt
    import numpy as np

    # Extract the resampled QRS amplitude and RRI signals from the features dict.
    # They are expected to be tuples: (time_array, raw_signal, filtered_signal)
    t_qrs, qrs_resampled, _ = features["QRS_amp_resampled"]
    t_rri, rri_resampled, _ = features["RRI_resampled"]
    
    # Check if we have enough data:
    if len(t_qrs) < 2 or len(t_rri) < 2:
        return np.array([]), np.array([])
    
    # Determine the overlapping time range:
    t_start = max(t_qrs[0], t_rri[0])
    t_end = min(t_qrs[-1], t_rri[-1])
    if t_start >= t_end:
        return np.array([]), np.array([])
    
    # Define a common time axis:
    t_common = np.arange(t_start, t_end, 1.0 / resampled_fs)
    
    # Interpolate both signals onto the common axis:
    qrs_common = np.interp(t_common, t_qrs, qrs_resampled)
    rri_common = np.interp(t_common, t_rri, rri_resampled)
    
    # Normalize each signal:
    norm_qrs = (qrs_common - np.mean(qrs_common)) / np.std(qrs_common)
    norm_rri = (rri_common - np.mean(rri_common)) / np.std(rri_common)
    
    # Combine the normalized signals:
    if weighting == "equal":
        edr_raw = 0.5 * (norm_qrs + norm_rri)
    elif weighting == "custom":
        if not custom_weights or len(custom_weights) != 2:
            raise ValueError("custom_weights must be a tuple of length 2, e.g. (0.7, 0.3).")
        w_qrs, w_rri = custom_weights
        edr_raw = w_qrs * norm_qrs + w_rri * norm_rri
    elif weighting == "auto":
        # Example: use inverse variance weighting (you can improve this)
        var_qrs = np.var(norm_qrs)
        var_rri = np.var(norm_rri)
        total = var_qrs + var_rri
        w_qrs = var_rri / total
        w_rri = var_qrs / total
        edr_raw = w_qrs * norm_qrs + w_rri * norm_rri
    else:
        raise ValueError("Invalid weighting mode. Use 'equal', 'custom', or 'auto'.")
    
    # Apply a bandpass filter if freq_band is provided:
    if freq_band is not None and len(freq_band) == 2:
        low, high = freq_band
        nyquist = 0.5 * resampled_fs
        low_cut = low / nyquist
        high_cut = high / nyquist
        b, a = butter(N=2, Wn=[low_cut, high_cut], btype='band')
        edr_filtered = filtfilt(b, a, edr_raw)
    else:
        # Fallback to a low-pass filter
        cutoff = 0.6  # Hz
        nyquist = 0.5 * resampled_fs
        b, a = butter(N=2, Wn=cutoff/nyquist, btype='low')
        edr_filtered = filtfilt(b, a, edr_raw)
    
    return t_common, edr_filtered

def get_combined_signal(
    resampled_fs=10,
    freq_band=(0.1, 0.5),
    weighting="equal",
    custom_weights=None
):
    """
    Create an ECG-derived respiration (EDR) signal by combining QRS amplitude and RRI signals,
    using the outputs from extract_ecg_features.
    
    Parameters
    ----------
    features : dict
        Dictionary returned by extract_ecg_features.
    resampled_fs : float, optional
        The target sampling frequency for resampled features (default=10).
    freq_band : tuple, optional
        The bandpass frequency range (in Hz) for final filtering (default=(0.1, 0.5)).
    weighting : str, optional
        Method to combine the normalized signals. Options:
          - "equal":  simple 50/50 average
          - "custom": use user-provided custom_weights (e.g., (0.7, 0.3))
          - "auto":   (example: inverse variance weighting)
    custom_weights : tuple or list, optional
        If weighting == "custom", pass a tuple like (w_qrs, w_rri) that sums to 1.
        
    Returns
    -------
    t_common : np.array
        The common time axis for the EDR signal.
    edr_filtered : np.array
        The combined and filtered EDR signal.
    """
    from scipy.signal import butter, filtfilt
    import numpy as np

    # Extract the resampled QRS amplitude and RRI signals from the features dict.
    # They are expected to be tuples: (time_array, raw_signal, filtered_signal)
    t_qrs, qrs_resampled, _ = QRS_amplitude_resampled
    t_rri, rri_resampled, _ = RRI_resampled
    
    # Check if we have enough data:
    if len(t_qrs) < 2 or len(t_rri) < 2:
        return np.array([]), np.array([])
    
    
    t_start = max(t_qrs[0], t_rri[0])
    t_end = min(t_qrs[-1], t_rri[-1])
    if t_start >= t_end:
        return np.array([]), np.array([])
    
    # Define a common time axis:
    t_common = np.arange(t_start, t_end, 1.0 / resampled_fs)
    
    
    qrs_common = np.interp(t_common, t_qrs, qrs_resampled)
    rri_common = np.interp(t_common, t_rri, rri_resampled)
    
   
    norm_qrs = (qrs_common - np.mean(qrs_common)) / np.std(qrs_common)
    norm_rri = (rri_common - np.mean(rri_common)) / np.std(rri_common)
    
    
    if weighting == "equal":
        edr_raw = 0.5 * (norm_qrs + norm_rri)
    elif weighting == "custom":
        if not custom_weights or len(custom_weights) != 2:
            raise ValueError("custom_weights must be a tuple of length 2, e.g. (0.7, 0.3).")
        w_qrs, w_rri = custom_weights
        edr_raw = w_qrs * norm_qrs + w_rri * norm_rri
    elif weighting == "auto":
        
        var_qrs = np.var(norm_qrs)
        var_rri = np.var(norm_rri)
        total = var_qrs + var_rri
        w_qrs = var_rri / total
        w_rri = var_qrs / total
        edr_raw = w_qrs * norm_qrs + w_rri * norm_rri
    else:
        raise ValueError("Invalid weighting mode. Use 'equal', 'custom', or 'auto'.")
    
    
    if freq_band is not None and len(freq_band) == 2:
        low, high = freq_band
        nyquist = 0.5 * resampled_fs
        low_cut = low / nyquist
        high_cut = high / nyquist
        b, a = butter(N=2, Wn=[low_cut, high_cut], btype='band')
        edr_filtered = filtfilt(b, a, edr_raw)
    else:
        # Fallback to a low-pass filter
        cutoff = 0.6  # Hz
        nyquist = 0.5 * resampled_fs
        b, a = butter(N=2, Wn=cutoff/nyquist, btype='low')
        edr_filtered = filtfilt(b, a, edr_raw)
    
    return t_common, edr_filtered





def get_QRS_effects(ecg, R_peaks, fs, search_window_len=0.06):
    BW_effect = np.zeros(len(R_peaks))
    AM_effect = np.zeros(len(R_peaks))
    half_win = int(search_window_len * fs)
    for i in range(len(R_peaks)):
        start = R_peaks[i] - half_win
        end = R_peaks[i] + half_win
        if start < 0 or end >= len(ecg):
            continue
        window = ecg[start:end]
        BW_effect[i] = np.mean([window.max(), window.min()])
        AM_effect[i] = window.max() - window.min()
    return BW_effect, AM_effect

def get_R_peak_amplitude(ecg, R_peaks):
    return ecg[R_peaks] if len(R_peaks) > 0 else np.array([])

def get_EDR_from_ecg(ecg_signal, R_peaks, fs, method='R_peak'):
    """
    If method=='R_peak', EDR is just the amplitude at the R-peaks.
    Otherwise, measure min/max in a small window around R-peaks.
    """
    import numpy as np
    from scipy.signal import butter, filtfilt

    if len(R_peaks) == 0:
        return np.array([]), np.array([])

    if method == 'R_peak':
        edr_raw = ecg_signal[R_peaks]  
    else:
        search_window_len = 0.01
        edr_raw = np.zeros(len(R_peaks))
        for i, peak in enumerate(R_peaks):
            start_idx = peak - int(search_window_len * fs)
            end_idx   = peak + int(search_window_len * fs)
            if start_idx < 0 or end_idx >= len(ecg_signal):
                continue
            window = ecg_signal[start_idx:end_idx]
            edr_raw[i] = window.max() - window.min()

    rpeak_times = R_peaks / fs
    if len(rpeak_times) < 2:
        return (np.array([]), np.array([]))

    from scipy.interpolate import interp1d
    interp_func = interp1d(rpeak_times, edr_raw, kind='linear', fill_value='extrapolate')
    resampled_fs = 10
    new_time = np.arange(rpeak_times[0], rpeak_times[-1], 1/resampled_fs)
    if len(new_time) < 2:
        return (np.array([]), np.array([]))

    edr_interpolated = interp_func(new_time)
    b, a = butter(2, 0.5, btype='low', fs=resampled_fs)
    edr_smooth = filtfilt(b, a, edr_interpolated)
    return new_time, edr_smooth

def get_EDR_from_RRI(R_peaks, fs, resampled_fs=10, lp_cutoff=1.0):
    RR_intervals = np.diff(R_peaks) / fs
    if len(RR_intervals) < 2:
        return np.array([]), np.array([])
    rr_times = R_peaks[1:] / fs
    from scipy.interpolate import interp1d
    rr_interp = interp1d(rr_times, RR_intervals, kind='linear', fill_value="extrapolate")

    t_min, t_max = rr_times[0], rr_times[-1]
    new_time = np.arange(t_min, t_max, 1/resampled_fs)
    if len(new_time) < 2:
        return (np.array([]), np.array([]))

    edr_raw = rr_interp(new_time)
    from scipy.signal import butter, filtfilt
    b, a = butter(2, lp_cutoff/(resampled_fs/2), btype='low')
    edr_smooth = filtfilt(b, a, edr_raw)
    return new_time, edr_smooth

def get_RRI_from_signal(signal, fs, RRI_fs=10):
  
    R_peaks = pan_detect(signal, fs)
    RRI = np.diff(R_peaks) / fs  
    

    RRI_resampled = interpolate_from_R_peaks(R_peaks[:-1], RRI, fs, RRI_fs, interp_method="linear", filter_type="low")

    return R_peaks, RRI, RRI_resampled








def detect_r_peaks_with_method(ecg_signal, fs, method="pan_tompkins"):
   
    if method == "pan_tompkins":
        from pan_tompkins import detect_ecg_peaks
        return detect_ecg_peaks(ecg_signal, fs)
    elif method == "engzee":
        from engzee_tompkins import detect_ecg_peaks_engzee
        return detect_ecg_peaks_engzee(ecg_signal, fs)
    elif method == "neurokit":
        from new_rpeak_detector import detect_r_peaks
        return detect_r_peaks(ecg_signal, fs)
    else:
        raise ValueError("method must be 'pan_tompkins', 'engzee', or 'neurokit'.")


def extract_ecg_features(
    ecg_signal,
    fs,
    method="pan_tompkins",
    RRI_fs=10,
    default_window_len=0.1,
    search_window_len=0.01,
    edr_method='R_peak'
):
    
    filtered_ecg = preprocess_signal(ecg_signal, fs)

    
    R_peaks = detect_r_peaks_with_method(filtered_ecg, fs, method=method)

    
    RRI = np.diff(R_peaks) / fs
    RRI_resampled = interpolate_from_R_peaks(
        R_peaks[:-1], RRI, fs,
        resampled_fs=RRI_fs,
        interp_method="linear",
        filter_type="low"
    )

    
    QRS_amp, QRS_amp_resampled = get_QRS_amplitude(
        ecg=filtered_ecg,
        R_peaks=R_peaks,
        fs=fs,
        default_window_len=default_window_len
    )

    
    BW_effect, AM_effect = get_QRS_effects(filtered_ecg, R_peaks, fs, search_window_len=search_window_len)

    
    R_peak_amp = get_R_peak_amplitude(filtered_ecg, R_peaks)
    R_peak_amp_resampled = interpolate_from_R_peaks(
        R_peaks, R_peak_amp, fs,
        resampled_fs=RRI_fs,
        interp_method="linear",
        filter_type="low"
    )

   
    edr_time_r_peak, edr_signal_r_peak = get_EDR_from_ecg(
        ecg_signal=filtered_ecg,
        R_peaks=R_peaks,
        fs=fs,
        method=edr_method
    )

    
    edr_rri_time, edr_rri_signal = get_EDR_from_RRI(R_peaks, fs, resampled_fs=RRI_fs, lp_cutoff=0.7)

    combined_time, combined_signal = get_combined_signal_from_features(
        features={
            "QRS_amp_resampled": QRS_amp_resampled,
            "RRI_resampled": RRI_resampled
        },
        resampled_fs=RRI_fs,           # you can adjust this if needed
        freq_band=(0.1, 0.5),           # adjust the frequency band as desired
        weighting="equal"             # or "custom" / "auto" per your requirements
    )

    return {
        "filtered_ecg": filtered_ecg,
        "R_peaks": R_peaks,
        "RRI": RRI,
        "RRI_resampled": RRI_resampled,
        "QRS_amp": QRS_amp,
        "QRS_amp_resampled": QRS_amp_resampled,
        "BW_effect": BW_effect,
        "AM_effect": AM_effect,
        "R_peak_amplitude": R_peak_amp,
        "R_peak_amplitude_resampled": R_peak_amp_resampled,
        "edr_time_r_peak": edr_time_r_peak,
        "edr_signal_r_peak": edr_signal_r_peak,
        "edr_rri_time": edr_rri_time,
        "edr_rri_signal": edr_rri_signal,
        # New keys for the combined signal:
        "combined_time": combined_time,
        "combined_signal": combined_signal
    }





# preprocessing new code 18-04:

In [ ]:
# preprocessing.py

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, iirnotch, find_peaks
from scipy.interpolate import interp1d
import importlib
import pan_tompkins
importlib.reload(pan_tompkins)
from pan_tompkins import detect_ecg_peaks as pan_detect
from engzee_tompkins import detect_ecg_peaks_engzee
from new_rpeak_detector import detect_r_peaks as neurokit_detect

def preprocess_signal(signal, fs): 
    b, a = butter(N=4, Wn=1, btype='high', fs=fs)
    filt_signal = filtfilt(b, a, signal)

    notch_cut_off = 50
    for co in np.arange(notch_cut_off, fs / 2, notch_cut_off):
        b, a = iirnotch(co / (fs / 2), 10)
        filt_signal = filtfilt(b, a, filt_signal)
    return filt_signal

def interpolate_from_R_peaks(loc, val, fs,
                             resampled_fs=10,
                             interp_method="linear",
                             filter_type="band"):
    import numpy as np
    from scipy.signal import butter, filtfilt
    from scipy.interpolate import interp1d

    # Convert lists to NumPy arrays
    loc = np.array(loc, dtype=float)
    val = np.array(val, dtype=float)

    if len(loc) < 2:
        # Not enough points to interpolate
        return ([], [], [])

    # Now we can do element-wise division
    sig_time = loc / fs

    # Next steps remain the same:
    f_interp = interp1d(sig_time, val, kind=interp_method, fill_value="extrapolate")
    new_time = np.arange(sig_time[0], sig_time[-1], 1.0 / resampled_fs)

    if len(new_time) < 2:
        return ([], [], [])

    resampled_sig = f_interp(new_time)

    if filter_type == "band":
        b, a = butter(N=2, Wn=[0.1, 0.5], btype='band', fs=resampled_fs)
    elif filter_type == "low":
        b, a = butter(N=2, Wn=0.4, btype='low', fs=resampled_fs)
    else:
        raise ValueError("Invalid filter_type. Choose 'band' or 'low'.")

    filtered_sig = filtfilt(b, a, resampled_sig)
    return (new_time, resampled_sig, filtered_sig)

from scipy.signal import butter, filtfilt, hilbert

def bandpass_qrs(x, fs, low=5, high=25, order=2):
    nyq = 0.5*fs
    b, a = butter(order, [low/nyq, high/nyq], btype='band')
    return filtfilt(b, a, x)

def get_QRS_amplitude(ecg, R_peaks, fs, default_window_len=0.1,
                      min_win=0.04, max_win=0.2):
    """
    - bandpass between 5–25 Hz to isolate the QRS
    - take analytic envelope via Hilbert
    - adapt half-win on RR but clip to [min_win, max_win]
    """
    if len(R_peaks)<2:
        return np.array([]), (np.array([]), None, np.array([]))

    # pre‐filter the entire ECG so each window is clean
    ecg_qrs = bandpass_qrs(ecg, fs)

    QRS_amp = np.zeros(len(R_peaks))
    RR      = np.diff(R_peaks)/fs

    for i, r in enumerate(R_peaks):
        # pick a window length based on local RR
        if   i==0:  win = default_window_len
        elif i==len(R_peaks)-1:  win = default_window_len
        else:
            local = 0.5*(RR[i-1]+RR[i])
            win = 0.15*local
        # clip to physiological extremes
        win = np.clip(win, min_win, max_win)

        hw   = int(win*fs)
        start, end = max(r-hw,0), min(r+hw, len(ecg_qrs))
        segment = ecg_qrs[start:end]

        if len(segment)<2:
            QRS_amp[i] = np.nan
            continue

        # compute the analytic envelope
        env = np.abs(hilbert(segment))
        # baseline correct by subtracting the median envelope
        env = env - np.median(env)

        # amplitude = peak envelope
        QRS_amp[i] = np.max(env)

    # now interpolate/resample exactly as before
    t_new, raw, filt = interpolate_from_R_peaks(
        R_peaks, QRS_amp, fs,
        resampled_fs=10, interp_method="linear", filter_type="band")

    return QRS_amp, (t_new, raw, filt)



def get_combined_signal_from_features(
    features,
    resampled_fs=10,
    freq_band=(0.1, 0.5),
    weighting="equal",
    custom_weights=None
):
    """
    Create an ECG-derived respiration (EDR) signal by combining QRS amplitude and RRI signals,
    using the outputs from extract_ecg_features.
    
    Parameters
    ----------
    features : dict
        Dictionary returned by extract_ecg_features.
    resampled_fs : float, optional
        The target sampling frequency for resampled features (default=10).
    freq_band : tuple, optional
        The bandpass frequency range (in Hz) for final filtering (default=(0.1, 0.5)).
    weighting : str, optional
        Method to combine the normalized signals. Options:
          - "equal":  simple 50/50 average
          - "custom": use user-provided custom_weights (e.g., (0.7, 0.3))
          - "auto":   (example: inverse variance weighting)
    custom_weights : tuple or list, optional
        If weighting == "custom", pass a tuple like (w_qrs, w_rri) that sums to 1.
        
    Returns
    -------
    t_common : np.array
        The common time axis for the EDR signal.
    edr_filtered : np.array
        The combined and filtered EDR signal.
    """
    from scipy.signal import butter, filtfilt
    import numpy as np

    # Extract the resampled QRS amplitude and RRI signals from the features dict.
    # They are expected to be tuples: (time_array, raw_signal, filtered_signal)
    t_qrs, qrs_resampled, _ = features["QRS_amp_resampled"]
    t_rri, rri_resampled, _ = features["RRI_resampled"]
    
    # Check if we have enough data:
    if len(t_qrs) < 2 or len(t_rri) < 2:
        return np.array([]), np.array([])
    
    # Determine the overlapping time range:
    t_start = max(t_qrs[0], t_rri[0])
    t_end = min(t_qrs[-1], t_rri[-1])
    if t_start >= t_end:
        return np.array([]), np.array([])
    
    # Define a common time axis:
    t_common = np.arange(t_start, t_end, 1.0 / resampled_fs)
    
    # Interpolate both signals onto the common axis:
    qrs_common = np.interp(t_common, t_qrs, qrs_resampled)
    rri_common = np.interp(t_common, t_rri, rri_resampled)
    
    # Normalize each signal:
    norm_qrs = (qrs_common - np.mean(qrs_common)) / np.std(qrs_common)
    norm_rri = (rri_common - np.mean(rri_common)) / np.std(rri_common)
    
    # Combine the normalized signals:
    if weighting == "equal":
        edr_raw = 0.5 * (norm_qrs + norm_rri)
    elif weighting == "custom":
        if not custom_weights or len(custom_weights) != 2:
            raise ValueError("custom_weights must be a tuple of length 2, e.g. (0.7, 0.3).")
        w_qrs, w_rri = custom_weights
        edr_raw = w_qrs * norm_qrs + w_rri * norm_rri
    elif weighting == "auto":
        # Example: use inverse variance weighting (you can improve this)
        var_qrs = np.var(norm_qrs)
        var_rri = np.var(norm_rri)
        total = var_qrs + var_rri
        w_qrs = var_rri / total
        w_rri = var_qrs / total
        edr_raw = w_qrs * norm_qrs + w_rri * norm_rri
    else:
        raise ValueError("Invalid weighting mode. Use 'equal', 'custom', or 'auto'.")
    
    # Apply a bandpass filter if freq_band is provided:
    if freq_band is not None and len(freq_band) == 2:
        low, high = freq_band
        nyquist = 0.5 * resampled_fs
        low_cut = low / nyquist
        high_cut = high / nyquist
        b, a = butter(N=2, Wn=[low_cut, high_cut], btype='band')
        edr_filtered = filtfilt(b, a, edr_raw)
    else:
        # Fallback to a low-pass filter
        cutoff = 0.6  # Hz
        nyquist = 0.5 * resampled_fs
        b, a = butter(N=2, Wn=cutoff/nyquist, btype='low')
        edr_filtered = filtfilt(b, a, edr_raw)
    
    return t_common, edr_filtered

def get_combined_signal(
    resampled_fs=10,
    freq_band=(0.1, 0.5),
    weighting="equal",
    custom_weights=None
):
    """
    Create an ECG-derived respiration (EDR) signal by combining QRS amplitude and RRI signals,
    using the outputs from extract_ecg_features.
    
    Parameters
    ----------
    features : dict
        Dictionary returned by extract_ecg_features.
    resampled_fs : float, optional
        The target sampling frequency for resampled features (default=10).
    freq_band : tuple, optional
        The bandpass frequency range (in Hz) for final filtering (default=(0.1, 0.5)).
    weighting : str, optional
        Method to combine the normalized signals. Options:
          - "equal":  simple 50/50 average
          - "custom": use user-provided custom_weights (e.g., (0.7, 0.3))
          - "auto":   (example: inverse variance weighting)
    custom_weights : tuple or list, optional
        If weighting == "custom", pass a tuple like (w_qrs, w_rri) that sums to 1.
        
    Returns
    -------
    t_common : np.array
        The common time axis for the EDR signal.
    edr_filtered : np.array
        The combined and filtered EDR signal.
    """
    from scipy.signal import butter, filtfilt
    import numpy as np

    # Extract the resampled QRS amplitude and RRI signals from the features dict.
    # They are expected to be tuples: (time_array, raw_signal, filtered_signal)
    t_qrs, qrs_resampled, _ = QRS_amplitude_resampled
    t_rri, rri_resampled, _ = RRI_resampled
    
    # Check if we have enough data:
    if len(t_qrs) < 2 or len(t_rri) < 2:
        return np.array([]), np.array([])
    
    
    t_start = max(t_qrs[0], t_rri[0])
    t_end = min(t_qrs[-1], t_rri[-1])
    if t_start >= t_end:
        return np.array([]), np.array([])
    
    # Define a common time axis:
    t_common = np.arange(t_start, t_end, 1.0 / resampled_fs)
    
    
    qrs_common = np.interp(t_common, t_qrs, qrs_resampled)
    rri_common = np.interp(t_common, t_rri, rri_resampled)
    
   
    norm_qrs = (qrs_common - np.mean(qrs_common)) / np.std(qrs_common)
    norm_rri = (rri_common - np.mean(rri_common)) / np.std(rri_common)
    
    
    if weighting == "equal":
        edr_raw = 0.5 * (norm_qrs + norm_rri)
    elif weighting == "custom":
        if not custom_weights or len(custom_weights) != 2:
            raise ValueError("custom_weights must be a tuple of length 2, e.g. (0.7, 0.3).")
        w_qrs, w_rri = custom_weights
        edr_raw = w_qrs * norm_qrs + w_rri * norm_rri
    elif weighting == "auto":
        
        var_qrs = np.var(norm_qrs)
        var_rri = np.var(norm_rri)
        total = var_qrs + var_rri
        w_qrs = var_rri / total
        w_rri = var_qrs / total
        edr_raw = w_qrs * norm_qrs + w_rri * norm_rri
    else:
        raise ValueError("Invalid weighting mode. Use 'equal', 'custom', or 'auto'.")
    
    
    if freq_band is not None and len(freq_band) == 2:
        low, high = freq_band
        nyquist = 0.5 * resampled_fs
        low_cut = low / nyquist
        high_cut = high / nyquist
        b, a = butter(N=2, Wn=[low_cut, high_cut], btype='band')
        edr_filtered = filtfilt(b, a, edr_raw)
    else:
        # Fallback to a low-pass filter
        cutoff = 0.6  # Hz
        nyquist = 0.5 * resampled_fs
        b, a = butter(N=2, Wn=cutoff/nyquist, btype='low')
        edr_filtered = filtfilt(b, a, edr_raw)
    
    return t_common, edr_filtered





def get_QRS_effects(ecg, R_peaks, fs, search_window_len=0.06):
    BW_effect = np.zeros(len(R_peaks))
    AM_effect = np.zeros(len(R_peaks))
    half_win = int(search_window_len * fs)
    for i in range(len(R_peaks)):
        start = R_peaks[i] - half_win
        end = R_peaks[i] + half_win
        if start < 0 or end >= len(ecg):
            continue
        window = ecg[start:end]
        BW_effect[i] = np.mean([window.max(), window.min()])
        AM_effect[i] = window.max() - window.min()
    return BW_effect, AM_effect

def get_R_peak_amplitude(ecg, R_peaks):
    return ecg[R_peaks] if len(R_peaks) > 0 else np.array([])

def get_EDR_from_ecg(ecg_signal, R_peaks, fs, method='R_peak'):
    """
    If method=='R_peak', EDR is just the amplitude at the R-peaks.
    Otherwise, measure min/max in a small window around R-peaks.
    """
    import numpy as np
    from scipy.signal import butter, filtfilt

    if len(R_peaks) == 0:
        return np.array([]), np.array([])

    if method == 'R_peak':
        edr_raw = ecg_signal[R_peaks]  
    else:
        search_window_len = 0.01
        edr_raw = np.zeros(len(R_peaks))
        for i, peak in enumerate(R_peaks):
            start_idx = peak - int(search_window_len * fs)
            end_idx   = peak + int(search_window_len * fs)
            if start_idx < 0 or end_idx >= len(ecg_signal):
                continue
            window = ecg_signal[start_idx:end_idx]
            edr_raw[i] = window.max() - window.min()

    rpeak_times = R_peaks / fs
    if len(rpeak_times) < 2:
        return (np.array([]), np.array([]))

    from scipy.interpolate import interp1d
    interp_func = interp1d(rpeak_times, edr_raw, kind='linear', fill_value='extrapolate')
    resampled_fs = 10
    new_time = np.arange(rpeak_times[0], rpeak_times[-1], 1/resampled_fs)
    if len(new_time) < 2:
        return (np.array([]), np.array([]))

    edr_interpolated = interp_func(new_time)
    b, a = butter(2, 0.5, btype='low', fs=resampled_fs)
    edr_smooth = filtfilt(b, a, edr_interpolated)
    return new_time, edr_smooth

def get_EDR_from_RRI(R_peaks, fs, resampled_fs=10, lp_cutoff=1.0):
    RR_intervals = np.diff(R_peaks) / fs
    if len(RR_intervals) < 2:
        return np.array([]), np.array([])
    rr_times = R_peaks[1:] / fs
    from scipy.interpolate import interp1d
    rr_interp = interp1d(rr_times, RR_intervals, kind='linear', fill_value="extrapolate")

    t_min, t_max = rr_times[0], rr_times[-1]
    new_time = np.arange(t_min, t_max, 1/resampled_fs)
    if len(new_time) < 2:
        return (np.array([]), np.array([]))

    edr_raw = rr_interp(new_time)
    from scipy.signal import butter, filtfilt
    b, a = butter(2, lp_cutoff/(resampled_fs/2), btype='low')
    edr_smooth = filtfilt(b, a, edr_raw)
    return new_time, edr_smooth

def get_RRI_from_signal(signal, fs, RRI_fs=10):
  
    R_peaks = pan_detect(signal, fs)
    RRI = np.diff(R_peaks) / fs  
    

    RRI_resampled = interpolate_from_R_peaks(R_peaks[:-1], RRI, fs, RRI_fs, interp_method="linear", filter_type="low")

    return R_peaks, RRI, RRI_resampled








def detect_r_peaks_with_method(ecg_signal, fs, method="pan_tompkins"):
   
    if method == "pan_tompkins":
        from pan_tompkins import detect_ecg_peaks
        return detect_ecg_peaks(ecg_signal, fs)
    elif method == "engzee":
        from engzee_tompkins import detect_ecg_peaks_engzee
        return detect_ecg_peaks_engzee(ecg_signal, fs)
    elif method == "neurokit":
        from new_rpeak_detector import detect_r_peaks
        return detect_r_peaks(ecg_signal, fs)
    else:
        raise ValueError("method must be 'pan_tompkins', 'engzee', or 'neurokit'.")


def extract_ecg_features(
    ecg_signal,
    fs,
    method="pan_tompkins",
    RRI_fs=10,
    default_window_len=0.1,
    search_window_len=0.01,
    edr_method='R_peak'
):
    
    filtered_ecg = preprocess_signal(ecg_signal, fs)

    
    R_peaks = detect_r_peaks_with_method(filtered_ecg, fs, method=method)

    
    RRI = np.diff(R_peaks) / fs
    RRI_resampled = interpolate_from_R_peaks(
        R_peaks[:-1], RRI, fs,
        resampled_fs=RRI_fs,
        interp_method="linear",
        filter_type="low"
    )

    
    QRS_amp, QRS_amp_resampled = get_QRS_amplitude(
    ecg=filtered_ecg,
    R_peaks=R_peaks,
    fs=fs,
    default_window_len=0.1,
    min_win=0.04,
    max_win=0.2
    )
# then when you interpolate, use resampled_fs=20


    
    BW_effect, AM_effect = get_QRS_effects(filtered_ecg, R_peaks, fs, search_window_len=search_window_len)

    
    R_peak_amp = get_R_peak_amplitude(filtered_ecg, R_peaks)
    R_peak_amp_resampled = interpolate_from_R_peaks(
        R_peaks, R_peak_amp, fs,
        resampled_fs=RRI_fs,
        interp_method="linear",
        filter_type="low"
    )

   
    edr_time_r_peak, edr_signal_r_peak = get_EDR_from_ecg(
        ecg_signal=filtered_ecg,
        R_peaks=R_peaks,
        fs=fs,
        method=edr_method
    )

    
    edr_rri_time, edr_rri_signal = get_EDR_from_RRI(R_peaks, fs, resampled_fs=RRI_fs, lp_cutoff=0.7)

    combined_time, combined_signal = get_combined_signal_from_features(
        features={
            "QRS_amp_resampled": QRS_amp_resampled,
            "RRI_resampled": RRI_resampled
        },
        resampled_fs=RRI_fs,           # you can adjust this if needed
        freq_band=(0.1, 0.5),           # adjust the frequency band as desired
        weighting="equal"             # or "custom" / "auto" per your requirements
    )

    return {
        "filtered_ecg": filtered_ecg,
        "R_peaks": R_peaks,
        "RRI": RRI,
        "RRI_resampled": RRI_resampled,
        "QRS_amp": QRS_amp,
        "QRS_amp_resampled": QRS_amp_resampled,
        "BW_effect": BW_effect,
        "AM_effect": AM_effect,
        "R_peak_amplitude": R_peak_amp,
        "R_peak_amplitude_resampled": R_peak_amp_resampled,
        "edr_time_r_peak": edr_time_r_peak,
        "edr_signal_r_peak": edr_signal_r_peak,
        "edr_rri_time": edr_rri_time,
        "edr_rri_signal": edr_rri_signal,
        # New keys for the combined signal:
        "combined_time": combined_time,
        "combined_signal": combined_signal
    }





# featres_3 with combined

In [ ]:
# features.py

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch
from scipy.interpolate import interp1d
import sys
import os

project_root = "C:/Users/Visnu/DIAMONDS"  
if project_root not in sys.path:
    sys.path.append(project_root)

import diamonds.data as dt

from pan_tompkins import detect_ecg_peaks
# We import define_complete_breath_cycles to get total breath duration from ex->ex
import importlib
import breath_detection
importlib.reload(breath_detection)
from breath_detection import (
    auto_detect_edr_breaths,
    define_breaths_insp_exp,
    define_complete_breath_cycles,
    compute_true_insp_exp_durations,
    DEFAULT_MIN_BREATH_SEC
)
from diamonds_definitions import pt, session, exercise, ecg_signal, fs

import importlib
import preprocessing
importlib.reload(preprocessing)
import pan_tompkins
import engzee_tompkins
import new_rpeak_detector
importlib.reload(pan_tompkins)
importlib.reload(engzee_tompkins)
importlib.reload(new_rpeak_detector)

record = pt


method_choice = "pan_tompkins"   # or "engzee" 


results = preprocessing.extract_ecg_features(
    ecg_signal,
    fs,
    method=method_choice,
    RRI_fs=10,
    default_window_len=0.1,
    search_window_len=0.01,
    edr_method='R_peak'
)

# The dictionary 'results' now has everything:
filtered_ecg = results["filtered_ecg"]
R_peaks       = results["R_peaks"]
RRI           = results["RRI"]
RRI_resampled = results["RRI_resampled"]
QRS_amplitude = results["QRS_amp"]
QRS_amplitude_resampled = results["QRS_amp_resampled"]
BW_effect     = results["BW_effect"]
AM_effect     = results["AM_effect"]
R_peak_amplitude             = results["R_peak_amplitude"]
R_peak_amplitude_resampled   = results["R_peak_amplitude_resampled"]
edr_time_r_peak              = results["edr_time_r_peak"]
edr_signal_r_peak            = results["edr_signal_r_peak"]
edr_rri_time                 = results["edr_rri_time"]
edr_rri_signal               = results["edr_rri_signal"]
combined_time = results["combined_time"] 
combined_signal = results["combined_signal"]

print(f"Method used: {method_choice}")
print(f"Detected {len(R_peaks)} R-peaks")




def compute_breath_count(in_times, t_start, t_end):

    valid_in = in_times[(in_times >= t_start) & (in_times <= t_end)]
    return len(valid_in)



def compute_peak_to_trough(edr_time, edr_signal, t_start, t_end):
    mask = (edr_time >= t_start) & (edr_time < t_end)
    segment = edr_signal[mask]
    if segment.size == 0:
        return np.nan
    return np.max(segment) - np.min(segment)


def plot_breath_cycles(edr_time, edr_signal, in_times, ex_times):
    from breath_detection import define_breaths_insp_exp
    breath_starts, breath_ends = define_breaths_insp_exp(in_times, ex_times)
    
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal')
    
    plt.plot(breath_starts,
             np.interp(breath_starts, edr_time, edr_signal),
             'go', label='Breath Start')
    plt.plot(breath_ends,
             np.interp(breath_ends, edr_time, edr_signal),
             'ro', label='Breath End')

    for t in breath_starts:
        plt.axvline(t, color='green', linestyle='--', alpha=0.5)
    for t in breath_ends:
        plt.axvline(t, color='red', linestyle='--', alpha=0.5)
    
    plt.title('Breath Cycles (Inspiration to Expiration)')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_edr_breaths(edr_time, edr_signal, in_times, ex_times):
    plt.figure(figsize=(10, 4))
    plt.plot(edr_time, edr_signal, label='EDR Signal (QRS Amplitude)')
    plt.plot(in_times,
             np.interp(in_times, edr_time, edr_signal),
             'o', label='Inspirations')
    plt.plot(ex_times,
             np.interp(ex_times, edr_time, edr_signal),
             'o', label='Expirations')
    plt.title('ECG-Derived Respiration with Detected Breaths')
    plt.xlabel('Time (s)')
    plt.ylabel('EDR Amplitude')
    plt.grid(True)
    plt.legend()
    plt.show()


def build_feature_table(
    patient,
    session,
    ecg_signal,
    fs,
    edr_time, edr_signal,
    interval_size: float = 10,
    step_len: float = 5,
):
    import numpy as np
    import pandas as pd
    from breath_detection import (
        auto_detect_edr_breaths,
        define_complete_breath_cycles,
        compute_true_insp_exp_durations,
    )
    import diamonds.data as dt

    # 1) grab global exercise bounds
    exercises         = session[dt.Exercise]
    all_ex_timestamps = np.array(exercises.timestamp, dtype=float)
    full_start        = float(all_ex_timestamps.min())
    full_end          = float(all_ex_timestamps.max())
    print(f"build_feature_table: Using exercise interval {full_start:.1f}–{full_end:.1f} s")

    # 2) prepare your EDR arrays
    edr_time_qrs   = edr_time
    edr_signal_qrs = edr_signal

    # 3) detect breaths once on the whole signal
    in_times, ex_times = auto_detect_edr_breaths(
        edr_time_qrs,
        edr_signal_qrs,
        init_min_breath_sec=1.5,
        adjust_factor=0.5,
        max_iterations=5,
        tol=0.2,
        min_allowed=0.5,
        max_allowed=10.0,
        adaptive=True
    )

    # 4) build sliding‑window grid
    seg_starts = np.arange(
        full_start,
        full_end - interval_size + 1e-9,
        step_len
    )

    feature_rows = []
    for window_start in seg_starts:
        window_end = window_start + interval_size

        # a) true insp/exp durations
        true_insp, true_exp = compute_true_insp_exp_durations(
            in_times, ex_times, window_start, window_end
        )
        ie_ratio = (
            true_insp / true_exp
            if not (np.isnan(true_insp) or np.isnan(true_exp)) and true_exp > 0
            else np.nan
        )

        # b) breath count & total breath duration (Ex→In→Ex cycles)
        local_ex = ex_times[(ex_times >= window_start) & (ex_times < window_end)]
        local_in = in_times[(in_times >= window_start) & (in_times < window_end)]
        b_starts, b_ends, _, _ = define_complete_breath_cycles(local_in, local_ex)
        n_breaths = len(b_starts)

        if n_breaths > 0:
            total_durations = b_ends - b_starts
            avg_total_breath_duration = np.mean(total_durations)
        else:
            avg_total_breath_duration = np.nan

        # c) EDR‑derived BPM from inspirations
        valid_ins = in_times[(in_times >= window_start) & (in_times < window_end)]
        if valid_ins.size >= 2:
            edr_bpm = 60.0 / np.mean(np.diff(valid_ins))
        else:
            edr_bpm = np.nan

        feature_rows.append({
            "subject_id":             patient.id,
            "window_start":           window_start,
            "window_end":             window_end,
            "EDR_BPM":                edr_bpm,
            "n_breaths":              n_breaths,
            "AvgTotalBreathDuration": avg_total_breath_duration,
            "TrueInspDuration":       true_insp,
            "TrueExpDuration":        true_exp,
            "IEratio":                ie_ratio
        })

    # 5) assemble, sort, interpolate and zero‑fill
    feature_df = pd.DataFrame(feature_rows)
    feature_df = feature_df.sort_values("window_start").reset_index(drop=True)

    # interpolate all numeric columns
    num_cols = feature_df.select_dtypes(include=[np.number]).columns
    feature_df[num_cols] = (
        feature_df[num_cols]
        .interpolate(method="linear", limit_direction="both", axis=0)
    )

    # any NaN breath‑counts → 0
    feature_df["n_breaths"] = feature_df["n_breaths"].fillna(0).astype(int)

    return feature_df





def main():
    
    edr_time = combined_time
    edr_signal = combined_signal

    in_times, ex_times = auto_detect_edr_breaths(
        edr_time,
        edr_signal
    )
    
    plot_breath_cycles(edr_time, edr_signal, in_times, ex_times)
    plot_edr_breaths(edr_time, edr_signal, in_times, ex_times)

    session = pt[dt.SeatedSession] 
    exercise_ann = session[dt.Exercise]

    interval_size = 10  
    feature_df = build_feature_table(
        record, session, ecg_signal, fs,
        edr_time, edr_signal, interval_size=interval_size, step_len=5
    )
    print("Feature Table:")
    print(feature_df)

   
    labels = [
        f"{int(ws)}–{int(we)}s"
        for ws, we in zip(feature_df["window_start"], feature_df["window_end"])
    ]
   
    mid_times = feature_df['window_start'] + (interval_size / 2)

  
    plt.figure()
    plt.plot(mid_times, feature_df['TrueInspDuration'], 'o-', label='True Insp Duration')
    plt.plot(mid_times, feature_df['TrueExpDuration'], 'o-', label='True Exp Duration')
    plt.title('True Inspiration and Expiration Durations')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

   
    plt.figure()
    plt.plot(mid_times, feature_df['n_breaths'], 'o-', label='Number of Breaths')
    plt.title('Number of Breaths (per 10s window)')
    plt.xlabel('Time (s)')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 3) Plot: AvgTotalBreathDuration
    plt.figure()
    plt.plot(mid_times, feature_df['AvgTotalBreathDuration'], 'o-', label='AvgTotalBreathDuration')
    plt.title('Average Total Breath Duration (Ex->Ex)')
    plt.xlabel('Time (s)')
    plt.ylabel('Duration (s)')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 4) Plot: Breathing Rate per Minute
    plt.figure()
    plt.plot(mid_times, feature_df['EDR_BPM'], 'o-', label='Breathing Rate (BPM)')
    plt.title('Breathing Rate per Minute')
    plt.xlabel('Time (s)')
    plt.ylabel('BPM')
    plt.grid(True)
    plt.legend()
    plt.show()

    # 5) Bar Plot: EDR_BPM
    plt.figure()
    plt.bar(labels, feature_df["EDR_BPM"], color="royalblue", alpha=0.7)
    plt.title("Breathing Rate per Minute (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("BPM")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()


    plt.figure()
    plt.bar(labels, feature_df["AvgTotalBreathDuration"], color="seagreen", alpha=0.7)
    plt.title("Average Total Breath Duration (Segmented Intervals)")
    plt.xlabel("Time Intervals")
    plt.ylabel("Duration (s)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()

